In [ ]:
# Sources
# https://pypi.org/project/chowtest/
# https://medium.com/@remycanario17/the-chow-test-dealing-with-heterogeneity-in-python-1b9057f0f07a
#  https://www.jstor.org/stable/2526374

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statistics
from statsmodels.formula.api import ols
from scipy import stats
from sklearn.impute import SimpleImputer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Junior/Dyanne JP/ABCD_Release4.0_Tabular_Dataset.csv')
df_baseline = df[df['eventname'] == 'baseline_year_1_arm_1']
df_2year = df[df['eventname'] == '2_year_follow_up_y_arm_1']

In [ ]:
baseline_2year = pd.merge(df_baseline, df_2year, on='subjectkey', how='left', suffixes=('_baseline', '_2year'))

In [ ]:
baseline_2year.dropna(subset=['eventname_2year'], inplace=True)

In [ ]:
check = ['nihtbx_picvocab_uncorrected_baseline','nihtbx_picvocab_uncorrected_2year','nihtbx_flanker_uncorrected_baseline',
         'nihtbx_flanker_uncorrected_2year','nihtbx_pattern_uncorrected_baseline','nihtbx_pattern_uncorrected_2year',
         'nihtbx_picture_uncorrected_baseline','nihtbx_picture_uncorrected_2year','nihtbx_reading_uncorrected_baseline',
         'nihtbx_reading_uncorrected_2year','nihtbx_cryst_uncorrected_baseline','nihtbx_cryst_uncorrected_2year']

cleaned = baseline_2year.dropna(subset=check)
#(7172,1055)

# drop columns that only have NaN values
cleaned.dropna(axis=1, how='all', inplace=True)
#(7172,892)

print('Number of participants with all 5 test scores')
print(cleaned.shape[0])

In [ ]:
non_numeric_columns = cleaned.select_dtypes(exclude=['number']).columns

print("Non-numeric columns:", non_numeric_columns.tolist())

In [ ]:
cleaned = cleaned.drop('sex_2year',axis=1)

In [ ]:
cleaned['sex_baseline'] = cleaned['sex_baseline'].replace({'M': 0, 'F': 1})

In [ ]:
def medianimpute(df):
    # Create a copy of the DataFrame to avoid changing the original data
    df_imputed = df.copy()

    # Identify numeric columns by data type
    numeric_cols = df_imputed.select_dtypes(include=[np.number]).columns

    # Define the imputer with a median strategy
    imputer = SimpleImputer(strategy='median')

    # Apply the imputer only to the numeric columns
    df_imputed[numeric_cols] = imputer.fit_transform(df_imputed[numeric_cols])

    return df_imputed

In [ ]:
imputed = medianimpute(cleaned)

In [ ]:
df_low = imputed[(imputed['income_baseline'] >= 1) & (imputed['income_baseline'] < 7)]
df_med = imputed[imputed['income_baseline'].isin([7])]
df_high = imputed[(imputed['income_baseline'] > 7) & (imputed['income_baseline'] <= 10)]

In [ ]:
low_diff = pd.DataFrame({
    'low_diff_picvocab': df_low['nihtbx_picvocab_uncorrected_2year'] - df_low['nihtbx_picvocab_uncorrected_baseline'],
    'low_diff_flanker': df_low['nihtbx_flanker_uncorrected_2year'] - df_low['nihtbx_flanker_uncorrected_baseline'],
    'low_diff_pattern': df_low['nihtbx_pattern_uncorrected_2year'] - df_low['nihtbx_pattern_uncorrected_baseline'],
    'low_diff_picture': df_low['nihtbx_picture_uncorrected_2year'] - df_low['nihtbx_picture_uncorrected_baseline'],
    'low_diff_reading': df_low['nihtbx_reading_uncorrected_2year'] - df_low['nihtbx_reading_uncorrected_baseline']
})

high_diff = pd.DataFrame({
    'high_diff_picvocab': df_high['nihtbx_picvocab_uncorrected_2year'] - df_high['nihtbx_picvocab_uncorrected_baseline'],
    'high_diff_flanker': df_high['nihtbx_flanker_uncorrected_2year'] - df_high['nihtbx_flanker_uncorrected_baseline'],
    'high_diff_pattern': df_high['nihtbx_pattern_uncorrected_2year'] - df_high['nihtbx_pattern_uncorrected_baseline'],
    'high_diff_picture': df_high['nihtbx_picture_uncorrected_2year'] - df_high['nihtbx_picture_uncorrected_baseline'],
    'high_diff_reading': df_high['nihtbx_reading_uncorrected_2year'] - df_high['nihtbx_reading_uncorrected_baseline']
})

df_low = pd.concat([df_low, low_diff], axis=1)
df_high = pd.concat([df_high, high_diff], axis=1)

In [ ]:
low_picvocab_neg = df_low[df_low['low_diff_picvocab']<0]
high_picvocab_neg = df_high[df_high['high_diff_picvocab']<0]
low_picvocab_pos = df_low[df_low['low_diff_picvocab']>0]
high_picvocab_pos = df_high[df_high['high_diff_picvocab']>0]

low_flanker_neg = df_low[df_low['low_diff_flanker']<0]
high_flanker_neg = df_high[df_high['high_diff_flanker']<0]
low_flanker_pos = df_low[df_low['low_diff_flanker']>0]
high_flanker_pos = df_high[df_high['high_diff_flanker']>0]

low_pattern_neg = df_low[df_low['low_diff_pattern']<0]
high_pattern_neg = df_high[df_high['high_diff_pattern']<0]
low_pattern_pos = df_low[df_low['low_diff_pattern']>0]
high_pattern_pos = df_high[df_high['high_diff_pattern']>0]

low_picture_neg = df_low[df_low['low_diff_picture']<0]
high_picture_neg = df_high[df_high['high_diff_picture']<0]
low_picture_pos = df_low[df_low['low_diff_picture']>0]
high_picture_pos = df_high[df_high['high_diff_picture']>0]

low_reading_neg = df_low[df_low['low_diff_reading']<0]
high_reading_neg = df_high[df_high['high_diff_reading']<0]
low_reading_pos = df_low[df_low['low_diff_reading']>0]
high_reading_pos = df_high[df_high['high_diff_reading']>0]

In [ ]:
def standardize(df):
    # Select numeric columns only.
    numeric_columns = df.select_dtypes(include=[np.number])
    # Subtract the mean and divide by the std dev for each column.
    standardized_df = (numeric_columns - numeric_columns.mean()) / numeric_columns.std()
    # If you want to keep non-numeric columns unchanged, combine back the non-numeric columns:
    non_numeric_columns = df.select_dtypes(exclude=[np.number])
    return pd.concat([standardized_df, non_numeric_columns], axis=1)

In [ ]:
low_picvocab_neg = standardize(low_picvocab_neg)
high_picvocab_neg = standardize(high_picvocab_neg)
low_picvocab_pos = standardize(low_picvocab_pos)
high_picvocab_pos = standardize(high_picvocab_pos)

low_flanker_neg = standardize(low_flanker_neg)
high_flanker_neg = standardize(high_flanker_neg)
low_flanker_pos = standardize(low_flanker_pos)
high_flanker_pos = standardize(high_flanker_pos)

low_pattern_neg = standardize(low_pattern_neg)
high_pattern_neg = standardize(high_pattern_neg)
low_pattern_pos = standardize(low_pattern_pos)
high_pattern_pos = standardize(high_pattern_pos)

low_picture_neg = standardize(low_picture_neg)
high_picture_neg = standardize(high_picture_neg)
low_picture_pos = standardize(low_picture_pos)
high_picture_pos = standardize(high_picture_pos)

low_reading_neg = standardize(low_reading_neg)
high_reading_neg = standardize(high_reading_neg)
low_reading_pos = standardize(low_reading_pos)
high_reading_pos = standardize(high_reading_pos)

In [ ]:
# nan_columns_low = low_picvocab_pos.columns[low_picvocab_neg.isna().any()].tolist()
# print(nan_columns_low)
# nan_columns_high = low_picvocab_pos.columns[high_picvocab_neg.isna().any()].tolist()
# print(nan_columns_high)
# remove prenatal_heroin_before_baseline, prenatal_heroin_after_baseline, prenatal_cocaine_after_baseline, prenatal_oxycontin_after_baseline, KSADS_Subst_p_baseline

In [ ]:
model_low = ols('nihtbx_picvocab_uncorrected_2year ~ KSADS_BD_y_baseline + KSADS_Anx_y_baseline + KSADS_Suicide_y_baseline + KSADS_Sleep_y_baseline + KSADS_total_y_baseline + KSADS_BD_y_2year + KSADS_Anx_y_2year + KSADS_Eat_y_2year + KSADS_Suicide_y_2year + KSADS_Sleep_y_2year + KSADS_total_y_2year + KSADS_BD_p_baseline + KSADS_Anx_p_baseline + KSADS_Eat_p_baseline + KSADS_Suicide_p_baseline + KSADS_Sleep_p_baseline + KSADS_total_p_baseline + KSADS_BD_p_2year + KSADS_Anx_p_2year + KSADS_Eat_p_2year + KSADS_Suicide_p_2year + KSADS_Sleep_p_2year + KSADS_total_p_2year + KSADS_Subst_p_2year + cbcl_internal_baseline + cbcl_internal_2year + cbcl_external_baseline + cbcl_external_2year + cbcl_totprob_baseline + cbcl_totprob_2year + upps_premeditation_baseline + upps_premeditation_2year + upps_perseverence_baseline + upps_perseverence_2year + upps_sensation_baseline + upps_sensation_2year + upps_negative_baseline + upps_negative_2year + upps_positive_baseline + upps_positive_2year + BIS_baseline + BIS_2year + BAS_RR_baseline + BAS_RR_2year + BAS_Drive_baseline + BAS_Drive_2year + BAS_Fun_baseline + BAS_Fun_2year + sleep_disturb_total_baseline + sleep_disturb_total_2year + ELS_total_baseline + ELS_total_2year + rh_adi_wsum1_baseline + rh_adi_wsum1_2year + rh_adi_wsum2_baseline + rh_adi_wsum2_2year + risk_alcohol_p_baseline + risk_cigarette_p_baseline + risk_electro_nicotine_p_baseline + risk_marijuana_p_baseline + risk_drug_p_baseline + risk_med_legal_p_baseline + risk_med_marijuana_p_baseline + risk_med_marijuana_pre_p_baseline + risk_family_med_marijuana_p_baseline + risk_alcohol_p_2year + risk_cigarette_p_2year + risk_electro_nicotine_p_2year + risk_marijuana_p_2year + risk_drug_p_2year + risk_med_legal_p_2year + risk_med_marijuana_p_2year + risk_med_marijuana_pre_p_2year + risk_family_med_marijuana_p_2year + risk_alcohol_y_2year + risk_cigarette_y_2year + risk_electro_nicotine_y_2year + risk_marijuana_y_2year + risk_med_legal_y_2year + risk_med_marijuana_y_2year + risk_med_marijuana_pre_y_2year + risk_family_med_marijuana_y_2year + risk_drug_y_2year + risk_gas_y_2year + risk_pain_y_2year + risk_anix_y_2year + risk_stimulant_y_2year + good_school_baseline + good_parent1_baseline + good_parent2_baseline + good_parent_baseline + good_school_2year + prenatal_tobacco_before_baseline + prenatal_alcohol_max_before_baseline + prenatal_alcohol_avg_before_baseline + prenatal_alcohol_eff_before_baseline + prenatal_marijuana_before_baseline + prenatal_cocaine_before_baseline + prenatal_oxycontin_before_baseline + prenatal_tobacco_after_baseline + prenatal_alcohol_max_after_baseline + prenatal_alcohol_avg_after_baseline + prenatal_alcohol_eff_after_baseline + prenatal_marijuana_after_baseline + prenatal_weeks_baseline + screentime_wkday_tv_baseline + screentime_wkday_videos_baseline + screentime_wkday_games_baseline + screentime_wkday_texting_baseline + screentime_wkday_sns_baseline + screentime_wkday_videochat_baseline + screentime_wkend_tv_baseline + screentime_wkend_videos_baseline + screentime_wkend_games_baseline + screentime_wkend_texting_baseline + screentime_wkend_sns_baseline + screentime_wkend_videochat_baseline + screentime_wkday_y_baseline + screentime_wkend_y_baseline + screentime_wkday_p_baseline + screentime_wkend_p_baseline + screentime_maturegames_baseline + screentime_rmovies_baseline + screentime_addict_p_2year + screentime_risk_p_2year + screentime_maturegames_2year + screentime_rmovies_2year + bpm_total_y_2year + bpm_total_t_baseline + bpm_total_t_2year + prosocial_y_baseline + prosocial_p_baseline + detention_baseline + detention_rea_baseline + friends_boys_baseline + friends_girls_baseline + Cfriends_boys_baseline + Cfriends_girls_baseline + friends_same_baseline + friends_diff_baseline + Cfriends_diff_baseline + Cfriends_same_baseline + prosocial_y_2year + prosocial_p_2year + friends_boys_2year + friends_girls_2year + Cfriends_boys_2year + Cfriends_girls_2year + friends_same_2year + friends_diff_2year + Cfriends_diff_2year + Cfriends_same_2year + cpeur2_baseline + eaeur1_baseline + depeur4_baseline + mddeur6_baseline + depmulti_baseline + bmieur4_baseline + bmimulti_baseline + insomniaeur6_baseline + snoringeur1_baseline + iqeur2_baseline + happieur4_baseline + ghappieur2_baseline + ghappimeaneur1_baseline + ghappihealth6_baseline + alcdep_eurauto_baseline + alcdep_afrauto_baseline + alcdep_metaauto_baseline + asdauto_baseline + aspauto_baseline + bipauto_baseline + cannabisauto_baseline + crossauto_baseline + drinkauto_baseline + edauto_baseline + neuroticismauto_baseline + ocdauto_baseline + risk4pcauto_baseline + risktolauto_baseline + scz_eurauto_baseline + scz_easauto_baseline + scz_metaauto_baseline + smokerauto_baseline + worryauto_baseline + anxietyauto_baseline + ptsdeur4_baseline + ptsdmeta6_baseline + adhdeur6_baseline + euro_baseline + sex_baseline + race_g_baseline + parent_identity_baseline + demo_brthdat_v2_baseline + demo_sex_v2_baseline + gender_identity_baseline + parent_age_baseline + foreign_born_family_baseline + married_baseline + high_educ_baseline + high_educ2_baseline + income_baseline + foreign_born_baseline + religion_prefer_baseline + gay_parent_baseline + gay_youth_baseline + race_ethnicity_baseline + age_baseline + family_adversity_baseline + height_baseline + weight_baseline + vol_baseline + bmi_baseline + total_ratio_baseline + history_ratio_baseline + euro_2year + race_g_2year + gay_youth_2year + age_2year + height_2year + weight_2year + vol_2year + bmi_2year', data=low_picvocab_neg).fit()
model_high = ols('nihtbx_picvocab_uncorrected_2year ~ KSADS_BD_y_baseline + KSADS_Anx_y_baseline + KSADS_Suicide_y_baseline + KSADS_Sleep_y_baseline + KSADS_total_y_baseline + KSADS_BD_y_2year + KSADS_Anx_y_2year + KSADS_Eat_y_2year + KSADS_Suicide_y_2year + KSADS_Sleep_y_2year + KSADS_total_y_2year + KSADS_BD_p_baseline + KSADS_Anx_p_baseline + KSADS_Eat_p_baseline + KSADS_Suicide_p_baseline + KSADS_Sleep_p_baseline + KSADS_total_p_baseline + KSADS_BD_p_2year + KSADS_Anx_p_2year + KSADS_Eat_p_2year + KSADS_Suicide_p_2year + KSADS_Sleep_p_2year + KSADS_total_p_2year + KSADS_Subst_p_2year + cbcl_internal_baseline + cbcl_internal_2year + cbcl_external_baseline + cbcl_external_2year + cbcl_totprob_baseline + cbcl_totprob_2year + upps_premeditation_baseline + upps_premeditation_2year + upps_perseverence_baseline + upps_perseverence_2year + upps_sensation_baseline + upps_sensation_2year + upps_negative_baseline + upps_negative_2year + upps_positive_baseline + upps_positive_2year + BIS_baseline + BIS_2year + BAS_RR_baseline + BAS_RR_2year + BAS_Drive_baseline + BAS_Drive_2year + BAS_Fun_baseline + BAS_Fun_2year + sleep_disturb_total_baseline + sleep_disturb_total_2year + ELS_total_baseline + ELS_total_2year + rh_adi_wsum1_baseline + rh_adi_wsum1_2year + rh_adi_wsum2_baseline + rh_adi_wsum2_2year + risk_alcohol_p_baseline + risk_cigarette_p_baseline + risk_electro_nicotine_p_baseline + risk_marijuana_p_baseline + risk_drug_p_baseline + risk_med_legal_p_baseline + risk_med_marijuana_p_baseline + risk_med_marijuana_pre_p_baseline + risk_family_med_marijuana_p_baseline + risk_alcohol_p_2year + risk_cigarette_p_2year + risk_electro_nicotine_p_2year + risk_marijuana_p_2year + risk_drug_p_2year + risk_med_legal_p_2year + risk_med_marijuana_p_2year + risk_med_marijuana_pre_p_2year + risk_family_med_marijuana_p_2year + risk_alcohol_y_2year + risk_cigarette_y_2year + risk_electro_nicotine_y_2year + risk_marijuana_y_2year + risk_med_legal_y_2year + risk_med_marijuana_y_2year + risk_med_marijuana_pre_y_2year + risk_family_med_marijuana_y_2year + risk_drug_y_2year + risk_gas_y_2year + risk_pain_y_2year + risk_anix_y_2year + risk_stimulant_y_2year + good_school_baseline + good_parent1_baseline + good_parent2_baseline + good_parent_baseline + good_school_2year + prenatal_tobacco_before_baseline + prenatal_alcohol_max_before_baseline + prenatal_alcohol_avg_before_baseline + prenatal_alcohol_eff_before_baseline + prenatal_marijuana_before_baseline + prenatal_cocaine_before_baseline + prenatal_oxycontin_before_baseline + prenatal_tobacco_after_baseline + prenatal_alcohol_max_after_baseline + prenatal_alcohol_avg_after_baseline + prenatal_alcohol_eff_after_baseline + prenatal_marijuana_after_baseline + prenatal_weeks_baseline + screentime_wkday_tv_baseline + screentime_wkday_videos_baseline + screentime_wkday_games_baseline + screentime_wkday_texting_baseline + screentime_wkday_sns_baseline + screentime_wkday_videochat_baseline + screentime_wkend_tv_baseline + screentime_wkend_videos_baseline + screentime_wkend_games_baseline + screentime_wkend_texting_baseline + screentime_wkend_sns_baseline + screentime_wkend_videochat_baseline + screentime_wkday_y_baseline + screentime_wkend_y_baseline + screentime_wkday_p_baseline + screentime_wkend_p_baseline + screentime_maturegames_baseline + screentime_rmovies_baseline + screentime_addict_p_2year + screentime_risk_p_2year + screentime_maturegames_2year + screentime_rmovies_2year + bpm_total_y_2year + bpm_total_t_baseline + bpm_total_t_2year + prosocial_y_baseline + prosocial_p_baseline + detention_baseline + detention_rea_baseline + friends_boys_baseline + friends_girls_baseline + Cfriends_boys_baseline + Cfriends_girls_baseline + friends_same_baseline + friends_diff_baseline + Cfriends_diff_baseline + Cfriends_same_baseline + prosocial_y_2year + prosocial_p_2year + friends_boys_2year + friends_girls_2year + Cfriends_boys_2year + Cfriends_girls_2year + friends_same_2year + friends_diff_2year + Cfriends_diff_2year + Cfriends_same_2year + cpeur2_baseline + eaeur1_baseline + depeur4_baseline + mddeur6_baseline + depmulti_baseline + bmieur4_baseline + bmimulti_baseline + insomniaeur6_baseline + snoringeur1_baseline + iqeur2_baseline + happieur4_baseline + ghappieur2_baseline + ghappimeaneur1_baseline + ghappihealth6_baseline + alcdep_eurauto_baseline + alcdep_afrauto_baseline + alcdep_metaauto_baseline + asdauto_baseline + aspauto_baseline + bipauto_baseline + cannabisauto_baseline + crossauto_baseline + drinkauto_baseline + edauto_baseline + neuroticismauto_baseline + ocdauto_baseline + risk4pcauto_baseline + risktolauto_baseline + scz_eurauto_baseline + scz_easauto_baseline + scz_metaauto_baseline + smokerauto_baseline + worryauto_baseline + anxietyauto_baseline + ptsdeur4_baseline + ptsdmeta6_baseline + adhdeur6_baseline + euro_baseline + sex_baseline + race_g_baseline + parent_identity_baseline + demo_brthdat_v2_baseline + demo_sex_v2_baseline + gender_identity_baseline + parent_age_baseline + foreign_born_family_baseline + married_baseline + high_educ_baseline + high_educ2_baseline + income_baseline + foreign_born_baseline + religion_prefer_baseline + gay_parent_baseline + gay_youth_baseline + race_ethnicity_baseline + age_baseline + family_adversity_baseline + height_baseline + weight_baseline + vol_baseline + bmi_baseline + total_ratio_baseline + history_ratio_baseline + euro_2year + race_g_2year + gay_youth_2year + age_2year + height_2year + weight_2year + vol_2year + bmi_2year', data=high_picvocab_neg).fit()

# Combine data for Chow Test
combined_data = pd.concat([low_picvocab_neg, high_picvocab_neg])
model_combined = ols('nihtbx_picvocab_uncorrected_2year ~ KSADS_BD_y_baseline + KSADS_Anx_y_baseline + KSADS_Suicide_y_baseline + KSADS_Sleep_y_baseline + KSADS_total_y_baseline + KSADS_BD_y_2year + KSADS_Anx_y_2year + KSADS_Eat_y_2year + KSADS_Suicide_y_2year + KSADS_Sleep_y_2year + KSADS_total_y_2year + KSADS_BD_p_baseline + KSADS_Anx_p_baseline + KSADS_Eat_p_baseline + KSADS_Suicide_p_baseline + KSADS_Sleep_p_baseline + KSADS_total_p_baseline + KSADS_BD_p_2year + KSADS_Anx_p_2year + KSADS_Eat_p_2year + KSADS_Suicide_p_2year + KSADS_Sleep_p_2year + KSADS_total_p_2year + KSADS_Subst_p_2year + cbcl_internal_baseline + cbcl_internal_2year + cbcl_external_baseline + cbcl_external_2year + cbcl_totprob_baseline + cbcl_totprob_2year + upps_premeditation_baseline + upps_premeditation_2year + upps_perseverence_baseline + upps_perseverence_2year + upps_sensation_baseline + upps_sensation_2year + upps_negative_baseline + upps_negative_2year + upps_positive_baseline + upps_positive_2year + BIS_baseline + BIS_2year + BAS_RR_baseline + BAS_RR_2year + BAS_Drive_baseline + BAS_Drive_2year + BAS_Fun_baseline + BAS_Fun_2year + sleep_disturb_total_baseline + sleep_disturb_total_2year + ELS_total_baseline + ELS_total_2year + rh_adi_wsum1_baseline + rh_adi_wsum1_2year + rh_adi_wsum2_baseline + rh_adi_wsum2_2year + risk_alcohol_p_baseline + risk_cigarette_p_baseline + risk_electro_nicotine_p_baseline + risk_marijuana_p_baseline + risk_drug_p_baseline + risk_med_legal_p_baseline + risk_med_marijuana_p_baseline + risk_med_marijuana_pre_p_baseline + risk_family_med_marijuana_p_baseline + risk_alcohol_p_2year + risk_cigarette_p_2year + risk_electro_nicotine_p_2year + risk_marijuana_p_2year + risk_drug_p_2year + risk_med_legal_p_2year + risk_med_marijuana_p_2year + risk_med_marijuana_pre_p_2year + risk_family_med_marijuana_p_2year + risk_alcohol_y_2year + risk_cigarette_y_2year + risk_electro_nicotine_y_2year + risk_marijuana_y_2year + risk_med_legal_y_2year + risk_med_marijuana_y_2year + risk_med_marijuana_pre_y_2year + risk_family_med_marijuana_y_2year + risk_drug_y_2year + risk_gas_y_2year + risk_pain_y_2year + risk_anix_y_2year + risk_stimulant_y_2year + good_school_baseline + good_parent1_baseline + good_parent2_baseline + good_parent_baseline + good_school_2year + prenatal_tobacco_before_baseline + prenatal_alcohol_max_before_baseline + prenatal_alcohol_avg_before_baseline + prenatal_alcohol_eff_before_baseline + prenatal_marijuana_before_baseline + prenatal_cocaine_before_baseline + prenatal_oxycontin_before_baseline + prenatal_tobacco_after_baseline + prenatal_alcohol_max_after_baseline + prenatal_alcohol_avg_after_baseline + prenatal_alcohol_eff_after_baseline + prenatal_marijuana_after_baseline + prenatal_weeks_baseline + screentime_wkday_tv_baseline + screentime_wkday_videos_baseline + screentime_wkday_games_baseline + screentime_wkday_texting_baseline + screentime_wkday_sns_baseline + screentime_wkday_videochat_baseline + screentime_wkend_tv_baseline + screentime_wkend_videos_baseline + screentime_wkend_games_baseline + screentime_wkend_texting_baseline + screentime_wkend_sns_baseline + screentime_wkend_videochat_baseline + screentime_wkday_y_baseline + screentime_wkend_y_baseline + screentime_wkday_p_baseline + screentime_wkend_p_baseline + screentime_maturegames_baseline + screentime_rmovies_baseline + screentime_addict_p_2year + screentime_risk_p_2year + screentime_maturegames_2year + screentime_rmovies_2year + bpm_total_y_2year + bpm_total_t_baseline + bpm_total_t_2year + prosocial_y_baseline + prosocial_p_baseline + detention_baseline + detention_rea_baseline + friends_boys_baseline + friends_girls_baseline + Cfriends_boys_baseline + Cfriends_girls_baseline + friends_same_baseline + friends_diff_baseline + Cfriends_diff_baseline + Cfriends_same_baseline + prosocial_y_2year + prosocial_p_2year + friends_boys_2year + friends_girls_2year + Cfriends_boys_2year + Cfriends_girls_2year + friends_same_2year + friends_diff_2year + Cfriends_diff_2year + Cfriends_same_2year + cpeur2_baseline + eaeur1_baseline + depeur4_baseline + mddeur6_baseline + depmulti_baseline + bmieur4_baseline + bmimulti_baseline + insomniaeur6_baseline + snoringeur1_baseline + iqeur2_baseline + happieur4_baseline + ghappieur2_baseline + ghappimeaneur1_baseline + ghappihealth6_baseline + alcdep_eurauto_baseline + alcdep_afrauto_baseline + alcdep_metaauto_baseline + asdauto_baseline + aspauto_baseline + bipauto_baseline + cannabisauto_baseline + crossauto_baseline + drinkauto_baseline + edauto_baseline + neuroticismauto_baseline + ocdauto_baseline + risk4pcauto_baseline + risktolauto_baseline + scz_eurauto_baseline + scz_easauto_baseline + scz_metaauto_baseline + smokerauto_baseline + worryauto_baseline + anxietyauto_baseline + ptsdeur4_baseline + ptsdmeta6_baseline + adhdeur6_baseline + euro_baseline + sex_baseline + race_g_baseline + parent_identity_baseline + demo_brthdat_v2_baseline + demo_sex_v2_baseline + gender_identity_baseline + parent_age_baseline + foreign_born_family_baseline + married_baseline + high_educ_baseline + high_educ2_baseline + income_baseline + foreign_born_baseline + religion_prefer_baseline + gay_parent_baseline + gay_youth_baseline + race_ethnicity_baseline + age_baseline + family_adversity_baseline + height_baseline + weight_baseline + vol_baseline + bmi_baseline + total_ratio_baseline + history_ratio_baseline + euro_2year + race_g_2year + gay_youth_2year + age_2year + height_2year + weight_2year + vol_2year + bmi_2year', data=combined_data).fit()

# Number of parameters in the model (excluding the intercept)
k = len(model_combined.params) - 1

# Chow Test formula
numerator = (model_combined.ssr - (model_low.ssr + model_high.ssr)) / k
denominator = (model_low.ssr + model_high.ssr) / (len(df_low) + len(df_high) - 2 * k)
f_statistic = numerator / denominator
p_value = stats.f.sf(f_statistic, k, len(df_low) + len(df_high) - 2 * k)  # Using survival function for p-value

print("F-statistic:", f_statistic)
print("P-value:", p_value)
# print(model_low.summary())
coef_low = model_low.params
p_low = model_low.pvalues

# Filter coefficients with p-values less than 0.05
sig_coef_low = coef_low[p_low < 0.05].index

coef_combined_low = model_combined.params[sig_coef_low]

coefficient_differences = abs(coef_combined_low - coef_low[sig_coef_low])

diff_low_df = pd.DataFrame({'Difference': coefficient_differences, 'Coefficients': coef_low[sig_coef_low], 'P-values': p_low[sig_coef_low]}).sort_values(by='Difference', ascending=False)

print(diff_low_df)

coef_high = model_high.params
p_high = model_high.pvalues

# Filter coefficients with p-values less than 0.05
sig_coef_high = coef_high[p_high < 0.05].index

coef_combined_high = model_combined.params[sig_coef_high]

coefficient_differences = abs(coef_combined_high - coef_high[sig_coef_high])

diff_high_df = pd.DataFrame({'Difference': coefficient_differences, 'Coefficients': coef_high[sig_coef_high], 'P-values': p_high[sig_coef_high]}).sort_values(by='Difference', ascending=False)

print(diff_high_df)

In [ ]:
# nan_columns_low = low_picvocab_pos.columns[low_picvocab_pos.isna().any()].tolist()
# print(nan_columns_low)
# nan_columns_high = low_picvocab_pos.columns[high_picvocab_pos.isna().any()].tolist()
# print(nan_columns_high)
# remove prenatal_heroin_before_baseline, prenatal_heroin_after_baseline, prenatal_oxycontin_after_baseline

In [ ]:
model_low = ols('nihtbx_picvocab_uncorrected_2year ~ KSADS_BD_y_baseline + KSADS_Anx_y_baseline + KSADS_Suicide_y_baseline + KSADS_Sleep_y_baseline + KSADS_total_y_baseline + KSADS_BD_y_2year + KSADS_Anx_y_2year + KSADS_Eat_y_2year + KSADS_Suicide_y_2year + KSADS_Sleep_y_2year + KSADS_total_y_2year + KSADS_BD_p_baseline + KSADS_Anx_p_baseline + KSADS_Eat_p_baseline + KSADS_Suicide_p_baseline + KSADS_Sleep_p_baseline + KSADS_total_p_baseline + KSADS_BD_p_2year + KSADS_Anx_p_2year + KSADS_Eat_p_2year + KSADS_Suicide_p_2year + KSADS_Sleep_p_2year + KSADS_total_p_2year + KSADS_Subst_p_baseline + KSADS_Subst_p_2year + cbcl_internal_baseline + cbcl_internal_2year + cbcl_external_baseline + cbcl_external_2year + cbcl_totprob_baseline + cbcl_totprob_2year + upps_premeditation_baseline + upps_premeditation_2year + upps_perseverence_baseline + upps_perseverence_2year + upps_sensation_baseline + upps_sensation_2year + upps_negative_baseline + upps_negative_2year + upps_positive_baseline + upps_positive_2year + BIS_baseline + BIS_2year + BAS_RR_baseline + BAS_RR_2year + BAS_Drive_baseline + BAS_Drive_2year + BAS_Fun_baseline + BAS_Fun_2year + sleep_disturb_total_baseline + sleep_disturb_total_2year + ELS_total_baseline + ELS_total_2year + rh_adi_wsum1_baseline + rh_adi_wsum1_2year + rh_adi_wsum2_baseline + rh_adi_wsum2_2year + risk_alcohol_p_baseline + risk_cigarette_p_baseline + risk_electro_nicotine_p_baseline + risk_marijuana_p_baseline + risk_drug_p_baseline + risk_med_legal_p_baseline + risk_med_marijuana_p_baseline + risk_med_marijuana_pre_p_baseline + risk_family_med_marijuana_p_baseline + risk_alcohol_p_2year + risk_cigarette_p_2year + risk_electro_nicotine_p_2year + risk_marijuana_p_2year + risk_drug_p_2year + risk_med_legal_p_2year + risk_med_marijuana_p_2year + risk_med_marijuana_pre_p_2year + risk_family_med_marijuana_p_2year + risk_alcohol_y_2year + risk_cigarette_y_2year + risk_electro_nicotine_y_2year + risk_marijuana_y_2year + risk_med_legal_y_2year + risk_med_marijuana_y_2year + risk_med_marijuana_pre_y_2year + risk_family_med_marijuana_y_2year + risk_drug_y_2year + risk_gas_y_2year + risk_pain_y_2year + risk_anix_y_2year + risk_stimulant_y_2year + good_school_baseline + good_parent1_baseline + good_parent2_baseline + good_parent_baseline + good_school_2year + prenatal_tobacco_before_baseline + prenatal_alcohol_max_before_baseline + prenatal_alcohol_avg_before_baseline + prenatal_alcohol_eff_before_baseline + prenatal_marijuana_before_baseline + prenatal_cocaine_before_baseline + prenatal_oxycontin_before_baseline + prenatal_tobacco_after_baseline + prenatal_alcohol_max_after_baseline + prenatal_alcohol_avg_after_baseline + prenatal_alcohol_eff_after_baseline + prenatal_marijuana_after_baseline + prenatal_cocaine_after_baseline + prenatal_weeks_baseline + screentime_wkday_tv_baseline + screentime_wkday_videos_baseline + screentime_wkday_games_baseline + screentime_wkday_texting_baseline + screentime_wkday_sns_baseline + screentime_wkday_videochat_baseline + screentime_wkend_tv_baseline + screentime_wkend_videos_baseline + screentime_wkend_games_baseline + screentime_wkend_texting_baseline + screentime_wkend_sns_baseline + screentime_wkend_videochat_baseline + screentime_wkday_y_baseline + screentime_wkend_y_baseline + screentime_wkday_p_baseline + screentime_wkend_p_baseline + screentime_maturegames_baseline + screentime_rmovies_baseline + screentime_addict_p_2year + screentime_risk_p_2year + screentime_maturegames_2year + screentime_rmovies_2year + bpm_total_y_2year + bpm_total_t_baseline + bpm_total_t_2year + prosocial_y_baseline + prosocial_p_baseline + detention_baseline + detention_rea_baseline + friends_boys_baseline + friends_girls_baseline + Cfriends_boys_baseline + Cfriends_girls_baseline + friends_same_baseline + friends_diff_baseline + Cfriends_diff_baseline + Cfriends_same_baseline + prosocial_y_2year + prosocial_p_2year + friends_boys_2year + friends_girls_2year + Cfriends_boys_2year + Cfriends_girls_2year + friends_same_2year + friends_diff_2year + Cfriends_diff_2year + Cfriends_same_2year + cpeur2_baseline + eaeur1_baseline + depeur4_baseline + mddeur6_baseline + depmulti_baseline + bmieur4_baseline + bmimulti_baseline + insomniaeur6_baseline + snoringeur1_baseline + iqeur2_baseline + happieur4_baseline + ghappieur2_baseline + ghappimeaneur1_baseline + ghappihealth6_baseline + alcdep_eurauto_baseline + alcdep_afrauto_baseline + alcdep_metaauto_baseline + asdauto_baseline + aspauto_baseline + bipauto_baseline + cannabisauto_baseline + crossauto_baseline + drinkauto_baseline + edauto_baseline + neuroticismauto_baseline + ocdauto_baseline + risk4pcauto_baseline + risktolauto_baseline + scz_eurauto_baseline + scz_easauto_baseline + scz_metaauto_baseline + smokerauto_baseline + worryauto_baseline + anxietyauto_baseline + ptsdeur4_baseline + ptsdmeta6_baseline + adhdeur6_baseline + euro_baseline + sex_baseline + race_g_baseline + parent_identity_baseline + demo_brthdat_v2_baseline + demo_sex_v2_baseline + gender_identity_baseline + parent_age_baseline + foreign_born_family_baseline + married_baseline + high_educ_baseline + high_educ2_baseline + income_baseline + foreign_born_baseline + religion_prefer_baseline + gay_parent_baseline + gay_youth_baseline + race_ethnicity_baseline + age_baseline + family_adversity_baseline + height_baseline + weight_baseline + vol_baseline + bmi_baseline + total_ratio_baseline + history_ratio_baseline + euro_2year + race_g_2year + gay_youth_2year + age_2year + height_2year + weight_2year + vol_2year + bmi_2year', data=low_picvocab_pos).fit()
model_high = ols('nihtbx_picvocab_uncorrected_2year ~ KSADS_BD_y_baseline + KSADS_Anx_y_baseline + KSADS_Suicide_y_baseline + KSADS_Sleep_y_baseline + KSADS_total_y_baseline + KSADS_BD_y_2year + KSADS_Anx_y_2year + KSADS_Eat_y_2year + KSADS_Suicide_y_2year + KSADS_Sleep_y_2year + KSADS_total_y_2year + KSADS_BD_p_baseline + KSADS_Anx_p_baseline + KSADS_Eat_p_baseline + KSADS_Suicide_p_baseline + KSADS_Sleep_p_baseline + KSADS_total_p_baseline + KSADS_BD_p_2year + KSADS_Anx_p_2year + KSADS_Eat_p_2year + KSADS_Suicide_p_2year + KSADS_Sleep_p_2year + KSADS_total_p_2year + KSADS_Subst_p_baseline + KSADS_Subst_p_2year + cbcl_internal_baseline + cbcl_internal_2year + cbcl_external_baseline + cbcl_external_2year + cbcl_totprob_baseline + cbcl_totprob_2year + upps_premeditation_baseline + upps_premeditation_2year + upps_perseverence_baseline + upps_perseverence_2year + upps_sensation_baseline + upps_sensation_2year + upps_negative_baseline + upps_negative_2year + upps_positive_baseline + upps_positive_2year + BIS_baseline + BIS_2year + BAS_RR_baseline + BAS_RR_2year + BAS_Drive_baseline + BAS_Drive_2year + BAS_Fun_baseline + BAS_Fun_2year + sleep_disturb_total_baseline + sleep_disturb_total_2year + ELS_total_baseline + ELS_total_2year + rh_adi_wsum1_baseline + rh_adi_wsum1_2year + rh_adi_wsum2_baseline + rh_adi_wsum2_2year + risk_alcohol_p_baseline + risk_cigarette_p_baseline + risk_electro_nicotine_p_baseline + risk_marijuana_p_baseline + risk_drug_p_baseline + risk_med_legal_p_baseline + risk_med_marijuana_p_baseline + risk_med_marijuana_pre_p_baseline + risk_family_med_marijuana_p_baseline + risk_alcohol_p_2year + risk_cigarette_p_2year + risk_electro_nicotine_p_2year + risk_marijuana_p_2year + risk_drug_p_2year + risk_med_legal_p_2year + risk_med_marijuana_p_2year + risk_med_marijuana_pre_p_2year + risk_family_med_marijuana_p_2year + risk_alcohol_y_2year + risk_cigarette_y_2year + risk_electro_nicotine_y_2year + risk_marijuana_y_2year + risk_med_legal_y_2year + risk_med_marijuana_y_2year + risk_med_marijuana_pre_y_2year + risk_family_med_marijuana_y_2year + risk_drug_y_2year + risk_gas_y_2year + risk_pain_y_2year + risk_anix_y_2year + risk_stimulant_y_2year + good_school_baseline + good_parent1_baseline + good_parent2_baseline + good_parent_baseline + good_school_2year + prenatal_tobacco_before_baseline + prenatal_alcohol_max_before_baseline + prenatal_alcohol_avg_before_baseline + prenatal_alcohol_eff_before_baseline + prenatal_marijuana_before_baseline + prenatal_cocaine_before_baseline + prenatal_oxycontin_before_baseline + prenatal_tobacco_after_baseline + prenatal_alcohol_max_after_baseline + prenatal_alcohol_avg_after_baseline + prenatal_alcohol_eff_after_baseline + prenatal_marijuana_after_baseline + prenatal_cocaine_after_baseline + prenatal_weeks_baseline + screentime_wkday_tv_baseline + screentime_wkday_videos_baseline + screentime_wkday_games_baseline + screentime_wkday_texting_baseline + screentime_wkday_sns_baseline + screentime_wkday_videochat_baseline + screentime_wkend_tv_baseline + screentime_wkend_videos_baseline + screentime_wkend_games_baseline + screentime_wkend_texting_baseline + screentime_wkend_sns_baseline + screentime_wkend_videochat_baseline + screentime_wkday_y_baseline + screentime_wkend_y_baseline + screentime_wkday_p_baseline + screentime_wkend_p_baseline + screentime_maturegames_baseline + screentime_rmovies_baseline + screentime_addict_p_2year + screentime_risk_p_2year + screentime_maturegames_2year + screentime_rmovies_2year + bpm_total_y_2year + bpm_total_t_baseline + bpm_total_t_2year + prosocial_y_baseline + prosocial_p_baseline + detention_baseline + detention_rea_baseline + friends_boys_baseline + friends_girls_baseline + Cfriends_boys_baseline + Cfriends_girls_baseline + friends_same_baseline + friends_diff_baseline + Cfriends_diff_baseline + Cfriends_same_baseline + prosocial_y_2year + prosocial_p_2year + friends_boys_2year + friends_girls_2year + Cfriends_boys_2year + Cfriends_girls_2year + friends_same_2year + friends_diff_2year + Cfriends_diff_2year + Cfriends_same_2year + cpeur2_baseline + eaeur1_baseline + depeur4_baseline + mddeur6_baseline + depmulti_baseline + bmieur4_baseline + bmimulti_baseline + insomniaeur6_baseline + snoringeur1_baseline + iqeur2_baseline + happieur4_baseline + ghappieur2_baseline + ghappimeaneur1_baseline + ghappihealth6_baseline + alcdep_eurauto_baseline + alcdep_afrauto_baseline + alcdep_metaauto_baseline + asdauto_baseline + aspauto_baseline + bipauto_baseline + cannabisauto_baseline + crossauto_baseline + drinkauto_baseline + edauto_baseline + neuroticismauto_baseline + ocdauto_baseline + risk4pcauto_baseline + risktolauto_baseline + scz_eurauto_baseline + scz_easauto_baseline + scz_metaauto_baseline + smokerauto_baseline + worryauto_baseline + anxietyauto_baseline + ptsdeur4_baseline + ptsdmeta6_baseline + adhdeur6_baseline + euro_baseline + sex_baseline + race_g_baseline + parent_identity_baseline + demo_brthdat_v2_baseline + demo_sex_v2_baseline + gender_identity_baseline + parent_age_baseline + foreign_born_family_baseline + married_baseline + high_educ_baseline + high_educ2_baseline + income_baseline + foreign_born_baseline + religion_prefer_baseline + gay_parent_baseline + gay_youth_baseline + race_ethnicity_baseline + age_baseline + family_adversity_baseline + height_baseline + weight_baseline + vol_baseline + bmi_baseline + total_ratio_baseline + history_ratio_baseline + euro_2year + race_g_2year + gay_youth_2year + age_2year + height_2year + weight_2year + vol_2year + bmi_2year', data=high_picvocab_pos).fit()

# Combine data for Chow Test
combined_data = pd.concat([low_picvocab_pos, high_picvocab_pos])
model_combined = ols('nihtbx_picvocab_uncorrected_2year ~ KSADS_BD_y_baseline + KSADS_Anx_y_baseline + KSADS_Suicide_y_baseline + KSADS_Sleep_y_baseline + KSADS_total_y_baseline + KSADS_BD_y_2year + KSADS_Anx_y_2year + KSADS_Eat_y_2year + KSADS_Suicide_y_2year + KSADS_Sleep_y_2year + KSADS_total_y_2year + KSADS_BD_p_baseline + KSADS_Anx_p_baseline + KSADS_Eat_p_baseline + KSADS_Suicide_p_baseline + KSADS_Sleep_p_baseline + KSADS_total_p_baseline + KSADS_BD_p_2year + KSADS_Anx_p_2year + KSADS_Eat_p_2year + KSADS_Suicide_p_2year + KSADS_Sleep_p_2year + KSADS_total_p_2year + KSADS_Subst_p_baseline + KSADS_Subst_p_2year + cbcl_internal_baseline + cbcl_internal_2year + cbcl_external_baseline + cbcl_external_2year + cbcl_totprob_baseline + cbcl_totprob_2year + upps_premeditation_baseline + upps_premeditation_2year + upps_perseverence_baseline + upps_perseverence_2year + upps_sensation_baseline + upps_sensation_2year + upps_negative_baseline + upps_negative_2year + upps_positive_baseline + upps_positive_2year + BIS_baseline + BIS_2year + BAS_RR_baseline + BAS_RR_2year + BAS_Drive_baseline + BAS_Drive_2year + BAS_Fun_baseline + BAS_Fun_2year + sleep_disturb_total_baseline + sleep_disturb_total_2year + ELS_total_baseline + ELS_total_2year + rh_adi_wsum1_baseline + rh_adi_wsum1_2year + rh_adi_wsum2_baseline + rh_adi_wsum2_2year + risk_alcohol_p_baseline + risk_cigarette_p_baseline + risk_electro_nicotine_p_baseline + risk_marijuana_p_baseline + risk_drug_p_baseline + risk_med_legal_p_baseline + risk_med_marijuana_p_baseline + risk_med_marijuana_pre_p_baseline + risk_family_med_marijuana_p_baseline + risk_alcohol_p_2year + risk_cigarette_p_2year + risk_electro_nicotine_p_2year + risk_marijuana_p_2year + risk_drug_p_2year + risk_med_legal_p_2year + risk_med_marijuana_p_2year + risk_med_marijuana_pre_p_2year + risk_family_med_marijuana_p_2year + risk_alcohol_y_2year + risk_cigarette_y_2year + risk_electro_nicotine_y_2year + risk_marijuana_y_2year + risk_med_legal_y_2year + risk_med_marijuana_y_2year + risk_med_marijuana_pre_y_2year + risk_family_med_marijuana_y_2year + risk_drug_y_2year + risk_gas_y_2year + risk_pain_y_2year + risk_anix_y_2year + risk_stimulant_y_2year + good_school_baseline + good_parent1_baseline + good_parent2_baseline + good_parent_baseline + good_school_2year + prenatal_tobacco_before_baseline + prenatal_alcohol_max_before_baseline + prenatal_alcohol_avg_before_baseline + prenatal_alcohol_eff_before_baseline + prenatal_marijuana_before_baseline + prenatal_cocaine_before_baseline + prenatal_oxycontin_before_baseline + prenatal_tobacco_after_baseline + prenatal_alcohol_max_after_baseline + prenatal_alcohol_avg_after_baseline + prenatal_alcohol_eff_after_baseline + prenatal_marijuana_after_baseline + prenatal_cocaine_after_baseline + prenatal_weeks_baseline + screentime_wkday_tv_baseline + screentime_wkday_videos_baseline + screentime_wkday_games_baseline + screentime_wkday_texting_baseline + screentime_wkday_sns_baseline + screentime_wkday_videochat_baseline + screentime_wkend_tv_baseline + screentime_wkend_videos_baseline + screentime_wkend_games_baseline + screentime_wkend_texting_baseline + screentime_wkend_sns_baseline + screentime_wkend_videochat_baseline + screentime_wkday_y_baseline + screentime_wkend_y_baseline + screentime_wkday_p_baseline + screentime_wkend_p_baseline + screentime_maturegames_baseline + screentime_rmovies_baseline + screentime_addict_p_2year + screentime_risk_p_2year + screentime_maturegames_2year + screentime_rmovies_2year + bpm_total_y_2year + bpm_total_t_baseline + bpm_total_t_2year + prosocial_y_baseline + prosocial_p_baseline + detention_baseline + detention_rea_baseline + friends_boys_baseline + friends_girls_baseline + Cfriends_boys_baseline + Cfriends_girls_baseline + friends_same_baseline + friends_diff_baseline + Cfriends_diff_baseline + Cfriends_same_baseline + prosocial_y_2year + prosocial_p_2year + friends_boys_2year + friends_girls_2year + Cfriends_boys_2year + Cfriends_girls_2year + friends_same_2year + friends_diff_2year + Cfriends_diff_2year + Cfriends_same_2year + cpeur2_baseline + eaeur1_baseline + depeur4_baseline + mddeur6_baseline + depmulti_baseline + bmieur4_baseline + bmimulti_baseline + insomniaeur6_baseline + snoringeur1_baseline + iqeur2_baseline + happieur4_baseline + ghappieur2_baseline + ghappimeaneur1_baseline + ghappihealth6_baseline + alcdep_eurauto_baseline + alcdep_afrauto_baseline + alcdep_metaauto_baseline + asdauto_baseline + aspauto_baseline + bipauto_baseline + cannabisauto_baseline + crossauto_baseline + drinkauto_baseline + edauto_baseline + neuroticismauto_baseline + ocdauto_baseline + risk4pcauto_baseline + risktolauto_baseline + scz_eurauto_baseline + scz_easauto_baseline + scz_metaauto_baseline + smokerauto_baseline + worryauto_baseline + anxietyauto_baseline + ptsdeur4_baseline + ptsdmeta6_baseline + adhdeur6_baseline + euro_baseline + sex_baseline + race_g_baseline + parent_identity_baseline + demo_brthdat_v2_baseline + demo_sex_v2_baseline + gender_identity_baseline + parent_age_baseline + foreign_born_family_baseline + married_baseline + high_educ_baseline + high_educ2_baseline + income_baseline + foreign_born_baseline + religion_prefer_baseline + gay_parent_baseline + gay_youth_baseline + race_ethnicity_baseline + age_baseline + family_adversity_baseline + height_baseline + weight_baseline + vol_baseline + bmi_baseline + total_ratio_baseline + history_ratio_baseline + euro_2year + race_g_2year + gay_youth_2year + age_2year + height_2year + weight_2year + vol_2year + bmi_2year', data=combined_data).fit()

# Number of parameters in the model (excluding the intercept)
k = len(model_combined.params) - 1

# Chow Test formula
numerator = (model_combined.ssr - (model_low.ssr + model_high.ssr)) / k
denominator = (model_low.ssr + model_high.ssr) / (len(df_low) + len(df_high) - 2 * k)
f_statistic = numerator / denominator
p_value = stats.f.sf(f_statistic, k, len(df_low) + len(df_high) - 2 * k)  # Using survival function for p-value

print("F-statistic:", f_statistic)
print("P-value:", p_value)

coef_low = model_low.params
p_low = model_low.pvalues

# Filter coefficients with p-values less than 0.05
sig_coef_low = coef_low[p_low < 0.05].index

coef_combined_low = model_combined.params[sig_coef_low]

coefficient_differences = abs(coef_combined_low - coef_low[sig_coef_low])

diff_low_df = pd.DataFrame({'Difference': coefficient_differences, 'Coefficients': coef_low[sig_coef_low], 'P-values': p_low[sig_coef_low]}).sort_values(by='Difference', ascending=False)

print(diff_low_df)

coef_high = model_high.params
p_high = model_high.pvalues

# Filter coefficients with p-values less than 0.05
sig_coef_high = coef_high[p_high < 0.05].index

coef_combined_high = model_combined.params[sig_coef_high]

coefficient_differences = abs(coef_combined_high - coef_high[sig_coef_high])

diff_high_df = pd.DataFrame({'Difference': coefficient_differences, 'Coefficients': coef_high[sig_coef_high], 'P-values': p_high[sig_coef_high]}).sort_values(by='Difference', ascending=False)

print(diff_high_df)

In [ ]:
# nan_columns_low = low_flanker_neg.columns[low_flanker_neg.isna().any()].tolist()
# print(nan_columns_low)
# nan_columns_high = high_flanker_neg.columns[high_flanker_neg.isna().any()].tolist()
# print(nan_columns_high)
# remove prenatal_heroin_before_baseline, prenatal_heroin_after_baseline, prenatal_cocaine_after_baseline, prenatal_oxycontin_after_baseline

In [ ]:
model_low = ols('nihtbx_flanker_uncorrected_2year ~ KSADS_BD_y_baseline + KSADS_Anx_y_baseline + KSADS_Suicide_y_baseline + KSADS_Sleep_y_baseline + KSADS_total_y_baseline + KSADS_BD_y_2year + KSADS_Anx_y_2year + KSADS_Eat_y_2year + KSADS_Suicide_y_2year + KSADS_Sleep_y_2year + KSADS_total_y_2year + KSADS_BD_p_baseline + KSADS_Anx_p_baseline + KSADS_Eat_p_baseline + KSADS_Suicide_p_baseline + KSADS_Sleep_p_baseline + KSADS_total_p_baseline + KSADS_BD_p_2year + KSADS_Anx_p_2year + KSADS_Eat_p_2year + KSADS_Suicide_p_2year + KSADS_Sleep_p_2year + KSADS_total_p_2year + KSADS_Subst_p_baseline + KSADS_Subst_p_2year + cbcl_internal_baseline + cbcl_internal_2year + cbcl_external_baseline + cbcl_external_2year + cbcl_totprob_baseline + cbcl_totprob_2year + upps_premeditation_baseline + upps_premeditation_2year + upps_perseverence_baseline + upps_perseverence_2year + upps_sensation_baseline + upps_sensation_2year + upps_negative_baseline + upps_negative_2year + upps_positive_baseline + upps_positive_2year + BIS_baseline + BIS_2year + BAS_RR_baseline + BAS_RR_2year + BAS_Drive_baseline + BAS_Drive_2year + BAS_Fun_baseline + BAS_Fun_2year + sleep_disturb_total_baseline + sleep_disturb_total_2year + ELS_total_baseline + ELS_total_2year + rh_adi_wsum1_baseline + rh_adi_wsum1_2year + rh_adi_wsum2_baseline + rh_adi_wsum2_2year + risk_alcohol_p_baseline + risk_cigarette_p_baseline + risk_electro_nicotine_p_baseline + risk_marijuana_p_baseline + risk_drug_p_baseline + risk_med_legal_p_baseline + risk_med_marijuana_p_baseline + risk_med_marijuana_pre_p_baseline + risk_family_med_marijuana_p_baseline + risk_alcohol_p_2year + risk_cigarette_p_2year + risk_electro_nicotine_p_2year + risk_marijuana_p_2year + risk_drug_p_2year + risk_med_legal_p_2year + risk_med_marijuana_p_2year + risk_med_marijuana_pre_p_2year + risk_family_med_marijuana_p_2year + risk_alcohol_y_2year + risk_cigarette_y_2year + risk_electro_nicotine_y_2year + risk_marijuana_y_2year + risk_med_legal_y_2year + risk_med_marijuana_y_2year + risk_med_marijuana_pre_y_2year + risk_family_med_marijuana_y_2year + risk_drug_y_2year + risk_gas_y_2year + risk_pain_y_2year + risk_anix_y_2year + risk_stimulant_y_2year + good_school_baseline + good_parent1_baseline + good_parent2_baseline + good_parent_baseline + good_school_2year + prenatal_tobacco_before_baseline + prenatal_alcohol_max_before_baseline + prenatal_alcohol_avg_before_baseline + prenatal_alcohol_eff_before_baseline + prenatal_marijuana_before_baseline + prenatal_cocaine_before_baseline + prenatal_oxycontin_before_baseline + prenatal_tobacco_after_baseline + prenatal_alcohol_max_after_baseline + prenatal_alcohol_avg_after_baseline + prenatal_alcohol_eff_after_baseline + prenatal_marijuana_after_baseline + prenatal_weeks_baseline + screentime_wkday_tv_baseline + screentime_wkday_videos_baseline + screentime_wkday_games_baseline + screentime_wkday_texting_baseline + screentime_wkday_sns_baseline + screentime_wkday_videochat_baseline + screentime_wkend_tv_baseline + screentime_wkend_videos_baseline + screentime_wkend_games_baseline + screentime_wkend_texting_baseline + screentime_wkend_sns_baseline + screentime_wkend_videochat_baseline + screentime_wkday_y_baseline + screentime_wkend_y_baseline + screentime_wkday_p_baseline + screentime_wkend_p_baseline + screentime_maturegames_baseline + screentime_rmovies_baseline + screentime_addict_p_2year + screentime_risk_p_2year + screentime_maturegames_2year + screentime_rmovies_2year + bpm_total_y_2year + bpm_total_t_baseline + bpm_total_t_2year + prosocial_y_baseline + prosocial_p_baseline + detention_baseline + detention_rea_baseline + friends_boys_baseline + friends_girls_baseline + Cfriends_boys_baseline + Cfriends_girls_baseline + friends_same_baseline + friends_diff_baseline + Cfriends_diff_baseline + Cfriends_same_baseline + prosocial_y_2year + prosocial_p_2year + friends_boys_2year + friends_girls_2year + Cfriends_boys_2year + Cfriends_girls_2year + friends_same_2year + friends_diff_2year + Cfriends_diff_2year + Cfriends_same_2year + cpeur2_baseline + eaeur1_baseline + depeur4_baseline + mddeur6_baseline + depmulti_baseline + bmieur4_baseline + bmimulti_baseline + insomniaeur6_baseline + snoringeur1_baseline + iqeur2_baseline + happieur4_baseline + ghappieur2_baseline + ghappimeaneur1_baseline + ghappihealth6_baseline + alcdep_eurauto_baseline + alcdep_afrauto_baseline + alcdep_metaauto_baseline + asdauto_baseline + aspauto_baseline + bipauto_baseline + cannabisauto_baseline + crossauto_baseline + drinkauto_baseline + edauto_baseline + neuroticismauto_baseline + ocdauto_baseline + risk4pcauto_baseline + risktolauto_baseline + scz_eurauto_baseline + scz_easauto_baseline + scz_metaauto_baseline + smokerauto_baseline + worryauto_baseline + anxietyauto_baseline + ptsdeur4_baseline + ptsdmeta6_baseline + adhdeur6_baseline + euro_baseline + sex_baseline + race_g_baseline + parent_identity_baseline + demo_brthdat_v2_baseline + demo_sex_v2_baseline + gender_identity_baseline + parent_age_baseline + foreign_born_family_baseline + married_baseline + high_educ_baseline + high_educ2_baseline + income_baseline + foreign_born_baseline + religion_prefer_baseline + gay_parent_baseline + gay_youth_baseline + race_ethnicity_baseline + age_baseline + family_adversity_baseline + height_baseline + weight_baseline + vol_baseline + bmi_baseline + total_ratio_baseline + history_ratio_baseline + euro_2year + race_g_2year + gay_youth_2year + age_2year + height_2year + weight_2year + vol_2year + bmi_2year', data=low_flanker_neg).fit()
model_high = ols('nihtbx_flanker_uncorrected_2year ~ KSADS_BD_y_baseline + KSADS_Anx_y_baseline + KSADS_Suicide_y_baseline + KSADS_Sleep_y_baseline + KSADS_total_y_baseline + KSADS_BD_y_2year + KSADS_Anx_y_2year + KSADS_Eat_y_2year + KSADS_Suicide_y_2year + KSADS_Sleep_y_2year + KSADS_total_y_2year + KSADS_BD_p_baseline + KSADS_Anx_p_baseline + KSADS_Eat_p_baseline + KSADS_Suicide_p_baseline + KSADS_Sleep_p_baseline + KSADS_total_p_baseline + KSADS_BD_p_2year + KSADS_Anx_p_2year + KSADS_Eat_p_2year + KSADS_Suicide_p_2year + KSADS_Sleep_p_2year + KSADS_total_p_2year + KSADS_Subst_p_baseline + KSADS_Subst_p_2year + cbcl_internal_baseline + cbcl_internal_2year + cbcl_external_baseline + cbcl_external_2year + cbcl_totprob_baseline + cbcl_totprob_2year + upps_premeditation_baseline + upps_premeditation_2year + upps_perseverence_baseline + upps_perseverence_2year + upps_sensation_baseline + upps_sensation_2year + upps_negative_baseline + upps_negative_2year + upps_positive_baseline + upps_positive_2year + BIS_baseline + BIS_2year + BAS_RR_baseline + BAS_RR_2year + BAS_Drive_baseline + BAS_Drive_2year + BAS_Fun_baseline + BAS_Fun_2year + sleep_disturb_total_baseline + sleep_disturb_total_2year + ELS_total_baseline + ELS_total_2year + rh_adi_wsum1_baseline + rh_adi_wsum1_2year + rh_adi_wsum2_baseline + rh_adi_wsum2_2year + risk_alcohol_p_baseline + risk_cigarette_p_baseline + risk_electro_nicotine_p_baseline + risk_marijuana_p_baseline + risk_drug_p_baseline + risk_med_legal_p_baseline + risk_med_marijuana_p_baseline + risk_med_marijuana_pre_p_baseline + risk_family_med_marijuana_p_baseline + risk_alcohol_p_2year + risk_cigarette_p_2year + risk_electro_nicotine_p_2year + risk_marijuana_p_2year + risk_drug_p_2year + risk_med_legal_p_2year + risk_med_marijuana_p_2year + risk_med_marijuana_pre_p_2year + risk_family_med_marijuana_p_2year + risk_alcohol_y_2year + risk_cigarette_y_2year + risk_electro_nicotine_y_2year + risk_marijuana_y_2year + risk_med_legal_y_2year + risk_med_marijuana_y_2year + risk_med_marijuana_pre_y_2year + risk_family_med_marijuana_y_2year + risk_drug_y_2year + risk_gas_y_2year + risk_pain_y_2year + risk_anix_y_2year + risk_stimulant_y_2year + good_school_baseline + good_parent1_baseline + good_parent2_baseline + good_parent_baseline + good_school_2year + prenatal_tobacco_before_baseline + prenatal_alcohol_max_before_baseline + prenatal_alcohol_avg_before_baseline + prenatal_alcohol_eff_before_baseline + prenatal_marijuana_before_baseline + prenatal_cocaine_before_baseline + prenatal_oxycontin_before_baseline + prenatal_tobacco_after_baseline + prenatal_alcohol_max_after_baseline + prenatal_alcohol_avg_after_baseline + prenatal_alcohol_eff_after_baseline + prenatal_marijuana_after_baseline + prenatal_weeks_baseline + screentime_wkday_tv_baseline + screentime_wkday_videos_baseline + screentime_wkday_games_baseline + screentime_wkday_texting_baseline + screentime_wkday_sns_baseline + screentime_wkday_videochat_baseline + screentime_wkend_tv_baseline + screentime_wkend_videos_baseline + screentime_wkend_games_baseline + screentime_wkend_texting_baseline + screentime_wkend_sns_baseline + screentime_wkend_videochat_baseline + screentime_wkday_y_baseline + screentime_wkend_y_baseline + screentime_wkday_p_baseline + screentime_wkend_p_baseline + screentime_maturegames_baseline + screentime_rmovies_baseline + screentime_addict_p_2year + screentime_risk_p_2year + screentime_maturegames_2year + screentime_rmovies_2year + bpm_total_y_2year + bpm_total_t_baseline + bpm_total_t_2year + prosocial_y_baseline + prosocial_p_baseline + detention_baseline + detention_rea_baseline + friends_boys_baseline + friends_girls_baseline + Cfriends_boys_baseline + Cfriends_girls_baseline + friends_same_baseline + friends_diff_baseline + Cfriends_diff_baseline + Cfriends_same_baseline + prosocial_y_2year + prosocial_p_2year + friends_boys_2year + friends_girls_2year + Cfriends_boys_2year + Cfriends_girls_2year + friends_same_2year + friends_diff_2year + Cfriends_diff_2year + Cfriends_same_2year + cpeur2_baseline + eaeur1_baseline + depeur4_baseline + mddeur6_baseline + depmulti_baseline + bmieur4_baseline + bmimulti_baseline + insomniaeur6_baseline + snoringeur1_baseline + iqeur2_baseline + happieur4_baseline + ghappieur2_baseline + ghappimeaneur1_baseline + ghappihealth6_baseline + alcdep_eurauto_baseline + alcdep_afrauto_baseline + alcdep_metaauto_baseline + asdauto_baseline + aspauto_baseline + bipauto_baseline + cannabisauto_baseline + crossauto_baseline + drinkauto_baseline + edauto_baseline + neuroticismauto_baseline + ocdauto_baseline + risk4pcauto_baseline + risktolauto_baseline + scz_eurauto_baseline + scz_easauto_baseline + scz_metaauto_baseline + smokerauto_baseline + worryauto_baseline + anxietyauto_baseline + ptsdeur4_baseline + ptsdmeta6_baseline + adhdeur6_baseline + euro_baseline + sex_baseline + race_g_baseline + parent_identity_baseline + demo_brthdat_v2_baseline + demo_sex_v2_baseline + gender_identity_baseline + parent_age_baseline + foreign_born_family_baseline + married_baseline + high_educ_baseline + high_educ2_baseline + income_baseline + foreign_born_baseline + religion_prefer_baseline + gay_parent_baseline + gay_youth_baseline + race_ethnicity_baseline + age_baseline + family_adversity_baseline + height_baseline + weight_baseline + vol_baseline + bmi_baseline + total_ratio_baseline + history_ratio_baseline + euro_2year + race_g_2year + gay_youth_2year + age_2year + height_2year + weight_2year + vol_2year + bmi_2year', data=high_flanker_neg).fit()

# Combine data for Chow Test
combined_data = pd.concat([low_flanker_neg, high_flanker_neg])
model_combined = ols('nihtbx_flanker_uncorrected_2year ~ KSADS_BD_y_baseline + KSADS_Anx_y_baseline + KSADS_Suicide_y_baseline + KSADS_Sleep_y_baseline + KSADS_total_y_baseline + KSADS_BD_y_2year + KSADS_Anx_y_2year + KSADS_Eat_y_2year + KSADS_Suicide_y_2year + KSADS_Sleep_y_2year + KSADS_total_y_2year + KSADS_BD_p_baseline + KSADS_Anx_p_baseline + KSADS_Eat_p_baseline + KSADS_Suicide_p_baseline + KSADS_Sleep_p_baseline + KSADS_total_p_baseline + KSADS_BD_p_2year + KSADS_Anx_p_2year + KSADS_Eat_p_2year + KSADS_Suicide_p_2year + KSADS_Sleep_p_2year + KSADS_total_p_2year + KSADS_Subst_p_baseline + KSADS_Subst_p_2year + cbcl_internal_baseline + cbcl_internal_2year + cbcl_external_baseline + cbcl_external_2year + cbcl_totprob_baseline + cbcl_totprob_2year + upps_premeditation_baseline + upps_premeditation_2year + upps_perseverence_baseline + upps_perseverence_2year + upps_sensation_baseline + upps_sensation_2year + upps_negative_baseline + upps_negative_2year + upps_positive_baseline + upps_positive_2year + BIS_baseline + BIS_2year + BAS_RR_baseline + BAS_RR_2year + BAS_Drive_baseline + BAS_Drive_2year + BAS_Fun_baseline + BAS_Fun_2year + sleep_disturb_total_baseline + sleep_disturb_total_2year + ELS_total_baseline + ELS_total_2year + rh_adi_wsum1_baseline + rh_adi_wsum1_2year + rh_adi_wsum2_baseline + rh_adi_wsum2_2year + risk_alcohol_p_baseline + risk_cigarette_p_baseline + risk_electro_nicotine_p_baseline + risk_marijuana_p_baseline + risk_drug_p_baseline + risk_med_legal_p_baseline + risk_med_marijuana_p_baseline + risk_med_marijuana_pre_p_baseline + risk_family_med_marijuana_p_baseline + risk_alcohol_p_2year + risk_cigarette_p_2year + risk_electro_nicotine_p_2year + risk_marijuana_p_2year + risk_drug_p_2year + risk_med_legal_p_2year + risk_med_marijuana_p_2year + risk_med_marijuana_pre_p_2year + risk_family_med_marijuana_p_2year + risk_alcohol_y_2year + risk_cigarette_y_2year + risk_electro_nicotine_y_2year + risk_marijuana_y_2year + risk_med_legal_y_2year + risk_med_marijuana_y_2year + risk_med_marijuana_pre_y_2year + risk_family_med_marijuana_y_2year + risk_drug_y_2year + risk_gas_y_2year + risk_pain_y_2year + risk_anix_y_2year + risk_stimulant_y_2year + good_school_baseline + good_parent1_baseline + good_parent2_baseline + good_parent_baseline + good_school_2year + prenatal_tobacco_before_baseline + prenatal_alcohol_max_before_baseline + prenatal_alcohol_avg_before_baseline + prenatal_alcohol_eff_before_baseline + prenatal_marijuana_before_baseline + prenatal_cocaine_before_baseline + prenatal_oxycontin_before_baseline + prenatal_tobacco_after_baseline + prenatal_alcohol_max_after_baseline + prenatal_alcohol_avg_after_baseline + prenatal_alcohol_eff_after_baseline + prenatal_marijuana_after_baseline + prenatal_weeks_baseline + screentime_wkday_tv_baseline + screentime_wkday_videos_baseline + screentime_wkday_games_baseline + screentime_wkday_texting_baseline + screentime_wkday_sns_baseline + screentime_wkday_videochat_baseline + screentime_wkend_tv_baseline + screentime_wkend_videos_baseline + screentime_wkend_games_baseline + screentime_wkend_texting_baseline + screentime_wkend_sns_baseline + screentime_wkend_videochat_baseline + screentime_wkday_y_baseline + screentime_wkend_y_baseline + screentime_wkday_p_baseline + screentime_wkend_p_baseline + screentime_maturegames_baseline + screentime_rmovies_baseline + screentime_addict_p_2year + screentime_risk_p_2year + screentime_maturegames_2year + screentime_rmovies_2year + bpm_total_y_2year + bpm_total_t_baseline + bpm_total_t_2year + prosocial_y_baseline + prosocial_p_baseline + detention_baseline + detention_rea_baseline + friends_boys_baseline + friends_girls_baseline + Cfriends_boys_baseline + Cfriends_girls_baseline + friends_same_baseline + friends_diff_baseline + Cfriends_diff_baseline + Cfriends_same_baseline + prosocial_y_2year + prosocial_p_2year + friends_boys_2year + friends_girls_2year + Cfriends_boys_2year + Cfriends_girls_2year + friends_same_2year + friends_diff_2year + Cfriends_diff_2year + Cfriends_same_2year + cpeur2_baseline + eaeur1_baseline + depeur4_baseline + mddeur6_baseline + depmulti_baseline + bmieur4_baseline + bmimulti_baseline + insomniaeur6_baseline + snoringeur1_baseline + iqeur2_baseline + happieur4_baseline + ghappieur2_baseline + ghappimeaneur1_baseline + ghappihealth6_baseline + alcdep_eurauto_baseline + alcdep_afrauto_baseline + alcdep_metaauto_baseline + asdauto_baseline + aspauto_baseline + bipauto_baseline + cannabisauto_baseline + crossauto_baseline + drinkauto_baseline + edauto_baseline + neuroticismauto_baseline + ocdauto_baseline + risk4pcauto_baseline + risktolauto_baseline + scz_eurauto_baseline + scz_easauto_baseline + scz_metaauto_baseline + smokerauto_baseline + worryauto_baseline + anxietyauto_baseline + ptsdeur4_baseline + ptsdmeta6_baseline + adhdeur6_baseline + euro_baseline + sex_baseline + race_g_baseline + parent_identity_baseline + demo_brthdat_v2_baseline + demo_sex_v2_baseline + gender_identity_baseline + parent_age_baseline + foreign_born_family_baseline + married_baseline + high_educ_baseline + high_educ2_baseline + income_baseline + foreign_born_baseline + religion_prefer_baseline + gay_parent_baseline + gay_youth_baseline + race_ethnicity_baseline + age_baseline + family_adversity_baseline + height_baseline + weight_baseline + vol_baseline + bmi_baseline + total_ratio_baseline + history_ratio_baseline + euro_2year + race_g_2year + gay_youth_2year + age_2year + height_2year + weight_2year + vol_2year + bmi_2year', data=combined_data).fit()

# Number of parameters in the model (excluding the intercept)
k = len(model_combined.params) - 1

# Chow Test formula
numerator = (model_combined.ssr - (model_low.ssr + model_high.ssr)) / k
denominator = (model_low.ssr + model_high.ssr) / (len(df_low) + len(df_high) - 2 * k)
f_statistic = numerator / denominator
p_value = stats.f.sf(f_statistic, k, len(df_low) + len(df_high) - 2 * k)  # Using survival function for p-value

print("F-statistic:", f_statistic)
print("P-value:", p_value)

coef_low = model_low.params
p_low = model_low.pvalues

# Filter coefficients with p-values less than 0.05
sig_coef_low = coef_low[p_low < 0.05].index

coef_combined_low = model_combined.params[sig_coef_low]

coefficient_differences = abs(coef_combined_low - coef_low[sig_coef_low])

diff_low_df = pd.DataFrame({'Difference': coefficient_differences, 'Coefficients': coef_low[sig_coef_low], 'P-values': p_low[sig_coef_low]}).sort_values(by='Difference', ascending=False)

print(diff_low_df)

coef_high = model_high.params
p_high = model_high.pvalues

# Filter coefficients with p-values less than 0.05
sig_coef_high = coef_high[p_high < 0.05].index

coef_combined_high = model_combined.params[sig_coef_high]

coefficient_differences = abs(coef_combined_high - coef_high[sig_coef_high])

diff_high_df = pd.DataFrame({'Difference': coefficient_differences, 'Coefficients': coef_high[sig_coef_high], 'P-values': p_high[sig_coef_high]}).sort_values(by='Difference', ascending=False)

print(diff_high_df)

In [ ]:
# nan_columns_low = low_flanker_pos.columns[low_flanker_pos.isna().any()].tolist()
# print(nan_columns_low)
# nan_columns_high = high_flanker_pos.columns[high_flanker_pos.isna().any()].tolist()
# print(nan_columns_high)
# remove prenatal_heroin_before_baseline, prenatal_heroin_after_baseline, prenatal_oxycontin_after_baseline

In [ ]:
model_low = ols('nihtbx_flanker_uncorrected_2year ~ KSADS_BD_y_baseline + KSADS_Anx_y_baseline + KSADS_Suicide_y_baseline + KSADS_Sleep_y_baseline + KSADS_total_y_baseline + KSADS_BD_y_2year + KSADS_Anx_y_2year + KSADS_Eat_y_2year + KSADS_Suicide_y_2year + KSADS_Sleep_y_2year + KSADS_total_y_2year + KSADS_BD_p_baseline + KSADS_Anx_p_baseline + KSADS_Eat_p_baseline + KSADS_Suicide_p_baseline + KSADS_Sleep_p_baseline + KSADS_total_p_baseline + KSADS_BD_p_2year + KSADS_Anx_p_2year + KSADS_Eat_p_2year + KSADS_Suicide_p_2year + KSADS_Sleep_p_2year + KSADS_total_p_2year + KSADS_Subst_p_baseline + KSADS_Subst_p_2year + cbcl_internal_baseline + cbcl_internal_2year + cbcl_external_baseline + cbcl_external_2year + cbcl_totprob_baseline + cbcl_totprob_2year + upps_premeditation_baseline + upps_premeditation_2year + upps_perseverence_baseline + upps_perseverence_2year + upps_sensation_baseline + upps_sensation_2year + upps_negative_baseline + upps_negative_2year + upps_positive_baseline + upps_positive_2year + BIS_baseline + BIS_2year + BAS_RR_baseline + BAS_RR_2year + BAS_Drive_baseline + BAS_Drive_2year + BAS_Fun_baseline + BAS_Fun_2year + sleep_disturb_total_baseline + sleep_disturb_total_2year + ELS_total_baseline + ELS_total_2year + rh_adi_wsum1_baseline + rh_adi_wsum1_2year + rh_adi_wsum2_baseline + rh_adi_wsum2_2year + risk_alcohol_p_baseline + risk_cigarette_p_baseline + risk_electro_nicotine_p_baseline + risk_marijuana_p_baseline + risk_drug_p_baseline + risk_med_legal_p_baseline + risk_med_marijuana_p_baseline + risk_med_marijuana_pre_p_baseline + risk_family_med_marijuana_p_baseline + risk_alcohol_p_2year + risk_cigarette_p_2year + risk_electro_nicotine_p_2year + risk_marijuana_p_2year + risk_drug_p_2year + risk_med_legal_p_2year + risk_med_marijuana_p_2year + risk_med_marijuana_pre_p_2year + risk_family_med_marijuana_p_2year + risk_alcohol_y_2year + risk_cigarette_y_2year + risk_electro_nicotine_y_2year + risk_marijuana_y_2year + risk_med_legal_y_2year + risk_med_marijuana_y_2year + risk_med_marijuana_pre_y_2year + risk_family_med_marijuana_y_2year + risk_drug_y_2year + risk_gas_y_2year + risk_pain_y_2year + risk_anix_y_2year + risk_stimulant_y_2year + good_school_baseline + good_parent1_baseline + good_parent2_baseline + good_parent_baseline + good_school_2year + prenatal_tobacco_before_baseline + prenatal_alcohol_max_before_baseline + prenatal_alcohol_avg_before_baseline + prenatal_alcohol_eff_before_baseline + prenatal_marijuana_before_baseline + prenatal_cocaine_before_baseline + prenatal_oxycontin_before_baseline + prenatal_tobacco_after_baseline + prenatal_alcohol_max_after_baseline + prenatal_alcohol_avg_after_baseline + prenatal_alcohol_eff_after_baseline + prenatal_marijuana_after_baseline + prenatal_cocaine_after_baseline + prenatal_weeks_baseline + screentime_wkday_tv_baseline + screentime_wkday_videos_baseline + screentime_wkday_games_baseline + screentime_wkday_texting_baseline + screentime_wkday_sns_baseline + screentime_wkday_videochat_baseline + screentime_wkend_tv_baseline + screentime_wkend_videos_baseline + screentime_wkend_games_baseline + screentime_wkend_texting_baseline + screentime_wkend_sns_baseline + screentime_wkend_videochat_baseline + screentime_wkday_y_baseline + screentime_wkend_y_baseline + screentime_wkday_p_baseline + screentime_wkend_p_baseline + screentime_maturegames_baseline + screentime_rmovies_baseline + screentime_addict_p_2year + screentime_risk_p_2year + screentime_maturegames_2year + screentime_rmovies_2year + bpm_total_y_2year + bpm_total_t_baseline + bpm_total_t_2year + prosocial_y_baseline + prosocial_p_baseline + detention_baseline + detention_rea_baseline + friends_boys_baseline + friends_girls_baseline + Cfriends_boys_baseline + Cfriends_girls_baseline + friends_same_baseline + friends_diff_baseline + Cfriends_diff_baseline + Cfriends_same_baseline + prosocial_y_2year + prosocial_p_2year + friends_boys_2year + friends_girls_2year + Cfriends_boys_2year + Cfriends_girls_2year + friends_same_2year + friends_diff_2year + Cfriends_diff_2year + Cfriends_same_2year + cpeur2_baseline + eaeur1_baseline + depeur4_baseline + mddeur6_baseline + depmulti_baseline + bmieur4_baseline + bmimulti_baseline + insomniaeur6_baseline + snoringeur1_baseline + iqeur2_baseline + happieur4_baseline + ghappieur2_baseline + ghappimeaneur1_baseline + ghappihealth6_baseline + alcdep_eurauto_baseline + alcdep_afrauto_baseline + alcdep_metaauto_baseline + asdauto_baseline + aspauto_baseline + bipauto_baseline + cannabisauto_baseline + crossauto_baseline + drinkauto_baseline + edauto_baseline + neuroticismauto_baseline + ocdauto_baseline + risk4pcauto_baseline + risktolauto_baseline + scz_eurauto_baseline + scz_easauto_baseline + scz_metaauto_baseline + smokerauto_baseline + worryauto_baseline + anxietyauto_baseline + ptsdeur4_baseline + ptsdmeta6_baseline + adhdeur6_baseline + euro_baseline + sex_baseline + race_g_baseline + parent_identity_baseline + demo_brthdat_v2_baseline + demo_sex_v2_baseline + gender_identity_baseline + parent_age_baseline + foreign_born_family_baseline + married_baseline + high_educ_baseline + high_educ2_baseline + income_baseline + foreign_born_baseline + religion_prefer_baseline + gay_parent_baseline + gay_youth_baseline + race_ethnicity_baseline + age_baseline + family_adversity_baseline + height_baseline + weight_baseline + vol_baseline + bmi_baseline + total_ratio_baseline + history_ratio_baseline + euro_2year + race_g_2year + gay_youth_2year + age_2year + height_2year + weight_2year + vol_2year + bmi_2year', data=low_flanker_pos).fit()
model_high = ols('nihtbx_flanker_uncorrected_2year ~ KSADS_BD_y_baseline + KSADS_Anx_y_baseline + KSADS_Suicide_y_baseline + KSADS_Sleep_y_baseline + KSADS_total_y_baseline + KSADS_BD_y_2year + KSADS_Anx_y_2year + KSADS_Eat_y_2year + KSADS_Suicide_y_2year + KSADS_Sleep_y_2year + KSADS_total_y_2year + KSADS_BD_p_baseline + KSADS_Anx_p_baseline + KSADS_Eat_p_baseline + KSADS_Suicide_p_baseline + KSADS_Sleep_p_baseline + KSADS_total_p_baseline + KSADS_BD_p_2year + KSADS_Anx_p_2year + KSADS_Eat_p_2year + KSADS_Suicide_p_2year + KSADS_Sleep_p_2year + KSADS_total_p_2year + KSADS_Subst_p_baseline + KSADS_Subst_p_2year + cbcl_internal_baseline + cbcl_internal_2year + cbcl_external_baseline + cbcl_external_2year + cbcl_totprob_baseline + cbcl_totprob_2year + upps_premeditation_baseline + upps_premeditation_2year + upps_perseverence_baseline + upps_perseverence_2year + upps_sensation_baseline + upps_sensation_2year + upps_negative_baseline + upps_negative_2year + upps_positive_baseline + upps_positive_2year + BIS_baseline + BIS_2year + BAS_RR_baseline + BAS_RR_2year + BAS_Drive_baseline + BAS_Drive_2year + BAS_Fun_baseline + BAS_Fun_2year + sleep_disturb_total_baseline + sleep_disturb_total_2year + ELS_total_baseline + ELS_total_2year + rh_adi_wsum1_baseline + rh_adi_wsum1_2year + rh_adi_wsum2_baseline + rh_adi_wsum2_2year + risk_alcohol_p_baseline + risk_cigarette_p_baseline + risk_electro_nicotine_p_baseline + risk_marijuana_p_baseline + risk_drug_p_baseline + risk_med_legal_p_baseline + risk_med_marijuana_p_baseline + risk_med_marijuana_pre_p_baseline + risk_family_med_marijuana_p_baseline + risk_alcohol_p_2year + risk_cigarette_p_2year + risk_electro_nicotine_p_2year + risk_marijuana_p_2year + risk_drug_p_2year + risk_med_legal_p_2year + risk_med_marijuana_p_2year + risk_med_marijuana_pre_p_2year + risk_family_med_marijuana_p_2year + risk_alcohol_y_2year + risk_cigarette_y_2year + risk_electro_nicotine_y_2year + risk_marijuana_y_2year + risk_med_legal_y_2year + risk_med_marijuana_y_2year + risk_med_marijuana_pre_y_2year + risk_family_med_marijuana_y_2year + risk_drug_y_2year + risk_gas_y_2year + risk_pain_y_2year + risk_anix_y_2year + risk_stimulant_y_2year + good_school_baseline + good_parent1_baseline + good_parent2_baseline + good_parent_baseline + good_school_2year + prenatal_tobacco_before_baseline + prenatal_alcohol_max_before_baseline + prenatal_alcohol_avg_before_baseline + prenatal_alcohol_eff_before_baseline + prenatal_marijuana_before_baseline + prenatal_cocaine_before_baseline + prenatal_oxycontin_before_baseline + prenatal_tobacco_after_baseline + prenatal_alcohol_max_after_baseline + prenatal_alcohol_avg_after_baseline + prenatal_alcohol_eff_after_baseline + prenatal_marijuana_after_baseline + prenatal_cocaine_after_baseline + prenatal_weeks_baseline + screentime_wkday_tv_baseline + screentime_wkday_videos_baseline + screentime_wkday_games_baseline + screentime_wkday_texting_baseline + screentime_wkday_sns_baseline + screentime_wkday_videochat_baseline + screentime_wkend_tv_baseline + screentime_wkend_videos_baseline + screentime_wkend_games_baseline + screentime_wkend_texting_baseline + screentime_wkend_sns_baseline + screentime_wkend_videochat_baseline + screentime_wkday_y_baseline + screentime_wkend_y_baseline + screentime_wkday_p_baseline + screentime_wkend_p_baseline + screentime_maturegames_baseline + screentime_rmovies_baseline + screentime_addict_p_2year + screentime_risk_p_2year + screentime_maturegames_2year + screentime_rmovies_2year + bpm_total_y_2year + bpm_total_t_baseline + bpm_total_t_2year + prosocial_y_baseline + prosocial_p_baseline + detention_baseline + detention_rea_baseline + friends_boys_baseline + friends_girls_baseline + Cfriends_boys_baseline + Cfriends_girls_baseline + friends_same_baseline + friends_diff_baseline + Cfriends_diff_baseline + Cfriends_same_baseline + prosocial_y_2year + prosocial_p_2year + friends_boys_2year + friends_girls_2year + Cfriends_boys_2year + Cfriends_girls_2year + friends_same_2year + friends_diff_2year + Cfriends_diff_2year + Cfriends_same_2year + cpeur2_baseline + eaeur1_baseline + depeur4_baseline + mddeur6_baseline + depmulti_baseline + bmieur4_baseline + bmimulti_baseline + insomniaeur6_baseline + snoringeur1_baseline + iqeur2_baseline + happieur4_baseline + ghappieur2_baseline + ghappimeaneur1_baseline + ghappihealth6_baseline + alcdep_eurauto_baseline + alcdep_afrauto_baseline + alcdep_metaauto_baseline + asdauto_baseline + aspauto_baseline + bipauto_baseline + cannabisauto_baseline + crossauto_baseline + drinkauto_baseline + edauto_baseline + neuroticismauto_baseline + ocdauto_baseline + risk4pcauto_baseline + risktolauto_baseline + scz_eurauto_baseline + scz_easauto_baseline + scz_metaauto_baseline + smokerauto_baseline + worryauto_baseline + anxietyauto_baseline + ptsdeur4_baseline + ptsdmeta6_baseline + adhdeur6_baseline + euro_baseline + sex_baseline + race_g_baseline + parent_identity_baseline + demo_brthdat_v2_baseline + demo_sex_v2_baseline + gender_identity_baseline + parent_age_baseline + foreign_born_family_baseline + married_baseline + high_educ_baseline + high_educ2_baseline + income_baseline + foreign_born_baseline + religion_prefer_baseline + gay_parent_baseline + gay_youth_baseline + race_ethnicity_baseline + age_baseline + family_adversity_baseline + height_baseline + weight_baseline + vol_baseline + bmi_baseline + total_ratio_baseline + history_ratio_baseline + euro_2year + race_g_2year + gay_youth_2year + age_2year + height_2year + weight_2year + vol_2year + bmi_2year', data=high_flanker_pos).fit()

# Combine data for Chow Test
combined_data = pd.concat([low_flanker_pos, high_flanker_pos])
model_combined = ols('nihtbx_flanker_uncorrected_2year ~ KSADS_BD_y_baseline + KSADS_Anx_y_baseline + KSADS_Suicide_y_baseline + KSADS_Sleep_y_baseline + KSADS_total_y_baseline + KSADS_BD_y_2year + KSADS_Anx_y_2year + KSADS_Eat_y_2year + KSADS_Suicide_y_2year + KSADS_Sleep_y_2year + KSADS_total_y_2year + KSADS_BD_p_baseline + KSADS_Anx_p_baseline + KSADS_Eat_p_baseline + KSADS_Suicide_p_baseline + KSADS_Sleep_p_baseline + KSADS_total_p_baseline + KSADS_BD_p_2year + KSADS_Anx_p_2year + KSADS_Eat_p_2year + KSADS_Suicide_p_2year + KSADS_Sleep_p_2year + KSADS_total_p_2year + KSADS_Subst_p_baseline + KSADS_Subst_p_2year + cbcl_internal_baseline + cbcl_internal_2year + cbcl_external_baseline + cbcl_external_2year + cbcl_totprob_baseline + cbcl_totprob_2year + upps_premeditation_baseline + upps_premeditation_2year + upps_perseverence_baseline + upps_perseverence_2year + upps_sensation_baseline + upps_sensation_2year + upps_negative_baseline + upps_negative_2year + upps_positive_baseline + upps_positive_2year + BIS_baseline + BIS_2year + BAS_RR_baseline + BAS_RR_2year + BAS_Drive_baseline + BAS_Drive_2year + BAS_Fun_baseline + BAS_Fun_2year + sleep_disturb_total_baseline + sleep_disturb_total_2year + ELS_total_baseline + ELS_total_2year + rh_adi_wsum1_baseline + rh_adi_wsum1_2year + rh_adi_wsum2_baseline + rh_adi_wsum2_2year + risk_alcohol_p_baseline + risk_cigarette_p_baseline + risk_electro_nicotine_p_baseline + risk_marijuana_p_baseline + risk_drug_p_baseline + risk_med_legal_p_baseline + risk_med_marijuana_p_baseline + risk_med_marijuana_pre_p_baseline + risk_family_med_marijuana_p_baseline + risk_alcohol_p_2year + risk_cigarette_p_2year + risk_electro_nicotine_p_2year + risk_marijuana_p_2year + risk_drug_p_2year + risk_med_legal_p_2year + risk_med_marijuana_p_2year + risk_med_marijuana_pre_p_2year + risk_family_med_marijuana_p_2year + risk_alcohol_y_2year + risk_cigarette_y_2year + risk_electro_nicotine_y_2year + risk_marijuana_y_2year + risk_med_legal_y_2year + risk_med_marijuana_y_2year + risk_med_marijuana_pre_y_2year + risk_family_med_marijuana_y_2year + risk_drug_y_2year + risk_gas_y_2year + risk_pain_y_2year + risk_anix_y_2year + risk_stimulant_y_2year + good_school_baseline + good_parent1_baseline + good_parent2_baseline + good_parent_baseline + good_school_2year + prenatal_tobacco_before_baseline + prenatal_alcohol_max_before_baseline + prenatal_alcohol_avg_before_baseline + prenatal_alcohol_eff_before_baseline + prenatal_marijuana_before_baseline + prenatal_cocaine_before_baseline + prenatal_oxycontin_before_baseline + prenatal_tobacco_after_baseline + prenatal_alcohol_max_after_baseline + prenatal_alcohol_avg_after_baseline + prenatal_alcohol_eff_after_baseline + prenatal_marijuana_after_baseline + prenatal_cocaine_after_baseline + prenatal_weeks_baseline + screentime_wkday_tv_baseline + screentime_wkday_videos_baseline + screentime_wkday_games_baseline + screentime_wkday_texting_baseline + screentime_wkday_sns_baseline + screentime_wkday_videochat_baseline + screentime_wkend_tv_baseline + screentime_wkend_videos_baseline + screentime_wkend_games_baseline + screentime_wkend_texting_baseline + screentime_wkend_sns_baseline + screentime_wkend_videochat_baseline + screentime_wkday_y_baseline + screentime_wkend_y_baseline + screentime_wkday_p_baseline + screentime_wkend_p_baseline + screentime_maturegames_baseline + screentime_rmovies_baseline + screentime_addict_p_2year + screentime_risk_p_2year + screentime_maturegames_2year + screentime_rmovies_2year + bpm_total_y_2year + bpm_total_t_baseline + bpm_total_t_2year + prosocial_y_baseline + prosocial_p_baseline + detention_baseline + detention_rea_baseline + friends_boys_baseline + friends_girls_baseline + Cfriends_boys_baseline + Cfriends_girls_baseline + friends_same_baseline + friends_diff_baseline + Cfriends_diff_baseline + Cfriends_same_baseline + prosocial_y_2year + prosocial_p_2year + friends_boys_2year + friends_girls_2year + Cfriends_boys_2year + Cfriends_girls_2year + friends_same_2year + friends_diff_2year + Cfriends_diff_2year + Cfriends_same_2year + cpeur2_baseline + eaeur1_baseline + depeur4_baseline + mddeur6_baseline + depmulti_baseline + bmieur4_baseline + bmimulti_baseline + insomniaeur6_baseline + snoringeur1_baseline + iqeur2_baseline + happieur4_baseline + ghappieur2_baseline + ghappimeaneur1_baseline + ghappihealth6_baseline + alcdep_eurauto_baseline + alcdep_afrauto_baseline + alcdep_metaauto_baseline + asdauto_baseline + aspauto_baseline + bipauto_baseline + cannabisauto_baseline + crossauto_baseline + drinkauto_baseline + edauto_baseline + neuroticismauto_baseline + ocdauto_baseline + risk4pcauto_baseline + risktolauto_baseline + scz_eurauto_baseline + scz_easauto_baseline + scz_metaauto_baseline + smokerauto_baseline + worryauto_baseline + anxietyauto_baseline + ptsdeur4_baseline + ptsdmeta6_baseline + adhdeur6_baseline + euro_baseline + sex_baseline + race_g_baseline + parent_identity_baseline + demo_brthdat_v2_baseline + demo_sex_v2_baseline + gender_identity_baseline + parent_age_baseline + foreign_born_family_baseline + married_baseline + high_educ_baseline + high_educ2_baseline + income_baseline + foreign_born_baseline + religion_prefer_baseline + gay_parent_baseline + gay_youth_baseline + race_ethnicity_baseline + age_baseline + family_adversity_baseline + height_baseline + weight_baseline + vol_baseline + bmi_baseline + total_ratio_baseline + history_ratio_baseline + euro_2year + race_g_2year + gay_youth_2year + age_2year + height_2year + weight_2year + vol_2year + bmi_2year', data=combined_data).fit()

# Number of parameters in the model (excluding the intercept)
k = len(model_combined.params) - 1

# Chow Test formula
numerator = (model_combined.ssr - (model_low.ssr + model_high.ssr)) / k
denominator = (model_low.ssr + model_high.ssr) / (len(df_low) + len(df_high) - 2 * k)
f_statistic = numerator / denominator
p_value = stats.f.sf(f_statistic, k, len(df_low) + len(df_high) - 2 * k)  # Using survival function for p-value

print("F-statistic:", f_statistic)
print("P-value:", p_value)

coef_low = model_low.params
p_low = model_low.pvalues

# Filter coefficients with p-values less than 0.05
sig_coef_low = coef_low[p_low < 0.05].index

coef_combined_low = model_combined.params[sig_coef_low]

coefficient_differences = abs(coef_combined_low - coef_low[sig_coef_low])

diff_low_df = pd.DataFrame({'Difference': coefficient_differences, 'Coefficients': coef_low[sig_coef_low], 'P-values': p_low[sig_coef_low]}).sort_values(by='Difference', ascending=False)

print(diff_low_df)

coef_high = model_high.params
p_high = model_high.pvalues

# Filter coefficients with p-values less than 0.05
sig_coef_high = coef_high[p_high < 0.05].index

coef_combined_high = model_combined.params[sig_coef_high]

coefficient_differences = abs(coef_combined_high - coef_high[sig_coef_high])

diff_high_df = pd.DataFrame({'Difference': coefficient_differences, 'Coefficients': coef_high[sig_coef_high], 'P-values': p_high[sig_coef_high]}).sort_values(by='Difference', ascending=False)

print(diff_high_df)

In [ ]:
# nan_columns_low = low_pattern_neg.columns[low_pattern_neg.isna().any()].tolist()
# print(nan_columns_low)
# nan_columns_high = high_pattern_neg.columns[high_pattern_neg.isna().any()].tolist()
# print(nan_columns_high)
# remove KSADS_Subst_p_baseline, prenatal_heroin_before_baseline, prenatal_heroin_after_baseline, prenatal_oxycontin_before_baseline, prenatal_oxycontin_after_baseline, prenatal_cocaine_before_baseline, prenatal_cocaine_after_baseline

In [ ]:
model_low = ols('nihtbx_pattern_uncorrected_2year ~ KSADS_BD_y_baseline + KSADS_Anx_y_baseline + KSADS_Suicide_y_baseline + KSADS_Sleep_y_baseline + KSADS_total_y_baseline + KSADS_BD_y_2year + KSADS_Anx_y_2year + KSADS_Eat_y_2year + KSADS_Suicide_y_2year + KSADS_Sleep_y_2year + KSADS_total_y_2year + KSADS_BD_p_baseline + KSADS_Anx_p_baseline + KSADS_Eat_p_baseline + KSADS_Suicide_p_baseline + KSADS_Sleep_p_baseline + KSADS_total_p_baseline + KSADS_BD_p_2year + KSADS_Anx_p_2year + KSADS_Eat_p_2year + KSADS_Suicide_p_2year + KSADS_Sleep_p_2year + KSADS_total_p_2year + KSADS_Subst_p_2year + cbcl_internal_baseline + cbcl_internal_2year + cbcl_external_baseline + cbcl_external_2year + cbcl_totprob_baseline + cbcl_totprob_2year + upps_premeditation_baseline + upps_premeditation_2year + upps_perseverence_baseline + upps_perseverence_2year + upps_sensation_baseline + upps_sensation_2year + upps_negative_baseline + upps_negative_2year + upps_positive_baseline + upps_positive_2year + BIS_baseline + BIS_2year + BAS_RR_baseline + BAS_RR_2year + BAS_Drive_baseline + BAS_Drive_2year + BAS_Fun_baseline + BAS_Fun_2year + sleep_disturb_total_baseline + sleep_disturb_total_2year + ELS_total_baseline + ELS_total_2year + rh_adi_wsum1_baseline + rh_adi_wsum1_2year + rh_adi_wsum2_baseline + rh_adi_wsum2_2year + risk_alcohol_p_baseline + risk_cigarette_p_baseline + risk_electro_nicotine_p_baseline + risk_marijuana_p_baseline + risk_drug_p_baseline + risk_med_legal_p_baseline + risk_med_marijuana_p_baseline + risk_med_marijuana_pre_p_baseline + risk_family_med_marijuana_p_baseline + risk_alcohol_p_2year + risk_cigarette_p_2year + risk_electro_nicotine_p_2year + risk_marijuana_p_2year + risk_drug_p_2year + risk_med_legal_p_2year + risk_med_marijuana_p_2year + risk_med_marijuana_pre_p_2year + risk_family_med_marijuana_p_2year + risk_alcohol_y_2year + risk_cigarette_y_2year + risk_electro_nicotine_y_2year + risk_marijuana_y_2year + risk_med_legal_y_2year + risk_med_marijuana_y_2year + risk_med_marijuana_pre_y_2year + risk_family_med_marijuana_y_2year + risk_drug_y_2year + risk_gas_y_2year + risk_pain_y_2year + risk_anix_y_2year + risk_stimulant_y_2year + good_school_baseline + good_parent1_baseline + good_parent2_baseline + good_parent_baseline + good_school_2year + prenatal_tobacco_before_baseline + prenatal_alcohol_max_before_baseline + prenatal_alcohol_avg_before_baseline + prenatal_alcohol_eff_before_baseline + prenatal_marijuana_before_baseline + prenatal_tobacco_after_baseline + prenatal_alcohol_max_after_baseline + prenatal_alcohol_avg_after_baseline + prenatal_alcohol_eff_after_baseline + prenatal_marijuana_after_baseline + prenatal_weeks_baseline + screentime_wkday_tv_baseline + screentime_wkday_videos_baseline + screentime_wkday_games_baseline + screentime_wkday_texting_baseline + screentime_wkday_sns_baseline + screentime_wkday_videochat_baseline + screentime_wkend_tv_baseline + screentime_wkend_videos_baseline + screentime_wkend_games_baseline + screentime_wkend_texting_baseline + screentime_wkend_sns_baseline + screentime_wkend_videochat_baseline + screentime_wkday_y_baseline + screentime_wkend_y_baseline + screentime_wkday_p_baseline + screentime_wkend_p_baseline + screentime_maturegames_baseline + screentime_rmovies_baseline + screentime_addict_p_2year + screentime_risk_p_2year + screentime_maturegames_2year + screentime_rmovies_2year + bpm_total_y_2year + bpm_total_t_baseline + bpm_total_t_2year + prosocial_y_baseline + prosocial_p_baseline + detention_baseline + detention_rea_baseline + friends_boys_baseline + friends_girls_baseline + Cfriends_boys_baseline + Cfriends_girls_baseline + friends_same_baseline + friends_diff_baseline + Cfriends_diff_baseline + Cfriends_same_baseline + prosocial_y_2year + prosocial_p_2year + friends_boys_2year + friends_girls_2year + Cfriends_boys_2year + Cfriends_girls_2year + friends_same_2year + friends_diff_2year + Cfriends_diff_2year + Cfriends_same_2year + cpeur2_baseline + eaeur1_baseline + depeur4_baseline + mddeur6_baseline + depmulti_baseline + bmieur4_baseline + bmimulti_baseline + insomniaeur6_baseline + snoringeur1_baseline + iqeur2_baseline + happieur4_baseline + ghappieur2_baseline + ghappimeaneur1_baseline + ghappihealth6_baseline + alcdep_eurauto_baseline + alcdep_afrauto_baseline + alcdep_metaauto_baseline + asdauto_baseline + aspauto_baseline + bipauto_baseline + cannabisauto_baseline + crossauto_baseline + drinkauto_baseline + edauto_baseline + neuroticismauto_baseline + ocdauto_baseline + risk4pcauto_baseline + risktolauto_baseline + scz_eurauto_baseline + scz_easauto_baseline + scz_metaauto_baseline + smokerauto_baseline + worryauto_baseline + anxietyauto_baseline + ptsdeur4_baseline + ptsdmeta6_baseline + adhdeur6_baseline + euro_baseline + sex_baseline + race_g_baseline + parent_identity_baseline + demo_brthdat_v2_baseline + demo_sex_v2_baseline + gender_identity_baseline + parent_age_baseline + foreign_born_family_baseline + married_baseline + high_educ_baseline + high_educ2_baseline + income_baseline + foreign_born_baseline + religion_prefer_baseline + gay_parent_baseline + gay_youth_baseline + race_ethnicity_baseline + age_baseline + family_adversity_baseline + height_baseline + weight_baseline + vol_baseline + bmi_baseline + total_ratio_baseline + history_ratio_baseline + euro_2year + race_g_2year + gay_youth_2year + age_2year + height_2year + weight_2year + vol_2year + bmi_2year', data=low_pattern_neg).fit()
model_high = ols('nihtbx_pattern_uncorrected_2year ~ KSADS_BD_y_baseline + KSADS_Anx_y_baseline + KSADS_Suicide_y_baseline + KSADS_Sleep_y_baseline + KSADS_total_y_baseline + KSADS_BD_y_2year + KSADS_Anx_y_2year + KSADS_Eat_y_2year + KSADS_Suicide_y_2year + KSADS_Sleep_y_2year + KSADS_total_y_2year + KSADS_BD_p_baseline + KSADS_Anx_p_baseline + KSADS_Eat_p_baseline + KSADS_Suicide_p_baseline + KSADS_Sleep_p_baseline + KSADS_total_p_baseline + KSADS_BD_p_2year + KSADS_Anx_p_2year + KSADS_Eat_p_2year + KSADS_Suicide_p_2year + KSADS_Sleep_p_2year + KSADS_total_p_2year + KSADS_Subst_p_2year + cbcl_internal_baseline + cbcl_internal_2year + cbcl_external_baseline + cbcl_external_2year + cbcl_totprob_baseline + cbcl_totprob_2year + upps_premeditation_baseline + upps_premeditation_2year + upps_perseverence_baseline + upps_perseverence_2year + upps_sensation_baseline + upps_sensation_2year + upps_negative_baseline + upps_negative_2year + upps_positive_baseline + upps_positive_2year + BIS_baseline + BIS_2year + BAS_RR_baseline + BAS_RR_2year + BAS_Drive_baseline + BAS_Drive_2year + BAS_Fun_baseline + BAS_Fun_2year + sleep_disturb_total_baseline + sleep_disturb_total_2year + ELS_total_baseline + ELS_total_2year + rh_adi_wsum1_baseline + rh_adi_wsum1_2year + rh_adi_wsum2_baseline + rh_adi_wsum2_2year + risk_alcohol_p_baseline + risk_cigarette_p_baseline + risk_electro_nicotine_p_baseline + risk_marijuana_p_baseline + risk_drug_p_baseline + risk_med_legal_p_baseline + risk_med_marijuana_p_baseline + risk_med_marijuana_pre_p_baseline + risk_family_med_marijuana_p_baseline + risk_alcohol_p_2year + risk_cigarette_p_2year + risk_electro_nicotine_p_2year + risk_marijuana_p_2year + risk_drug_p_2year + risk_med_legal_p_2year + risk_med_marijuana_p_2year + risk_med_marijuana_pre_p_2year + risk_family_med_marijuana_p_2year + risk_alcohol_y_2year + risk_cigarette_y_2year + risk_electro_nicotine_y_2year + risk_marijuana_y_2year + risk_med_legal_y_2year + risk_med_marijuana_y_2year + risk_med_marijuana_pre_y_2year + risk_family_med_marijuana_y_2year + risk_drug_y_2year + risk_gas_y_2year + risk_pain_y_2year + risk_anix_y_2year + risk_stimulant_y_2year + good_school_baseline + good_parent1_baseline + good_parent2_baseline + good_parent_baseline + good_school_2year + prenatal_tobacco_before_baseline + prenatal_alcohol_max_before_baseline + prenatal_alcohol_avg_before_baseline + prenatal_alcohol_eff_before_baseline + prenatal_marijuana_before_baseline + prenatal_tobacco_after_baseline + prenatal_alcohol_max_after_baseline + prenatal_alcohol_avg_after_baseline + prenatal_alcohol_eff_after_baseline + prenatal_marijuana_after_baseline + prenatal_weeks_baseline + screentime_wkday_tv_baseline + screentime_wkday_videos_baseline + screentime_wkday_games_baseline + screentime_wkday_texting_baseline + screentime_wkday_sns_baseline + screentime_wkday_videochat_baseline + screentime_wkend_tv_baseline + screentime_wkend_videos_baseline + screentime_wkend_games_baseline + screentime_wkend_texting_baseline + screentime_wkend_sns_baseline + screentime_wkend_videochat_baseline + screentime_wkday_y_baseline + screentime_wkend_y_baseline + screentime_wkday_p_baseline + screentime_wkend_p_baseline + screentime_maturegames_baseline + screentime_rmovies_baseline + screentime_addict_p_2year + screentime_risk_p_2year + screentime_maturegames_2year + screentime_rmovies_2year + bpm_total_y_2year + bpm_total_t_baseline + bpm_total_t_2year + prosocial_y_baseline + prosocial_p_baseline + detention_baseline + detention_rea_baseline + friends_boys_baseline + friends_girls_baseline + Cfriends_boys_baseline + Cfriends_girls_baseline + friends_same_baseline + friends_diff_baseline + Cfriends_diff_baseline + Cfriends_same_baseline + prosocial_y_2year + prosocial_p_2year + friends_boys_2year + friends_girls_2year + Cfriends_boys_2year + Cfriends_girls_2year + friends_same_2year + friends_diff_2year + Cfriends_diff_2year + Cfriends_same_2year + cpeur2_baseline + eaeur1_baseline + depeur4_baseline + mddeur6_baseline + depmulti_baseline + bmieur4_baseline + bmimulti_baseline + insomniaeur6_baseline + snoringeur1_baseline + iqeur2_baseline + happieur4_baseline + ghappieur2_baseline + ghappimeaneur1_baseline + ghappihealth6_baseline + alcdep_eurauto_baseline + alcdep_afrauto_baseline + alcdep_metaauto_baseline + asdauto_baseline + aspauto_baseline + bipauto_baseline + cannabisauto_baseline + crossauto_baseline + drinkauto_baseline + edauto_baseline + neuroticismauto_baseline + ocdauto_baseline + risk4pcauto_baseline + risktolauto_baseline + scz_eurauto_baseline + scz_easauto_baseline + scz_metaauto_baseline + smokerauto_baseline + worryauto_baseline + anxietyauto_baseline + ptsdeur4_baseline + ptsdmeta6_baseline + adhdeur6_baseline + euro_baseline + sex_baseline + race_g_baseline + parent_identity_baseline + demo_brthdat_v2_baseline + demo_sex_v2_baseline + gender_identity_baseline + parent_age_baseline + foreign_born_family_baseline + married_baseline + high_educ_baseline + high_educ2_baseline + income_baseline + foreign_born_baseline + religion_prefer_baseline + gay_parent_baseline + gay_youth_baseline + race_ethnicity_baseline + age_baseline + family_adversity_baseline + height_baseline + weight_baseline + vol_baseline + bmi_baseline + total_ratio_baseline + history_ratio_baseline + euro_2year + race_g_2year + gay_youth_2year + age_2year + height_2year + weight_2year + vol_2year + bmi_2year', data=high_pattern_neg).fit()

# Combine data for Chow Test
combined_data = pd.concat([low_pattern_neg, high_pattern_neg])
model_combined = ols('nihtbx_pattern_uncorrected_2year ~ KSADS_BD_y_baseline + KSADS_Anx_y_baseline + KSADS_Suicide_y_baseline + KSADS_Sleep_y_baseline + KSADS_total_y_baseline + KSADS_BD_y_2year + KSADS_Anx_y_2year + KSADS_Eat_y_2year + KSADS_Suicide_y_2year + KSADS_Sleep_y_2year + KSADS_total_y_2year + KSADS_BD_p_baseline + KSADS_Anx_p_baseline + KSADS_Eat_p_baseline + KSADS_Suicide_p_baseline + KSADS_Sleep_p_baseline + KSADS_total_p_baseline + KSADS_BD_p_2year + KSADS_Anx_p_2year + KSADS_Eat_p_2year + KSADS_Suicide_p_2year + KSADS_Sleep_p_2year + KSADS_total_p_2year + KSADS_Subst_p_2year + cbcl_internal_baseline + cbcl_internal_2year + cbcl_external_baseline + cbcl_external_2year + cbcl_totprob_baseline + cbcl_totprob_2year + upps_premeditation_baseline + upps_premeditation_2year + upps_perseverence_baseline + upps_perseverence_2year + upps_sensation_baseline + upps_sensation_2year + upps_negative_baseline + upps_negative_2year + upps_positive_baseline + upps_positive_2year + BIS_baseline + BIS_2year + BAS_RR_baseline + BAS_RR_2year + BAS_Drive_baseline + BAS_Drive_2year + BAS_Fun_baseline + BAS_Fun_2year + sleep_disturb_total_baseline + sleep_disturb_total_2year + ELS_total_baseline + ELS_total_2year + rh_adi_wsum1_baseline + rh_adi_wsum1_2year + rh_adi_wsum2_baseline + rh_adi_wsum2_2year + risk_alcohol_p_baseline + risk_cigarette_p_baseline + risk_electro_nicotine_p_baseline + risk_marijuana_p_baseline + risk_drug_p_baseline + risk_med_legal_p_baseline + risk_med_marijuana_p_baseline + risk_med_marijuana_pre_p_baseline + risk_family_med_marijuana_p_baseline + risk_alcohol_p_2year + risk_cigarette_p_2year + risk_electro_nicotine_p_2year + risk_marijuana_p_2year + risk_drug_p_2year + risk_med_legal_p_2year + risk_med_marijuana_p_2year + risk_med_marijuana_pre_p_2year + risk_family_med_marijuana_p_2year + risk_alcohol_y_2year + risk_cigarette_y_2year + risk_electro_nicotine_y_2year + risk_marijuana_y_2year + risk_med_legal_y_2year + risk_med_marijuana_y_2year + risk_med_marijuana_pre_y_2year + risk_family_med_marijuana_y_2year + risk_drug_y_2year + risk_gas_y_2year + risk_pain_y_2year + risk_anix_y_2year + risk_stimulant_y_2year + good_school_baseline + good_parent1_baseline + good_parent2_baseline + good_parent_baseline + good_school_2year + prenatal_tobacco_before_baseline + prenatal_alcohol_max_before_baseline + prenatal_alcohol_avg_before_baseline + prenatal_alcohol_eff_before_baseline + prenatal_marijuana_before_baseline + prenatal_tobacco_after_baseline + prenatal_alcohol_max_after_baseline + prenatal_alcohol_avg_after_baseline + prenatal_alcohol_eff_after_baseline + prenatal_marijuana_after_baseline + prenatal_weeks_baseline + screentime_wkday_tv_baseline + screentime_wkday_videos_baseline + screentime_wkday_games_baseline + screentime_wkday_texting_baseline + screentime_wkday_sns_baseline + screentime_wkday_videochat_baseline + screentime_wkend_tv_baseline + screentime_wkend_videos_baseline + screentime_wkend_games_baseline + screentime_wkend_texting_baseline + screentime_wkend_sns_baseline + screentime_wkend_videochat_baseline + screentime_wkday_y_baseline + screentime_wkend_y_baseline + screentime_wkday_p_baseline + screentime_wkend_p_baseline + screentime_maturegames_baseline + screentime_rmovies_baseline + screentime_addict_p_2year + screentime_risk_p_2year + screentime_maturegames_2year + screentime_rmovies_2year + bpm_total_y_2year + bpm_total_t_baseline + bpm_total_t_2year + prosocial_y_baseline + prosocial_p_baseline + detention_baseline + detention_rea_baseline + friends_boys_baseline + friends_girls_baseline + Cfriends_boys_baseline + Cfriends_girls_baseline + friends_same_baseline + friends_diff_baseline + Cfriends_diff_baseline + Cfriends_same_baseline + prosocial_y_2year + prosocial_p_2year + friends_boys_2year + friends_girls_2year + Cfriends_boys_2year + Cfriends_girls_2year + friends_same_2year + friends_diff_2year + Cfriends_diff_2year + Cfriends_same_2year + cpeur2_baseline + eaeur1_baseline + depeur4_baseline + mddeur6_baseline + depmulti_baseline + bmieur4_baseline + bmimulti_baseline + insomniaeur6_baseline + snoringeur1_baseline + iqeur2_baseline + happieur4_baseline + ghappieur2_baseline + ghappimeaneur1_baseline + ghappihealth6_baseline + alcdep_eurauto_baseline + alcdep_afrauto_baseline + alcdep_metaauto_baseline + asdauto_baseline + aspauto_baseline + bipauto_baseline + cannabisauto_baseline + crossauto_baseline + drinkauto_baseline + edauto_baseline + neuroticismauto_baseline + ocdauto_baseline + risk4pcauto_baseline + risktolauto_baseline + scz_eurauto_baseline + scz_easauto_baseline + scz_metaauto_baseline + smokerauto_baseline + worryauto_baseline + anxietyauto_baseline + ptsdeur4_baseline + ptsdmeta6_baseline + adhdeur6_baseline + euro_baseline + sex_baseline + race_g_baseline + parent_identity_baseline + demo_brthdat_v2_baseline + demo_sex_v2_baseline + gender_identity_baseline + parent_age_baseline + foreign_born_family_baseline + married_baseline + high_educ_baseline + high_educ2_baseline + income_baseline + foreign_born_baseline + religion_prefer_baseline + gay_parent_baseline + gay_youth_baseline + race_ethnicity_baseline + age_baseline + family_adversity_baseline + height_baseline + weight_baseline + vol_baseline + bmi_baseline + total_ratio_baseline + history_ratio_baseline + euro_2year + race_g_2year + gay_youth_2year + age_2year + height_2year + weight_2year + vol_2year + bmi_2year', data=combined_data).fit()

# Number of parameters in the model (excluding the intercept)
k = len(model_combined.params) - 1

# Chow Test formula
numerator = (model_combined.ssr - (model_low.ssr + model_high.ssr)) / k
denominator = (model_low.ssr + model_high.ssr) / (len(df_low) + len(df_high) - 2 * k)
f_statistic = numerator / denominator
p_value = stats.f.sf(f_statistic, k, len(df_low) + len(df_high) - 2 * k)  # Using survival function for p-value

print("F-statistic:", f_statistic)
print("P-value:", p_value)

coef_low = model_low.params
p_low = model_low.pvalues

# Filter coefficients with p-values less than 0.05
sig_coef_low = coef_low[p_low < 0.05].index

coef_combined_low = model_combined.params[sig_coef_low]

coefficient_differences = abs(coef_combined_low - coef_low[sig_coef_low])

diff_low_df = pd.DataFrame({'Difference': coefficient_differences, 'Coefficients': coef_low[sig_coef_low], 'P-values': p_low[sig_coef_low]}).sort_values(by='Difference', ascending=False)

print(diff_low_df)

coef_high = model_high.params
p_high = model_high.pvalues

# Filter coefficients with p-values less than 0.05
sig_coef_high = coef_high[p_high < 0.05].index

coef_combined_high = model_combined.params[sig_coef_high]

coefficient_differences = abs(coef_combined_high - coef_high[sig_coef_high])

diff_high_df = pd.DataFrame({'Difference': coefficient_differences, 'Coefficients': coef_high[sig_coef_high], 'P-values': p_high[sig_coef_high]}).sort_values(by='Difference', ascending=False)

print(diff_high_df)

In [ ]:
nan_columns_low = low_pattern_pos.columns[low_pattern_pos.isna().any()].tolist()
print(nan_columns_low)
nan_columns_high = high_pattern_pos.columns[high_pattern_pos.isna().any()].tolist()
print(nan_columns_high)
# remove prenatal_heroin_before_baseline, prenatal_heroin_after_baseline, prenatal_oxycontin_after_baseline

In [ ]:
model_low = ols('nihtbx_pattern_uncorrected_2year ~ KSADS_BD_y_baseline + KSADS_Anx_y_baseline + KSADS_Suicide_y_baseline + KSADS_Sleep_y_baseline + KSADS_total_y_baseline + KSADS_BD_y_2year + KSADS_Anx_y_2year + KSADS_Eat_y_2year + KSADS_Suicide_y_2year + KSADS_Sleep_y_2year + KSADS_total_y_2year + KSADS_BD_p_baseline + KSADS_Anx_p_baseline + KSADS_Eat_p_baseline + KSADS_Suicide_p_baseline + KSADS_Sleep_p_baseline + KSADS_total_p_baseline + KSADS_BD_p_2year + KSADS_Anx_p_2year + KSADS_Eat_p_2year + KSADS_Suicide_p_2year + KSADS_Sleep_p_2year + KSADS_total_p_2year + KSADS_Subst_p_baseline + KSADS_Subst_p_2year + cbcl_internal_baseline + cbcl_internal_2year + cbcl_external_baseline + cbcl_external_2year + cbcl_totprob_baseline + cbcl_totprob_2year + upps_premeditation_baseline + upps_premeditation_2year + upps_perseverence_baseline + upps_perseverence_2year + upps_sensation_baseline + upps_sensation_2year + upps_negative_baseline + upps_negative_2year + upps_positive_baseline + upps_positive_2year + BIS_baseline + BIS_2year + BAS_RR_baseline + BAS_RR_2year + BAS_Drive_baseline + BAS_Drive_2year + BAS_Fun_baseline + BAS_Fun_2year + sleep_disturb_total_baseline + sleep_disturb_total_2year + ELS_total_baseline + ELS_total_2year + rh_adi_wsum1_baseline + rh_adi_wsum1_2year + rh_adi_wsum2_baseline + rh_adi_wsum2_2year + risk_alcohol_p_baseline + risk_cigarette_p_baseline + risk_electro_nicotine_p_baseline + risk_marijuana_p_baseline + risk_drug_p_baseline + risk_med_legal_p_baseline + risk_med_marijuana_p_baseline + risk_med_marijuana_pre_p_baseline + risk_family_med_marijuana_p_baseline + risk_alcohol_p_2year + risk_cigarette_p_2year + risk_electro_nicotine_p_2year + risk_marijuana_p_2year + risk_drug_p_2year + risk_med_legal_p_2year + risk_med_marijuana_p_2year + risk_med_marijuana_pre_p_2year + risk_family_med_marijuana_p_2year + risk_alcohol_y_2year + risk_cigarette_y_2year + risk_electro_nicotine_y_2year + risk_marijuana_y_2year + risk_med_legal_y_2year + risk_med_marijuana_y_2year + risk_med_marijuana_pre_y_2year + risk_family_med_marijuana_y_2year + risk_drug_y_2year + risk_gas_y_2year + risk_pain_y_2year + risk_anix_y_2year + risk_stimulant_y_2year + good_school_baseline + good_parent1_baseline + good_parent2_baseline + good_parent_baseline + good_school_2year + prenatal_tobacco_before_baseline + prenatal_alcohol_max_before_baseline + prenatal_alcohol_avg_before_baseline + prenatal_alcohol_eff_before_baseline + prenatal_marijuana_before_baseline + prenatal_cocaine_before_baseline + prenatal_oxycontin_before_baseline + prenatal_tobacco_after_baseline + prenatal_alcohol_max_after_baseline + prenatal_alcohol_avg_after_baseline + prenatal_alcohol_eff_after_baseline + prenatal_marijuana_after_baseline + prenatal_cocaine_after_baseline + prenatal_weeks_baseline + screentime_wkday_tv_baseline + screentime_wkday_videos_baseline + screentime_wkday_games_baseline + screentime_wkday_texting_baseline + screentime_wkday_sns_baseline + screentime_wkday_videochat_baseline + screentime_wkend_tv_baseline + screentime_wkend_videos_baseline + screentime_wkend_games_baseline + screentime_wkend_texting_baseline + screentime_wkend_sns_baseline + screentime_wkend_videochat_baseline + screentime_wkday_y_baseline + screentime_wkend_y_baseline + screentime_wkday_p_baseline + screentime_wkend_p_baseline + screentime_maturegames_baseline + screentime_rmovies_baseline + screentime_addict_p_2year + screentime_risk_p_2year + screentime_maturegames_2year + screentime_rmovies_2year + bpm_total_y_2year + bpm_total_t_baseline + bpm_total_t_2year + prosocial_y_baseline + prosocial_p_baseline + detention_baseline + detention_rea_baseline + friends_boys_baseline + friends_girls_baseline + Cfriends_boys_baseline + Cfriends_girls_baseline + friends_same_baseline + friends_diff_baseline + Cfriends_diff_baseline + Cfriends_same_baseline + prosocial_y_2year + prosocial_p_2year + friends_boys_2year + friends_girls_2year + Cfriends_boys_2year + Cfriends_girls_2year + friends_same_2year + friends_diff_2year + Cfriends_diff_2year + Cfriends_same_2year + cpeur2_baseline + eaeur1_baseline + depeur4_baseline + mddeur6_baseline + depmulti_baseline + bmieur4_baseline + bmimulti_baseline + insomniaeur6_baseline + snoringeur1_baseline + iqeur2_baseline + happieur4_baseline + ghappieur2_baseline + ghappimeaneur1_baseline + ghappihealth6_baseline + alcdep_eurauto_baseline + alcdep_afrauto_baseline + alcdep_metaauto_baseline + asdauto_baseline + aspauto_baseline + bipauto_baseline + cannabisauto_baseline + crossauto_baseline + drinkauto_baseline + edauto_baseline + neuroticismauto_baseline + ocdauto_baseline + risk4pcauto_baseline + risktolauto_baseline + scz_eurauto_baseline + scz_easauto_baseline + scz_metaauto_baseline + smokerauto_baseline + worryauto_baseline + anxietyauto_baseline + ptsdeur4_baseline + ptsdmeta6_baseline + adhdeur6_baseline + euro_baseline + sex_baseline + race_g_baseline + parent_identity_baseline + demo_brthdat_v2_baseline + demo_sex_v2_baseline + gender_identity_baseline + parent_age_baseline + foreign_born_family_baseline + married_baseline + high_educ_baseline + high_educ2_baseline + income_baseline + foreign_born_baseline + religion_prefer_baseline + gay_parent_baseline + gay_youth_baseline + race_ethnicity_baseline + age_baseline + family_adversity_baseline + height_baseline + weight_baseline + vol_baseline + bmi_baseline + total_ratio_baseline + history_ratio_baseline + euro_2year + race_g_2year + gay_youth_2year + age_2year + height_2year + weight_2year + vol_2year + bmi_2year', data=low_pattern_pos).fit()
model_high = ols('nihtbx_pattern_uncorrected_2year ~ KSADS_BD_y_baseline + KSADS_Anx_y_baseline + KSADS_Suicide_y_baseline + KSADS_Sleep_y_baseline + KSADS_total_y_baseline + KSADS_BD_y_2year + KSADS_Anx_y_2year + KSADS_Eat_y_2year + KSADS_Suicide_y_2year + KSADS_Sleep_y_2year + KSADS_total_y_2year + KSADS_BD_p_baseline + KSADS_Anx_p_baseline + KSADS_Eat_p_baseline + KSADS_Suicide_p_baseline + KSADS_Sleep_p_baseline + KSADS_total_p_baseline + KSADS_BD_p_2year + KSADS_Anx_p_2year + KSADS_Eat_p_2year + KSADS_Suicide_p_2year + KSADS_Sleep_p_2year + KSADS_total_p_2year + KSADS_Subst_p_baseline + KSADS_Subst_p_2year + cbcl_internal_baseline + cbcl_internal_2year + cbcl_external_baseline + cbcl_external_2year + cbcl_totprob_baseline + cbcl_totprob_2year + upps_premeditation_baseline + upps_premeditation_2year + upps_perseverence_baseline + upps_perseverence_2year + upps_sensation_baseline + upps_sensation_2year + upps_negative_baseline + upps_negative_2year + upps_positive_baseline + upps_positive_2year + BIS_baseline + BIS_2year + BAS_RR_baseline + BAS_RR_2year + BAS_Drive_baseline + BAS_Drive_2year + BAS_Fun_baseline + BAS_Fun_2year + sleep_disturb_total_baseline + sleep_disturb_total_2year + ELS_total_baseline + ELS_total_2year + rh_adi_wsum1_baseline + rh_adi_wsum1_2year + rh_adi_wsum2_baseline + rh_adi_wsum2_2year + risk_alcohol_p_baseline + risk_cigarette_p_baseline + risk_electro_nicotine_p_baseline + risk_marijuana_p_baseline + risk_drug_p_baseline + risk_med_legal_p_baseline + risk_med_marijuana_p_baseline + risk_med_marijuana_pre_p_baseline + risk_family_med_marijuana_p_baseline + risk_alcohol_p_2year + risk_cigarette_p_2year + risk_electro_nicotine_p_2year + risk_marijuana_p_2year + risk_drug_p_2year + risk_med_legal_p_2year + risk_med_marijuana_p_2year + risk_med_marijuana_pre_p_2year + risk_family_med_marijuana_p_2year + risk_alcohol_y_2year + risk_cigarette_y_2year + risk_electro_nicotine_y_2year + risk_marijuana_y_2year + risk_med_legal_y_2year + risk_med_marijuana_y_2year + risk_med_marijuana_pre_y_2year + risk_family_med_marijuana_y_2year + risk_drug_y_2year + risk_gas_y_2year + risk_pain_y_2year + risk_anix_y_2year + risk_stimulant_y_2year + good_school_baseline + good_parent1_baseline + good_parent2_baseline + good_parent_baseline + good_school_2year + prenatal_tobacco_before_baseline + prenatal_alcohol_max_before_baseline + prenatal_alcohol_avg_before_baseline + prenatal_alcohol_eff_before_baseline + prenatal_marijuana_before_baseline + prenatal_cocaine_before_baseline + prenatal_oxycontin_before_baseline + prenatal_tobacco_after_baseline + prenatal_alcohol_max_after_baseline + prenatal_alcohol_avg_after_baseline + prenatal_alcohol_eff_after_baseline + prenatal_marijuana_after_baseline + prenatal_cocaine_after_baseline + prenatal_weeks_baseline + screentime_wkday_tv_baseline + screentime_wkday_videos_baseline + screentime_wkday_games_baseline + screentime_wkday_texting_baseline + screentime_wkday_sns_baseline + screentime_wkday_videochat_baseline + screentime_wkend_tv_baseline + screentime_wkend_videos_baseline + screentime_wkend_games_baseline + screentime_wkend_texting_baseline + screentime_wkend_sns_baseline + screentime_wkend_videochat_baseline + screentime_wkday_y_baseline + screentime_wkend_y_baseline + screentime_wkday_p_baseline + screentime_wkend_p_baseline + screentime_maturegames_baseline + screentime_rmovies_baseline + screentime_addict_p_2year + screentime_risk_p_2year + screentime_maturegames_2year + screentime_rmovies_2year + bpm_total_y_2year + bpm_total_t_baseline + bpm_total_t_2year + prosocial_y_baseline + prosocial_p_baseline + detention_baseline + detention_rea_baseline + friends_boys_baseline + friends_girls_baseline + Cfriends_boys_baseline + Cfriends_girls_baseline + friends_same_baseline + friends_diff_baseline + Cfriends_diff_baseline + Cfriends_same_baseline + prosocial_y_2year + prosocial_p_2year + friends_boys_2year + friends_girls_2year + Cfriends_boys_2year + Cfriends_girls_2year + friends_same_2year + friends_diff_2year + Cfriends_diff_2year + Cfriends_same_2year + cpeur2_baseline + eaeur1_baseline + depeur4_baseline + mddeur6_baseline + depmulti_baseline + bmieur4_baseline + bmimulti_baseline + insomniaeur6_baseline + snoringeur1_baseline + iqeur2_baseline + happieur4_baseline + ghappieur2_baseline + ghappimeaneur1_baseline + ghappihealth6_baseline + alcdep_eurauto_baseline + alcdep_afrauto_baseline + alcdep_metaauto_baseline + asdauto_baseline + aspauto_baseline + bipauto_baseline + cannabisauto_baseline + crossauto_baseline + drinkauto_baseline + edauto_baseline + neuroticismauto_baseline + ocdauto_baseline + risk4pcauto_baseline + risktolauto_baseline + scz_eurauto_baseline + scz_easauto_baseline + scz_metaauto_baseline + smokerauto_baseline + worryauto_baseline + anxietyauto_baseline + ptsdeur4_baseline + ptsdmeta6_baseline + adhdeur6_baseline + euro_baseline + sex_baseline + race_g_baseline + parent_identity_baseline + demo_brthdat_v2_baseline + demo_sex_v2_baseline + gender_identity_baseline + parent_age_baseline + foreign_born_family_baseline + married_baseline + high_educ_baseline + high_educ2_baseline + income_baseline + foreign_born_baseline + religion_prefer_baseline + gay_parent_baseline + gay_youth_baseline + race_ethnicity_baseline + age_baseline + family_adversity_baseline + height_baseline + weight_baseline + vol_baseline + bmi_baseline + total_ratio_baseline + history_ratio_baseline + euro_2year + race_g_2year + gay_youth_2year + age_2year + height_2year + weight_2year + vol_2year + bmi_2year', data=high_pattern_pos).fit()

# Combine data for Chow Test
combined_data = pd.concat([low_pattern_pos, high_pattern_pos])
model_combined = ols('nihtbx_pattern_uncorrected_2year ~ KSADS_BD_y_baseline + KSADS_Anx_y_baseline + KSADS_Suicide_y_baseline + KSADS_Sleep_y_baseline + KSADS_total_y_baseline + KSADS_BD_y_2year + KSADS_Anx_y_2year + KSADS_Eat_y_2year + KSADS_Suicide_y_2year + KSADS_Sleep_y_2year + KSADS_total_y_2year + KSADS_BD_p_baseline + KSADS_Anx_p_baseline + KSADS_Eat_p_baseline + KSADS_Suicide_p_baseline + KSADS_Sleep_p_baseline + KSADS_total_p_baseline + KSADS_BD_p_2year + KSADS_Anx_p_2year + KSADS_Eat_p_2year + KSADS_Suicide_p_2year + KSADS_Sleep_p_2year + KSADS_total_p_2year + KSADS_Subst_p_baseline + KSADS_Subst_p_2year + cbcl_internal_baseline + cbcl_internal_2year + cbcl_external_baseline + cbcl_external_2year + cbcl_totprob_baseline + cbcl_totprob_2year + upps_premeditation_baseline + upps_premeditation_2year + upps_perseverence_baseline + upps_perseverence_2year + upps_sensation_baseline + upps_sensation_2year + upps_negative_baseline + upps_negative_2year + upps_positive_baseline + upps_positive_2year + BIS_baseline + BIS_2year + BAS_RR_baseline + BAS_RR_2year + BAS_Drive_baseline + BAS_Drive_2year + BAS_Fun_baseline + BAS_Fun_2year + sleep_disturb_total_baseline + sleep_disturb_total_2year + ELS_total_baseline + ELS_total_2year + rh_adi_wsum1_baseline + rh_adi_wsum1_2year + rh_adi_wsum2_baseline + rh_adi_wsum2_2year + risk_alcohol_p_baseline + risk_cigarette_p_baseline + risk_electro_nicotine_p_baseline + risk_marijuana_p_baseline + risk_drug_p_baseline + risk_med_legal_p_baseline + risk_med_marijuana_p_baseline + risk_med_marijuana_pre_p_baseline + risk_family_med_marijuana_p_baseline + risk_alcohol_p_2year + risk_cigarette_p_2year + risk_electro_nicotine_p_2year + risk_marijuana_p_2year + risk_drug_p_2year + risk_med_legal_p_2year + risk_med_marijuana_p_2year + risk_med_marijuana_pre_p_2year + risk_family_med_marijuana_p_2year + risk_alcohol_y_2year + risk_cigarette_y_2year + risk_electro_nicotine_y_2year + risk_marijuana_y_2year + risk_med_legal_y_2year + risk_med_marijuana_y_2year + risk_med_marijuana_pre_y_2year + risk_family_med_marijuana_y_2year + risk_drug_y_2year + risk_gas_y_2year + risk_pain_y_2year + risk_anix_y_2year + risk_stimulant_y_2year + good_school_baseline + good_parent1_baseline + good_parent2_baseline + good_parent_baseline + good_school_2year + prenatal_tobacco_before_baseline + prenatal_alcohol_max_before_baseline + prenatal_alcohol_avg_before_baseline + prenatal_alcohol_eff_before_baseline + prenatal_marijuana_before_baseline + prenatal_cocaine_before_baseline + prenatal_oxycontin_before_baseline + prenatal_tobacco_after_baseline + prenatal_alcohol_max_after_baseline + prenatal_alcohol_avg_after_baseline + prenatal_alcohol_eff_after_baseline + prenatal_marijuana_after_baseline + prenatal_cocaine_after_baseline + prenatal_weeks_baseline + screentime_wkday_tv_baseline + screentime_wkday_videos_baseline + screentime_wkday_games_baseline + screentime_wkday_texting_baseline + screentime_wkday_sns_baseline + screentime_wkday_videochat_baseline + screentime_wkend_tv_baseline + screentime_wkend_videos_baseline + screentime_wkend_games_baseline + screentime_wkend_texting_baseline + screentime_wkend_sns_baseline + screentime_wkend_videochat_baseline + screentime_wkday_y_baseline + screentime_wkend_y_baseline + screentime_wkday_p_baseline + screentime_wkend_p_baseline + screentime_maturegames_baseline + screentime_rmovies_baseline + screentime_addict_p_2year + screentime_risk_p_2year + screentime_maturegames_2year + screentime_rmovies_2year + bpm_total_y_2year + bpm_total_t_baseline + bpm_total_t_2year + prosocial_y_baseline + prosocial_p_baseline + detention_baseline + detention_rea_baseline + friends_boys_baseline + friends_girls_baseline + Cfriends_boys_baseline + Cfriends_girls_baseline + friends_same_baseline + friends_diff_baseline + Cfriends_diff_baseline + Cfriends_same_baseline + prosocial_y_2year + prosocial_p_2year + friends_boys_2year + friends_girls_2year + Cfriends_boys_2year + Cfriends_girls_2year + friends_same_2year + friends_diff_2year + Cfriends_diff_2year + Cfriends_same_2year + cpeur2_baseline + eaeur1_baseline + depeur4_baseline + mddeur6_baseline + depmulti_baseline + bmieur4_baseline + bmimulti_baseline + insomniaeur6_baseline + snoringeur1_baseline + iqeur2_baseline + happieur4_baseline + ghappieur2_baseline + ghappimeaneur1_baseline + ghappihealth6_baseline + alcdep_eurauto_baseline + alcdep_afrauto_baseline + alcdep_metaauto_baseline + asdauto_baseline + aspauto_baseline + bipauto_baseline + cannabisauto_baseline + crossauto_baseline + drinkauto_baseline + edauto_baseline + neuroticismauto_baseline + ocdauto_baseline + risk4pcauto_baseline + risktolauto_baseline + scz_eurauto_baseline + scz_easauto_baseline + scz_metaauto_baseline + smokerauto_baseline + worryauto_baseline + anxietyauto_baseline + ptsdeur4_baseline + ptsdmeta6_baseline + adhdeur6_baseline + euro_baseline + sex_baseline + race_g_baseline + parent_identity_baseline + demo_brthdat_v2_baseline + demo_sex_v2_baseline + gender_identity_baseline + parent_age_baseline + foreign_born_family_baseline + married_baseline + high_educ_baseline + high_educ2_baseline + income_baseline + foreign_born_baseline + religion_prefer_baseline + gay_parent_baseline + gay_youth_baseline + race_ethnicity_baseline + age_baseline + family_adversity_baseline + height_baseline + weight_baseline + vol_baseline + bmi_baseline + total_ratio_baseline + history_ratio_baseline + euro_2year + race_g_2year + gay_youth_2year + age_2year + height_2year + weight_2year + vol_2year + bmi_2year', data=combined_data).fit()

# Number of parameters in the model (excluding the intercept)
k = len(model_combined.params) - 1

# Chow Test formula
numerator = (model_combined.ssr - (model_low.ssr + model_high.ssr)) / k
denominator = (model_low.ssr + model_high.ssr) / (len(df_low) + len(df_high) - 2 * k)
f_statistic = numerator / denominator
p_value = stats.f.sf(f_statistic, k, len(df_low) + len(df_high) - 2 * k)  # Using survival function for p-value

print("F-statistic:", f_statistic)
print("P-value:", p_value)

coef_low = model_low.params
p_low = model_low.pvalues

# Filter coefficients with p-values less than 0.05
sig_coef_low = coef_low[p_low < 0.05].index

coef_combined_low = model_combined.params[sig_coef_low]

coefficient_differences = abs(coef_combined_low - coef_low[sig_coef_low])

diff_low_df = pd.DataFrame({'Difference': coefficient_differences, 'Coefficients': coef_low[sig_coef_low], 'P-values': p_low[sig_coef_low]}).sort_values(by='Difference', ascending=False)

print(diff_low_df)

coef_high = model_high.params
p_high = model_high.pvalues

# Filter coefficients with p-values less than 0.05
sig_coef_high = coef_high[p_high < 0.05].index

coef_combined_high = model_combined.params[sig_coef_high]

coefficient_differences = abs(coef_combined_high - coef_high[sig_coef_high])

diff_high_df = pd.DataFrame({'Difference': coefficient_differences, 'Coefficients': coef_high[sig_coef_high], 'P-values': p_high[sig_coef_high]}).sort_values(by='Difference', ascending=False)

print(diff_high_df)

In [ ]:
nan_columns_low = low_picture_neg.columns[low_picture_neg.isna().any()].tolist()
print(nan_columns_low)
nan_columns_high = high_picture_neg.columns[high_picture_neg.isna().any()].tolist()
print(nan_columns_high)
# remove KSADS_Subst_p_baseline, prenatal_heroin_before_baseline, prenatal_heroin_after_baseline, prenatal_oxycontin_before_baseline, prenatal_oxycontin_after_baseline, prenatal_cocaine_after_baseline

In [ ]:
model_low = ols('nihtbx_picture_uncorrected_2year ~ KSADS_BD_y_baseline + KSADS_Anx_y_baseline + KSADS_Suicide_y_baseline + KSADS_Sleep_y_baseline + KSADS_total_y_baseline + KSADS_BD_y_2year + KSADS_Anx_y_2year + KSADS_Eat_y_2year + KSADS_Suicide_y_2year + KSADS_Sleep_y_2year + KSADS_total_y_2year + KSADS_BD_p_baseline + KSADS_Anx_p_baseline + KSADS_Eat_p_baseline + KSADS_Suicide_p_baseline + KSADS_Sleep_p_baseline + KSADS_total_p_baseline + KSADS_BD_p_2year + KSADS_Anx_p_2year + KSADS_Eat_p_2year + KSADS_Suicide_p_2year + KSADS_Sleep_p_2year + KSADS_total_p_2year + KSADS_Subst_p_2year + cbcl_internal_baseline + cbcl_internal_2year + cbcl_external_baseline + cbcl_external_2year + cbcl_totprob_baseline + cbcl_totprob_2year + upps_premeditation_baseline + upps_premeditation_2year + upps_perseverence_baseline + upps_perseverence_2year + upps_sensation_baseline + upps_sensation_2year + upps_negative_baseline + upps_negative_2year + upps_positive_baseline + upps_positive_2year + BIS_baseline + BIS_2year + BAS_RR_baseline + BAS_RR_2year + BAS_Drive_baseline + BAS_Drive_2year + BAS_Fun_baseline + BAS_Fun_2year + sleep_disturb_total_baseline + sleep_disturb_total_2year + ELS_total_baseline + ELS_total_2year + rh_adi_wsum1_baseline + rh_adi_wsum1_2year + rh_adi_wsum2_baseline + rh_adi_wsum2_2year + risk_alcohol_p_baseline + risk_cigarette_p_baseline + risk_electro_nicotine_p_baseline + risk_marijuana_p_baseline + risk_drug_p_baseline + risk_med_legal_p_baseline + risk_med_marijuana_p_baseline + risk_med_marijuana_pre_p_baseline + risk_family_med_marijuana_p_baseline + risk_alcohol_p_2year + risk_cigarette_p_2year + risk_electro_nicotine_p_2year + risk_marijuana_p_2year + risk_drug_p_2year + risk_med_legal_p_2year + risk_med_marijuana_p_2year + risk_med_marijuana_pre_p_2year + risk_family_med_marijuana_p_2year + risk_alcohol_y_2year + risk_cigarette_y_2year + risk_electro_nicotine_y_2year + risk_marijuana_y_2year + risk_med_legal_y_2year + risk_med_marijuana_y_2year + risk_med_marijuana_pre_y_2year + risk_family_med_marijuana_y_2year + risk_drug_y_2year + risk_gas_y_2year + risk_pain_y_2year + risk_anix_y_2year + risk_stimulant_y_2year + good_school_baseline + good_parent1_baseline + good_parent2_baseline + good_parent_baseline + good_school_2year + prenatal_tobacco_before_baseline + prenatal_alcohol_max_before_baseline + prenatal_alcohol_avg_before_baseline + prenatal_alcohol_eff_before_baseline + prenatal_marijuana_before_baseline + prenatal_tobacco_after_baseline + prenatal_alcohol_max_after_baseline + prenatal_alcohol_avg_after_baseline + prenatal_alcohol_eff_after_baseline + prenatal_marijuana_after_baseline + prenatal_cocaine_before_baseline + prenatal_weeks_baseline + screentime_wkday_tv_baseline + screentime_wkday_videos_baseline + screentime_wkday_games_baseline + screentime_wkday_texting_baseline + screentime_wkday_sns_baseline + screentime_wkday_videochat_baseline + screentime_wkend_tv_baseline + screentime_wkend_videos_baseline + screentime_wkend_games_baseline + screentime_wkend_texting_baseline + screentime_wkend_sns_baseline + screentime_wkend_videochat_baseline + screentime_wkday_y_baseline + screentime_wkend_y_baseline + screentime_wkday_p_baseline + screentime_wkend_p_baseline + screentime_maturegames_baseline + screentime_rmovies_baseline + screentime_addict_p_2year + screentime_risk_p_2year + screentime_maturegames_2year + screentime_rmovies_2year + bpm_total_y_2year + bpm_total_t_baseline + bpm_total_t_2year + prosocial_y_baseline + prosocial_p_baseline + detention_baseline + detention_rea_baseline + friends_boys_baseline + friends_girls_baseline + Cfriends_boys_baseline + Cfriends_girls_baseline + friends_same_baseline + friends_diff_baseline + Cfriends_diff_baseline + Cfriends_same_baseline + prosocial_y_2year + prosocial_p_2year + friends_boys_2year + friends_girls_2year + Cfriends_boys_2year + Cfriends_girls_2year + friends_same_2year + friends_diff_2year + Cfriends_diff_2year + Cfriends_same_2year + cpeur2_baseline + eaeur1_baseline + depeur4_baseline + mddeur6_baseline + depmulti_baseline + bmieur4_baseline + bmimulti_baseline + insomniaeur6_baseline + snoringeur1_baseline + iqeur2_baseline + happieur4_baseline + ghappieur2_baseline + ghappimeaneur1_baseline + ghappihealth6_baseline + alcdep_eurauto_baseline + alcdep_afrauto_baseline + alcdep_metaauto_baseline + asdauto_baseline + aspauto_baseline + bipauto_baseline + cannabisauto_baseline + crossauto_baseline + drinkauto_baseline + edauto_baseline + neuroticismauto_baseline + ocdauto_baseline + risk4pcauto_baseline + risktolauto_baseline + scz_eurauto_baseline + scz_easauto_baseline + scz_metaauto_baseline + smokerauto_baseline + worryauto_baseline + anxietyauto_baseline + ptsdeur4_baseline + ptsdmeta6_baseline + adhdeur6_baseline + euro_baseline + sex_baseline + race_g_baseline + parent_identity_baseline + demo_brthdat_v2_baseline + demo_sex_v2_baseline + gender_identity_baseline + parent_age_baseline + foreign_born_family_baseline + married_baseline + high_educ_baseline + high_educ2_baseline + income_baseline + foreign_born_baseline + religion_prefer_baseline + gay_parent_baseline + gay_youth_baseline + race_ethnicity_baseline + age_baseline + family_adversity_baseline + height_baseline + weight_baseline + vol_baseline + bmi_baseline + total_ratio_baseline + history_ratio_baseline + euro_2year + race_g_2year + gay_youth_2year + age_2year + height_2year + weight_2year + vol_2year + bmi_2year', data=low_picture_neg).fit()
model_high = ols('nihtbx_picture_uncorrected_2year ~ KSADS_BD_y_baseline + KSADS_Anx_y_baseline + KSADS_Suicide_y_baseline + KSADS_Sleep_y_baseline + KSADS_total_y_baseline + KSADS_BD_y_2year + KSADS_Anx_y_2year + KSADS_Eat_y_2year + KSADS_Suicide_y_2year + KSADS_Sleep_y_2year + KSADS_total_y_2year + KSADS_BD_p_baseline + KSADS_Anx_p_baseline + KSADS_Eat_p_baseline + KSADS_Suicide_p_baseline + KSADS_Sleep_p_baseline + KSADS_total_p_baseline + KSADS_BD_p_2year + KSADS_Anx_p_2year + KSADS_Eat_p_2year + KSADS_Suicide_p_2year + KSADS_Sleep_p_2year + KSADS_total_p_2year + KSADS_Subst_p_2year + cbcl_internal_baseline + cbcl_internal_2year + cbcl_external_baseline + cbcl_external_2year + cbcl_totprob_baseline + cbcl_totprob_2year + upps_premeditation_baseline + upps_premeditation_2year + upps_perseverence_baseline + upps_perseverence_2year + upps_sensation_baseline + upps_sensation_2year + upps_negative_baseline + upps_negative_2year + upps_positive_baseline + upps_positive_2year + BIS_baseline + BIS_2year + BAS_RR_baseline + BAS_RR_2year + BAS_Drive_baseline + BAS_Drive_2year + BAS_Fun_baseline + BAS_Fun_2year + sleep_disturb_total_baseline + sleep_disturb_total_2year + ELS_total_baseline + ELS_total_2year + rh_adi_wsum1_baseline + rh_adi_wsum1_2year + rh_adi_wsum2_baseline + rh_adi_wsum2_2year + risk_alcohol_p_baseline + risk_cigarette_p_baseline + risk_electro_nicotine_p_baseline + risk_marijuana_p_baseline + risk_drug_p_baseline + risk_med_legal_p_baseline + risk_med_marijuana_p_baseline + risk_med_marijuana_pre_p_baseline + risk_family_med_marijuana_p_baseline + risk_alcohol_p_2year + risk_cigarette_p_2year + risk_electro_nicotine_p_2year + risk_marijuana_p_2year + risk_drug_p_2year + risk_med_legal_p_2year + risk_med_marijuana_p_2year + risk_med_marijuana_pre_p_2year + risk_family_med_marijuana_p_2year + risk_alcohol_y_2year + risk_cigarette_y_2year + risk_electro_nicotine_y_2year + risk_marijuana_y_2year + risk_med_legal_y_2year + risk_med_marijuana_y_2year + risk_med_marijuana_pre_y_2year + risk_family_med_marijuana_y_2year + risk_drug_y_2year + risk_gas_y_2year + risk_pain_y_2year + risk_anix_y_2year + risk_stimulant_y_2year + good_school_baseline + good_parent1_baseline + good_parent2_baseline + good_parent_baseline + good_school_2year + prenatal_tobacco_before_baseline + prenatal_alcohol_max_before_baseline + prenatal_alcohol_avg_before_baseline + prenatal_alcohol_eff_before_baseline + prenatal_marijuana_before_baseline + prenatal_tobacco_after_baseline + prenatal_alcohol_max_after_baseline + prenatal_alcohol_avg_after_baseline + prenatal_alcohol_eff_after_baseline + prenatal_marijuana_after_baseline + prenatal_cocaine_before_baseline + prenatal_weeks_baseline + screentime_wkday_tv_baseline + screentime_wkday_videos_baseline + screentime_wkday_games_baseline + screentime_wkday_texting_baseline + screentime_wkday_sns_baseline + screentime_wkday_videochat_baseline + screentime_wkend_tv_baseline + screentime_wkend_videos_baseline + screentime_wkend_games_baseline + screentime_wkend_texting_baseline + screentime_wkend_sns_baseline + screentime_wkend_videochat_baseline + screentime_wkday_y_baseline + screentime_wkend_y_baseline + screentime_wkday_p_baseline + screentime_wkend_p_baseline + screentime_maturegames_baseline + screentime_rmovies_baseline + screentime_addict_p_2year + screentime_risk_p_2year + screentime_maturegames_2year + screentime_rmovies_2year + bpm_total_y_2year + bpm_total_t_baseline + bpm_total_t_2year + prosocial_y_baseline + prosocial_p_baseline + detention_baseline + detention_rea_baseline + friends_boys_baseline + friends_girls_baseline + Cfriends_boys_baseline + Cfriends_girls_baseline + friends_same_baseline + friends_diff_baseline + Cfriends_diff_baseline + Cfriends_same_baseline + prosocial_y_2year + prosocial_p_2year + friends_boys_2year + friends_girls_2year + Cfriends_boys_2year + Cfriends_girls_2year + friends_same_2year + friends_diff_2year + Cfriends_diff_2year + Cfriends_same_2year + cpeur2_baseline + eaeur1_baseline + depeur4_baseline + mddeur6_baseline + depmulti_baseline + bmieur4_baseline + bmimulti_baseline + insomniaeur6_baseline + snoringeur1_baseline + iqeur2_baseline + happieur4_baseline + ghappieur2_baseline + ghappimeaneur1_baseline + ghappihealth6_baseline + alcdep_eurauto_baseline + alcdep_afrauto_baseline + alcdep_metaauto_baseline + asdauto_baseline + aspauto_baseline + bipauto_baseline + cannabisauto_baseline + crossauto_baseline + drinkauto_baseline + edauto_baseline + neuroticismauto_baseline + ocdauto_baseline + risk4pcauto_baseline + risktolauto_baseline + scz_eurauto_baseline + scz_easauto_baseline + scz_metaauto_baseline + smokerauto_baseline + worryauto_baseline + anxietyauto_baseline + ptsdeur4_baseline + ptsdmeta6_baseline + adhdeur6_baseline + euro_baseline + sex_baseline + race_g_baseline + parent_identity_baseline + demo_brthdat_v2_baseline + demo_sex_v2_baseline + gender_identity_baseline + parent_age_baseline + foreign_born_family_baseline + married_baseline + high_educ_baseline + high_educ2_baseline + income_baseline + foreign_born_baseline + religion_prefer_baseline + gay_parent_baseline + gay_youth_baseline + race_ethnicity_baseline + age_baseline + family_adversity_baseline + height_baseline + weight_baseline + vol_baseline + bmi_baseline + total_ratio_baseline + history_ratio_baseline + euro_2year + race_g_2year + gay_youth_2year + age_2year + height_2year + weight_2year + vol_2year + bmi_2year', data=high_picture_neg).fit()

# Combine data for Chow Test
combined_data = pd.concat([low_picture_neg, high_picture_neg])
model_combined = ols('nihtbx_picture_uncorrected_2year ~ KSADS_BD_y_baseline + KSADS_Anx_y_baseline + KSADS_Suicide_y_baseline + KSADS_Sleep_y_baseline + KSADS_total_y_baseline + KSADS_BD_y_2year + KSADS_Anx_y_2year + KSADS_Eat_y_2year + KSADS_Suicide_y_2year + KSADS_Sleep_y_2year + KSADS_total_y_2year + KSADS_BD_p_baseline + KSADS_Anx_p_baseline + KSADS_Eat_p_baseline + KSADS_Suicide_p_baseline + KSADS_Sleep_p_baseline + KSADS_total_p_baseline + KSADS_BD_p_2year + KSADS_Anx_p_2year + KSADS_Eat_p_2year + KSADS_Suicide_p_2year + KSADS_Sleep_p_2year + KSADS_total_p_2year + KSADS_Subst_p_2year + cbcl_internal_baseline + cbcl_internal_2year + cbcl_external_baseline + cbcl_external_2year + cbcl_totprob_baseline + cbcl_totprob_2year + upps_premeditation_baseline + upps_premeditation_2year + upps_perseverence_baseline + upps_perseverence_2year + upps_sensation_baseline + upps_sensation_2year + upps_negative_baseline + upps_negative_2year + upps_positive_baseline + upps_positive_2year + BIS_baseline + BIS_2year + BAS_RR_baseline + BAS_RR_2year + BAS_Drive_baseline + BAS_Drive_2year + BAS_Fun_baseline + BAS_Fun_2year + sleep_disturb_total_baseline + sleep_disturb_total_2year + ELS_total_baseline + ELS_total_2year + rh_adi_wsum1_baseline + rh_adi_wsum1_2year + rh_adi_wsum2_baseline + rh_adi_wsum2_2year + risk_alcohol_p_baseline + risk_cigarette_p_baseline + risk_electro_nicotine_p_baseline + risk_marijuana_p_baseline + risk_drug_p_baseline + risk_med_legal_p_baseline + risk_med_marijuana_p_baseline + risk_med_marijuana_pre_p_baseline + risk_family_med_marijuana_p_baseline + risk_alcohol_p_2year + risk_cigarette_p_2year + risk_electro_nicotine_p_2year + risk_marijuana_p_2year + risk_drug_p_2year + risk_med_legal_p_2year + risk_med_marijuana_p_2year + risk_med_marijuana_pre_p_2year + risk_family_med_marijuana_p_2year + risk_alcohol_y_2year + risk_cigarette_y_2year + risk_electro_nicotine_y_2year + risk_marijuana_y_2year + risk_med_legal_y_2year + risk_med_marijuana_y_2year + risk_med_marijuana_pre_y_2year + risk_family_med_marijuana_y_2year + risk_drug_y_2year + risk_gas_y_2year + risk_pain_y_2year + risk_anix_y_2year + risk_stimulant_y_2year + good_school_baseline + good_parent1_baseline + good_parent2_baseline + good_parent_baseline + good_school_2year + prenatal_tobacco_before_baseline + prenatal_alcohol_max_before_baseline + prenatal_alcohol_avg_before_baseline + prenatal_alcohol_eff_before_baseline + prenatal_marijuana_before_baseline + prenatal_tobacco_after_baseline + prenatal_alcohol_max_after_baseline + prenatal_alcohol_avg_after_baseline + prenatal_alcohol_eff_after_baseline + prenatal_marijuana_after_baseline + prenatal_cocaine_before_baseline + prenatal_weeks_baseline + screentime_wkday_tv_baseline + screentime_wkday_videos_baseline + screentime_wkday_games_baseline + screentime_wkday_texting_baseline + screentime_wkday_sns_baseline + screentime_wkday_videochat_baseline + screentime_wkend_tv_baseline + screentime_wkend_videos_baseline + screentime_wkend_games_baseline + screentime_wkend_texting_baseline + screentime_wkend_sns_baseline + screentime_wkend_videochat_baseline + screentime_wkday_y_baseline + screentime_wkend_y_baseline + screentime_wkday_p_baseline + screentime_wkend_p_baseline + screentime_maturegames_baseline + screentime_rmovies_baseline + screentime_addict_p_2year + screentime_risk_p_2year + screentime_maturegames_2year + screentime_rmovies_2year + bpm_total_y_2year + bpm_total_t_baseline + bpm_total_t_2year + prosocial_y_baseline + prosocial_p_baseline + detention_baseline + detention_rea_baseline + friends_boys_baseline + friends_girls_baseline + Cfriends_boys_baseline + Cfriends_girls_baseline + friends_same_baseline + friends_diff_baseline + Cfriends_diff_baseline + Cfriends_same_baseline + prosocial_y_2year + prosocial_p_2year + friends_boys_2year + friends_girls_2year + Cfriends_boys_2year + Cfriends_girls_2year + friends_same_2year + friends_diff_2year + Cfriends_diff_2year + Cfriends_same_2year + cpeur2_baseline + eaeur1_baseline + depeur4_baseline + mddeur6_baseline + depmulti_baseline + bmieur4_baseline + bmimulti_baseline + insomniaeur6_baseline + snoringeur1_baseline + iqeur2_baseline + happieur4_baseline + ghappieur2_baseline + ghappimeaneur1_baseline + ghappihealth6_baseline + alcdep_eurauto_baseline + alcdep_afrauto_baseline + alcdep_metaauto_baseline + asdauto_baseline + aspauto_baseline + bipauto_baseline + cannabisauto_baseline + crossauto_baseline + drinkauto_baseline + edauto_baseline + neuroticismauto_baseline + ocdauto_baseline + risk4pcauto_baseline + risktolauto_baseline + scz_eurauto_baseline + scz_easauto_baseline + scz_metaauto_baseline + smokerauto_baseline + worryauto_baseline + anxietyauto_baseline + ptsdeur4_baseline + ptsdmeta6_baseline + adhdeur6_baseline + euro_baseline + sex_baseline + race_g_baseline + parent_identity_baseline + demo_brthdat_v2_baseline + demo_sex_v2_baseline + gender_identity_baseline + parent_age_baseline + foreign_born_family_baseline + married_baseline + high_educ_baseline + high_educ2_baseline + income_baseline + foreign_born_baseline + religion_prefer_baseline + gay_parent_baseline + gay_youth_baseline + race_ethnicity_baseline + age_baseline + family_adversity_baseline + height_baseline + weight_baseline + vol_baseline + bmi_baseline + total_ratio_baseline + history_ratio_baseline + euro_2year + race_g_2year + gay_youth_2year + age_2year + height_2year + weight_2year + vol_2year + bmi_2year', data=combined_data).fit()

# Number of parameters in the model (excluding the intercept)
k = len(model_combined.params) - 1

# Chow Test formula
numerator = (model_combined.ssr - (model_low.ssr + model_high.ssr)) / k
denominator = (model_low.ssr + model_high.ssr) / (len(df_low) + len(df_high) - 2 * k)
f_statistic = numerator / denominator
p_value = stats.f.sf(f_statistic, k, len(df_low) + len(df_high) - 2 * k)  # Using survival function for p-value

print("F-statistic:", f_statistic)
print("P-value:", p_value)

coef_low = model_low.params
p_low = model_low.pvalues

# Filter coefficients with p-values less than 0.05
sig_coef_low = coef_low[p_low < 0.05].index

coef_combined_low = model_combined.params[sig_coef_low]

coefficient_differences = abs(coef_combined_low - coef_low[sig_coef_low])

diff_low_df = pd.DataFrame({'Difference': coefficient_differences, 'Coefficients': coef_low[sig_coef_low], 'P-values': p_low[sig_coef_low]}).sort_values(by='Difference', ascending=False)

print(diff_low_df)

coef_high = model_high.params
p_high = model_high.pvalues

# Filter coefficients with p-values less than 0.05
sig_coef_high = coef_high[p_high < 0.05].index

coef_combined_high = model_combined.params[sig_coef_high]

coefficient_differences = abs(coef_combined_high - coef_high[sig_coef_high])

diff_high_df = pd.DataFrame({'Difference': coefficient_differences, 'Coefficients': coef_high[sig_coef_high], 'P-values': p_high[sig_coef_high]}).sort_values(by='Difference', ascending=False)

print(diff_high_df)

In [ ]:
nan_columns_low = low_picture_pos.columns[low_picture_pos.isna().any()].tolist()
print(nan_columns_low)
nan_columns_high = high_picture_pos.columns[high_picture_pos.isna().any()].tolist()
print(nan_columns_high)
# remove prenatal_heroin_before_baseline, prenatal_heroin_after_baseline, prenatal_cocaine_after_baseline, prenatal_oxycontin_after_baseline

In [ ]:
model_low = ols('nihtbx_picture_uncorrected_2year ~ KSADS_BD_y_baseline + KSADS_Anx_y_baseline + KSADS_Suicide_y_baseline + KSADS_Sleep_y_baseline + KSADS_total_y_baseline + KSADS_BD_y_2year + KSADS_Anx_y_2year + KSADS_Eat_y_2year + KSADS_Suicide_y_2year + KSADS_Sleep_y_2year + KSADS_total_y_2year + KSADS_BD_p_baseline + KSADS_Anx_p_baseline + KSADS_Eat_p_baseline + KSADS_Suicide_p_baseline + KSADS_Sleep_p_baseline + KSADS_total_p_baseline + KSADS_BD_p_2year + KSADS_Anx_p_2year + KSADS_Eat_p_2year + KSADS_Suicide_p_2year + KSADS_Sleep_p_2year + KSADS_total_p_2year + KSADS_Subst_p_2year + cbcl_internal_baseline + cbcl_internal_2year + cbcl_external_baseline + cbcl_external_2year + cbcl_totprob_baseline + cbcl_totprob_2year + upps_premeditation_baseline + upps_premeditation_2year + upps_perseverence_baseline + upps_perseverence_2year + upps_sensation_baseline + upps_sensation_2year + upps_negative_baseline + upps_negative_2year + upps_positive_baseline + upps_positive_2year + BIS_baseline + BIS_2year + BAS_RR_baseline + BAS_RR_2year + BAS_Drive_baseline + BAS_Drive_2year + BAS_Fun_baseline + BAS_Fun_2year + sleep_disturb_total_baseline + sleep_disturb_total_2year + ELS_total_baseline + ELS_total_2year + rh_adi_wsum1_baseline + rh_adi_wsum1_2year + rh_adi_wsum2_baseline + rh_adi_wsum2_2year + risk_alcohol_p_baseline + risk_cigarette_p_baseline + risk_electro_nicotine_p_baseline + risk_marijuana_p_baseline + risk_drug_p_baseline + risk_med_legal_p_baseline + risk_med_marijuana_p_baseline + risk_med_marijuana_pre_p_baseline + risk_family_med_marijuana_p_baseline + risk_alcohol_p_2year + risk_cigarette_p_2year + risk_electro_nicotine_p_2year + risk_marijuana_p_2year + risk_drug_p_2year + risk_med_legal_p_2year + risk_med_marijuana_p_2year + risk_med_marijuana_pre_p_2year + risk_family_med_marijuana_p_2year + risk_alcohol_y_2year + risk_cigarette_y_2year + risk_electro_nicotine_y_2year + risk_marijuana_y_2year + risk_med_legal_y_2year + risk_med_marijuana_y_2year + risk_med_marijuana_pre_y_2year + risk_family_med_marijuana_y_2year + risk_drug_y_2year + risk_gas_y_2year + risk_pain_y_2year + risk_anix_y_2year + risk_stimulant_y_2year + good_school_baseline + good_parent1_baseline + good_parent2_baseline + good_parent_baseline + good_school_2year + prenatal_tobacco_before_baseline + prenatal_alcohol_max_before_baseline + prenatal_alcohol_avg_before_baseline + prenatal_alcohol_eff_before_baseline + prenatal_marijuana_before_baseline + prenatal_oxycontin_before_baseline + prenatal_tobacco_after_baseline + prenatal_alcohol_max_after_baseline + prenatal_alcohol_avg_after_baseline + prenatal_alcohol_eff_after_baseline + prenatal_marijuana_after_baseline + prenatal_cocaine_before_baseline + prenatal_weeks_baseline + screentime_wkday_tv_baseline + screentime_wkday_videos_baseline + screentime_wkday_games_baseline + screentime_wkday_texting_baseline + screentime_wkday_sns_baseline + screentime_wkday_videochat_baseline + screentime_wkend_tv_baseline + screentime_wkend_videos_baseline + screentime_wkend_games_baseline + screentime_wkend_texting_baseline + screentime_wkend_sns_baseline + screentime_wkend_videochat_baseline + screentime_wkday_y_baseline + screentime_wkend_y_baseline + screentime_wkday_p_baseline + screentime_wkend_p_baseline + screentime_maturegames_baseline + screentime_rmovies_baseline + screentime_addict_p_2year + screentime_risk_p_2year + screentime_maturegames_2year + screentime_rmovies_2year + bpm_total_y_2year + bpm_total_t_baseline + bpm_total_t_2year + prosocial_y_baseline + prosocial_p_baseline + detention_baseline + detention_rea_baseline + friends_boys_baseline + friends_girls_baseline + Cfriends_boys_baseline + Cfriends_girls_baseline + friends_same_baseline + friends_diff_baseline + Cfriends_diff_baseline + Cfriends_same_baseline + prosocial_y_2year + prosocial_p_2year + friends_boys_2year + friends_girls_2year + Cfriends_boys_2year + Cfriends_girls_2year + friends_same_2year + friends_diff_2year + Cfriends_diff_2year + Cfriends_same_2year + cpeur2_baseline + eaeur1_baseline + depeur4_baseline + mddeur6_baseline + depmulti_baseline + bmieur4_baseline + bmimulti_baseline + insomniaeur6_baseline + snoringeur1_baseline + iqeur2_baseline + happieur4_baseline + ghappieur2_baseline + ghappimeaneur1_baseline + ghappihealth6_baseline + alcdep_eurauto_baseline + alcdep_afrauto_baseline + alcdep_metaauto_baseline + asdauto_baseline + aspauto_baseline + bipauto_baseline + cannabisauto_baseline + crossauto_baseline + drinkauto_baseline + edauto_baseline + neuroticismauto_baseline + ocdauto_baseline + risk4pcauto_baseline + risktolauto_baseline + scz_eurauto_baseline + scz_easauto_baseline + scz_metaauto_baseline + smokerauto_baseline + worryauto_baseline + anxietyauto_baseline + ptsdeur4_baseline + ptsdmeta6_baseline + adhdeur6_baseline + euro_baseline + sex_baseline + race_g_baseline + parent_identity_baseline + demo_brthdat_v2_baseline + demo_sex_v2_baseline + gender_identity_baseline + parent_age_baseline + foreign_born_family_baseline + married_baseline + high_educ_baseline + high_educ2_baseline + income_baseline + foreign_born_baseline + religion_prefer_baseline + gay_parent_baseline + gay_youth_baseline + race_ethnicity_baseline + age_baseline + family_adversity_baseline + height_baseline + weight_baseline + vol_baseline + bmi_baseline + total_ratio_baseline + history_ratio_baseline + euro_2year + race_g_2year + gay_youth_2year + age_2year + height_2year + weight_2year + vol_2year + bmi_2year', data=low_picture_pos).fit()
model_high = ols('nihtbx_picture_uncorrected_2year ~ KSADS_BD_y_baseline + KSADS_Anx_y_baseline + KSADS_Suicide_y_baseline + KSADS_Sleep_y_baseline + KSADS_total_y_baseline + KSADS_BD_y_2year + KSADS_Anx_y_2year + KSADS_Eat_y_2year + KSADS_Suicide_y_2year + KSADS_Sleep_y_2year + KSADS_total_y_2year + KSADS_BD_p_baseline + KSADS_Anx_p_baseline + KSADS_Eat_p_baseline + KSADS_Suicide_p_baseline + KSADS_Sleep_p_baseline + KSADS_total_p_baseline + KSADS_BD_p_2year + KSADS_Anx_p_2year + KSADS_Eat_p_2year + KSADS_Suicide_p_2year + KSADS_Sleep_p_2year + KSADS_total_p_2year + KSADS_Subst_p_2year + cbcl_internal_baseline + cbcl_internal_2year + cbcl_external_baseline + cbcl_external_2year + cbcl_totprob_baseline + cbcl_totprob_2year + upps_premeditation_baseline + upps_premeditation_2year + upps_perseverence_baseline + upps_perseverence_2year + upps_sensation_baseline + upps_sensation_2year + upps_negative_baseline + upps_negative_2year + upps_positive_baseline + upps_positive_2year + BIS_baseline + BIS_2year + BAS_RR_baseline + BAS_RR_2year + BAS_Drive_baseline + BAS_Drive_2year + BAS_Fun_baseline + BAS_Fun_2year + sleep_disturb_total_baseline + sleep_disturb_total_2year + ELS_total_baseline + ELS_total_2year + rh_adi_wsum1_baseline + rh_adi_wsum1_2year + rh_adi_wsum2_baseline + rh_adi_wsum2_2year + risk_alcohol_p_baseline + risk_cigarette_p_baseline + risk_electro_nicotine_p_baseline + risk_marijuana_p_baseline + risk_drug_p_baseline + risk_med_legal_p_baseline + risk_med_marijuana_p_baseline + risk_med_marijuana_pre_p_baseline + risk_family_med_marijuana_p_baseline + risk_alcohol_p_2year + risk_cigarette_p_2year + risk_electro_nicotine_p_2year + risk_marijuana_p_2year + risk_drug_p_2year + risk_med_legal_p_2year + risk_med_marijuana_p_2year + risk_med_marijuana_pre_p_2year + risk_family_med_marijuana_p_2year + risk_alcohol_y_2year + risk_cigarette_y_2year + risk_electro_nicotine_y_2year + risk_marijuana_y_2year + risk_med_legal_y_2year + risk_med_marijuana_y_2year + risk_med_marijuana_pre_y_2year + risk_family_med_marijuana_y_2year + risk_drug_y_2year + risk_gas_y_2year + risk_pain_y_2year + risk_anix_y_2year + risk_stimulant_y_2year + good_school_baseline + good_parent1_baseline + good_parent2_baseline + good_parent_baseline + good_school_2year + prenatal_tobacco_before_baseline + prenatal_alcohol_max_before_baseline + prenatal_alcohol_avg_before_baseline + prenatal_alcohol_eff_before_baseline + prenatal_marijuana_before_baseline + prenatal_oxycontin_before_baseline + prenatal_tobacco_after_baseline + prenatal_alcohol_max_after_baseline + prenatal_alcohol_avg_after_baseline + prenatal_alcohol_eff_after_baseline + prenatal_marijuana_after_baseline + prenatal_cocaine_before_baseline + prenatal_weeks_baseline + screentime_wkday_tv_baseline + screentime_wkday_videos_baseline + screentime_wkday_games_baseline + screentime_wkday_texting_baseline + screentime_wkday_sns_baseline + screentime_wkday_videochat_baseline + screentime_wkend_tv_baseline + screentime_wkend_videos_baseline + screentime_wkend_games_baseline + screentime_wkend_texting_baseline + screentime_wkend_sns_baseline + screentime_wkend_videochat_baseline + screentime_wkday_y_baseline + screentime_wkend_y_baseline + screentime_wkday_p_baseline + screentime_wkend_p_baseline + screentime_maturegames_baseline + screentime_rmovies_baseline + screentime_addict_p_2year + screentime_risk_p_2year + screentime_maturegames_2year + screentime_rmovies_2year + bpm_total_y_2year + bpm_total_t_baseline + bpm_total_t_2year + prosocial_y_baseline + prosocial_p_baseline + detention_baseline + detention_rea_baseline + friends_boys_baseline + friends_girls_baseline + Cfriends_boys_baseline + Cfriends_girls_baseline + friends_same_baseline + friends_diff_baseline + Cfriends_diff_baseline + Cfriends_same_baseline + prosocial_y_2year + prosocial_p_2year + friends_boys_2year + friends_girls_2year + Cfriends_boys_2year + Cfriends_girls_2year + friends_same_2year + friends_diff_2year + Cfriends_diff_2year + Cfriends_same_2year + cpeur2_baseline + eaeur1_baseline + depeur4_baseline + mddeur6_baseline + depmulti_baseline + bmieur4_baseline + bmimulti_baseline + insomniaeur6_baseline + snoringeur1_baseline + iqeur2_baseline + happieur4_baseline + ghappieur2_baseline + ghappimeaneur1_baseline + ghappihealth6_baseline + alcdep_eurauto_baseline + alcdep_afrauto_baseline + alcdep_metaauto_baseline + asdauto_baseline + aspauto_baseline + bipauto_baseline + cannabisauto_baseline + crossauto_baseline + drinkauto_baseline + edauto_baseline + neuroticismauto_baseline + ocdauto_baseline + risk4pcauto_baseline + risktolauto_baseline + scz_eurauto_baseline + scz_easauto_baseline + scz_metaauto_baseline + smokerauto_baseline + worryauto_baseline + anxietyauto_baseline + ptsdeur4_baseline + ptsdmeta6_baseline + adhdeur6_baseline + euro_baseline + sex_baseline + race_g_baseline + parent_identity_baseline + demo_brthdat_v2_baseline + demo_sex_v2_baseline + gender_identity_baseline + parent_age_baseline + foreign_born_family_baseline + married_baseline + high_educ_baseline + high_educ2_baseline + income_baseline + foreign_born_baseline + religion_prefer_baseline + gay_parent_baseline + gay_youth_baseline + race_ethnicity_baseline + age_baseline + family_adversity_baseline + height_baseline + weight_baseline + vol_baseline + bmi_baseline + total_ratio_baseline + history_ratio_baseline + euro_2year + race_g_2year + gay_youth_2year + age_2year + height_2year + weight_2year + vol_2year + bmi_2year', data=high_picture_pos).fit()

# Combine data for Chow Test
combined_data = pd.concat([low_picture_pos, high_picture_pos])
model_combined = ols('nihtbx_picture_uncorrected_2year ~ KSADS_BD_y_baseline + KSADS_Anx_y_baseline + KSADS_Suicide_y_baseline + KSADS_Sleep_y_baseline + KSADS_total_y_baseline + KSADS_BD_y_2year + KSADS_Anx_y_2year + KSADS_Eat_y_2year + KSADS_Suicide_y_2year + KSADS_Sleep_y_2year + KSADS_total_y_2year + KSADS_BD_p_baseline + KSADS_Anx_p_baseline + KSADS_Eat_p_baseline + KSADS_Suicide_p_baseline + KSADS_Sleep_p_baseline + KSADS_total_p_baseline + KSADS_BD_p_2year + KSADS_Anx_p_2year + KSADS_Eat_p_2year + KSADS_Suicide_p_2year + KSADS_Sleep_p_2year + KSADS_total_p_2year + KSADS_Subst_p_2year + cbcl_internal_baseline + cbcl_internal_2year + cbcl_external_baseline + cbcl_external_2year + cbcl_totprob_baseline + cbcl_totprob_2year + upps_premeditation_baseline + upps_premeditation_2year + upps_perseverence_baseline + upps_perseverence_2year + upps_sensation_baseline + upps_sensation_2year + upps_negative_baseline + upps_negative_2year + upps_positive_baseline + upps_positive_2year + BIS_baseline + BIS_2year + BAS_RR_baseline + BAS_RR_2year + BAS_Drive_baseline + BAS_Drive_2year + BAS_Fun_baseline + BAS_Fun_2year + sleep_disturb_total_baseline + sleep_disturb_total_2year + ELS_total_baseline + ELS_total_2year + rh_adi_wsum1_baseline + rh_adi_wsum1_2year + rh_adi_wsum2_baseline + rh_adi_wsum2_2year + risk_alcohol_p_baseline + risk_cigarette_p_baseline + risk_electro_nicotine_p_baseline + risk_marijuana_p_baseline + risk_drug_p_baseline + risk_med_legal_p_baseline + risk_med_marijuana_p_baseline + risk_med_marijuana_pre_p_baseline + risk_family_med_marijuana_p_baseline + risk_alcohol_p_2year + risk_cigarette_p_2year + risk_electro_nicotine_p_2year + risk_marijuana_p_2year + risk_drug_p_2year + risk_med_legal_p_2year + risk_med_marijuana_p_2year + risk_med_marijuana_pre_p_2year + risk_family_med_marijuana_p_2year + risk_alcohol_y_2year + risk_cigarette_y_2year + risk_electro_nicotine_y_2year + risk_marijuana_y_2year + risk_med_legal_y_2year + risk_med_marijuana_y_2year + risk_med_marijuana_pre_y_2year + risk_family_med_marijuana_y_2year + risk_drug_y_2year + risk_gas_y_2year + risk_pain_y_2year + risk_anix_y_2year + risk_stimulant_y_2year + good_school_baseline + good_parent1_baseline + good_parent2_baseline + good_parent_baseline + good_school_2year + prenatal_tobacco_before_baseline + prenatal_alcohol_max_before_baseline + prenatal_alcohol_avg_before_baseline + prenatal_alcohol_eff_before_baseline + prenatal_marijuana_before_baseline + prenatal_oxycontin_before_baseline + prenatal_tobacco_after_baseline + prenatal_alcohol_max_after_baseline + prenatal_alcohol_avg_after_baseline + prenatal_alcohol_eff_after_baseline + prenatal_marijuana_after_baseline + prenatal_cocaine_before_baseline + prenatal_weeks_baseline + screentime_wkday_tv_baseline + screentime_wkday_videos_baseline + screentime_wkday_games_baseline + screentime_wkday_texting_baseline + screentime_wkday_sns_baseline + screentime_wkday_videochat_baseline + screentime_wkend_tv_baseline + screentime_wkend_videos_baseline + screentime_wkend_games_baseline + screentime_wkend_texting_baseline + screentime_wkend_sns_baseline + screentime_wkend_videochat_baseline + screentime_wkday_y_baseline + screentime_wkend_y_baseline + screentime_wkday_p_baseline + screentime_wkend_p_baseline + screentime_maturegames_baseline + screentime_rmovies_baseline + screentime_addict_p_2year + screentime_risk_p_2year + screentime_maturegames_2year + screentime_rmovies_2year + bpm_total_y_2year + bpm_total_t_baseline + bpm_total_t_2year + prosocial_y_baseline + prosocial_p_baseline + detention_baseline + detention_rea_baseline + friends_boys_baseline + friends_girls_baseline + Cfriends_boys_baseline + Cfriends_girls_baseline + friends_same_baseline + friends_diff_baseline + Cfriends_diff_baseline + Cfriends_same_baseline + prosocial_y_2year + prosocial_p_2year + friends_boys_2year + friends_girls_2year + Cfriends_boys_2year + Cfriends_girls_2year + friends_same_2year + friends_diff_2year + Cfriends_diff_2year + Cfriends_same_2year + cpeur2_baseline + eaeur1_baseline + depeur4_baseline + mddeur6_baseline + depmulti_baseline + bmieur4_baseline + bmimulti_baseline + insomniaeur6_baseline + snoringeur1_baseline + iqeur2_baseline + happieur4_baseline + ghappieur2_baseline + ghappimeaneur1_baseline + ghappihealth6_baseline + alcdep_eurauto_baseline + alcdep_afrauto_baseline + alcdep_metaauto_baseline + asdauto_baseline + aspauto_baseline + bipauto_baseline + cannabisauto_baseline + crossauto_baseline + drinkauto_baseline + edauto_baseline + neuroticismauto_baseline + ocdauto_baseline + risk4pcauto_baseline + risktolauto_baseline + scz_eurauto_baseline + scz_easauto_baseline + scz_metaauto_baseline + smokerauto_baseline + worryauto_baseline + anxietyauto_baseline + ptsdeur4_baseline + ptsdmeta6_baseline + adhdeur6_baseline + euro_baseline + sex_baseline + race_g_baseline + parent_identity_baseline + demo_brthdat_v2_baseline + demo_sex_v2_baseline + gender_identity_baseline + parent_age_baseline + foreign_born_family_baseline + married_baseline + high_educ_baseline + high_educ2_baseline + income_baseline + foreign_born_baseline + religion_prefer_baseline + gay_parent_baseline + gay_youth_baseline + race_ethnicity_baseline + age_baseline + family_adversity_baseline + height_baseline + weight_baseline + vol_baseline + bmi_baseline + total_ratio_baseline + history_ratio_baseline + euro_2year + race_g_2year + gay_youth_2year + age_2year + height_2year + weight_2year + vol_2year + bmi_2year', data=combined_data).fit()

# Number of parameters in the model (excluding the intercept)
k = len(model_combined.params) - 1

# Chow Test formula
numerator = (model_combined.ssr - (model_low.ssr + model_high.ssr)) / k
denominator = (model_low.ssr + model_high.ssr) / (len(df_low) + len(df_high) - 2 * k)
f_statistic = numerator / denominator
p_value = stats.f.sf(f_statistic, k, len(df_low) + len(df_high) - 2 * k)  # Using survival function for p-value

print("F-statistic:", f_statistic)
print("P-value:", p_value)

coef_low = model_low.params
p_low = model_low.pvalues

# Filter coefficients with p-values less than 0.05
sig_coef_low = coef_low[p_low < 0.05].index

coef_combined_low = model_combined.params[sig_coef_low]

coefficient_differences = abs(coef_combined_low - coef_low[sig_coef_low])

diff_low_df = pd.DataFrame({'Difference': coefficient_differences, 'Coefficients': coef_low[sig_coef_low], 'P-values': p_low[sig_coef_low]}).sort_values(by='Difference', ascending=False)

print(diff_low_df)

coef_high = model_high.params
p_high = model_high.pvalues

# Filter coefficients with p-values less than 0.05
sig_coef_high = coef_high[p_high < 0.05].index

coef_combined_high = model_combined.params[sig_coef_high]

coefficient_differences = abs(coef_combined_high - coef_high[sig_coef_high])

diff_high_df = pd.DataFrame({'Difference': coefficient_differences, 'Coefficients': coef_high[sig_coef_high], 'P-values': p_high[sig_coef_high]}).sort_values(by='Difference', ascending=False)

print(diff_high_df)

In [ ]:
# nan_columns_low = low_reading_neg.columns[low_reading_neg.isna().any()].tolist()
# print(nan_columns_low)
# nan_columns_high = high_reading_neg.columns[high_reading_neg.isna().any()].tolist()
# print(nan_columns_high)
# remove KSADS_Subst_p_baseline, KSADS_Subst_p_2year, prenatal_heroin_before_baseline, prenatal_heroin_after_baseline, prenatal_oxycontin_before_baseline, prenatal_oxycontin_after_baseline, prenatal_cocaine_before_baseline, prenatal_cocaine_after_baseline, prenatal_alcohol_max_after_baseline, prenatal_alcohol_avg_after_baseline, prenatal_alcohol_eff_after_baseline, prenatal_marijuana_after_baseline

In [ ]:
model_low = ols('nihtbx_reading_uncorrected_2year ~ KSADS_BD_y_baseline + KSADS_Anx_y_baseline + KSADS_Suicide_y_baseline + KSADS_Sleep_y_baseline + KSADS_total_y_baseline + KSADS_BD_y_2year + KSADS_Anx_y_2year + KSADS_Eat_y_2year + KSADS_Suicide_y_2year + KSADS_Sleep_y_2year + KSADS_total_y_2year + KSADS_BD_p_baseline + KSADS_Anx_p_baseline + KSADS_Eat_p_baseline + KSADS_Suicide_p_baseline + KSADS_Sleep_p_baseline + KSADS_total_p_baseline + KSADS_BD_p_2year + KSADS_Anx_p_2year + KSADS_Eat_p_2year + KSADS_Suicide_p_2year + KSADS_Sleep_p_2year + KSADS_total_p_2year + cbcl_internal_baseline + cbcl_internal_2year + cbcl_external_baseline + cbcl_external_2year + cbcl_totprob_baseline + cbcl_totprob_2year + upps_premeditation_baseline + upps_premeditation_2year + upps_perseverence_baseline + upps_perseverence_2year + upps_sensation_baseline + upps_sensation_2year + upps_negative_baseline + upps_negative_2year + upps_positive_baseline + upps_positive_2year + BIS_baseline + BIS_2year + BAS_RR_baseline + BAS_RR_2year + BAS_Drive_baseline + BAS_Drive_2year + BAS_Fun_baseline + BAS_Fun_2year + sleep_disturb_total_baseline + sleep_disturb_total_2year + ELS_total_baseline + ELS_total_2year + rh_adi_wsum1_baseline + rh_adi_wsum1_2year + rh_adi_wsum2_baseline + rh_adi_wsum2_2year + risk_alcohol_p_baseline + risk_cigarette_p_baseline + risk_electro_nicotine_p_baseline + risk_marijuana_p_baseline + risk_drug_p_baseline + risk_med_legal_p_baseline + risk_med_marijuana_p_baseline + risk_med_marijuana_pre_p_baseline + risk_family_med_marijuana_p_baseline + risk_alcohol_p_2year + risk_cigarette_p_2year + risk_electro_nicotine_p_2year + risk_marijuana_p_2year + risk_drug_p_2year + risk_med_legal_p_2year + risk_med_marijuana_p_2year + risk_med_marijuana_pre_p_2year + risk_family_med_marijuana_p_2year + risk_alcohol_y_2year + risk_cigarette_y_2year + risk_electro_nicotine_y_2year + risk_marijuana_y_2year + risk_med_legal_y_2year + risk_med_marijuana_y_2year + risk_med_marijuana_pre_y_2year + risk_family_med_marijuana_y_2year + risk_drug_y_2year + risk_gas_y_2year + risk_pain_y_2year + risk_anix_y_2year + risk_stimulant_y_2year + good_school_baseline + good_parent1_baseline + good_parent2_baseline + good_parent_baseline + good_school_2year + prenatal_tobacco_before_baseline + prenatal_alcohol_max_before_baseline + prenatal_alcohol_avg_before_baseline + prenatal_alcohol_eff_before_baseline + prenatal_marijuana_before_baseline + prenatal_tobacco_after_baseline + prenatal_weeks_baseline + screentime_wkday_tv_baseline + screentime_wkday_videos_baseline + screentime_wkday_games_baseline + screentime_wkday_texting_baseline + screentime_wkday_sns_baseline + screentime_wkday_videochat_baseline + screentime_wkend_tv_baseline + screentime_wkend_videos_baseline + screentime_wkend_games_baseline + screentime_wkend_texting_baseline + screentime_wkend_sns_baseline + screentime_wkend_videochat_baseline + screentime_wkday_y_baseline + screentime_wkend_y_baseline + screentime_wkday_p_baseline + screentime_wkend_p_baseline + screentime_maturegames_baseline + screentime_rmovies_baseline + screentime_addict_p_2year + screentime_risk_p_2year + screentime_maturegames_2year + screentime_rmovies_2year + bpm_total_y_2year + bpm_total_t_baseline + bpm_total_t_2year + prosocial_y_baseline + prosocial_p_baseline + detention_baseline + detention_rea_baseline + friends_boys_baseline + friends_girls_baseline + Cfriends_boys_baseline + Cfriends_girls_baseline + friends_same_baseline + friends_diff_baseline + Cfriends_diff_baseline + Cfriends_same_baseline + prosocial_y_2year + prosocial_p_2year + friends_boys_2year + friends_girls_2year + Cfriends_boys_2year + Cfriends_girls_2year + friends_same_2year + friends_diff_2year + Cfriends_diff_2year + Cfriends_same_2year + cpeur2_baseline + eaeur1_baseline + depeur4_baseline + mddeur6_baseline + depmulti_baseline + bmieur4_baseline + bmimulti_baseline + insomniaeur6_baseline + snoringeur1_baseline + iqeur2_baseline + happieur4_baseline + ghappieur2_baseline + ghappimeaneur1_baseline + ghappihealth6_baseline + alcdep_eurauto_baseline + alcdep_afrauto_baseline + alcdep_metaauto_baseline + asdauto_baseline + aspauto_baseline + bipauto_baseline + cannabisauto_baseline + crossauto_baseline + drinkauto_baseline + edauto_baseline + neuroticismauto_baseline + ocdauto_baseline + risk4pcauto_baseline + risktolauto_baseline + scz_eurauto_baseline + scz_easauto_baseline + scz_metaauto_baseline + smokerauto_baseline + worryauto_baseline + anxietyauto_baseline + ptsdeur4_baseline + ptsdmeta6_baseline + adhdeur6_baseline + euro_baseline + sex_baseline + race_g_baseline + parent_identity_baseline + demo_brthdat_v2_baseline + demo_sex_v2_baseline + gender_identity_baseline + parent_age_baseline + foreign_born_family_baseline + married_baseline + high_educ_baseline + high_educ2_baseline + income_baseline + foreign_born_baseline + religion_prefer_baseline + gay_parent_baseline + gay_youth_baseline + race_ethnicity_baseline + age_baseline + family_adversity_baseline + height_baseline + weight_baseline + vol_baseline + bmi_baseline + total_ratio_baseline + history_ratio_baseline + euro_2year + race_g_2year + gay_youth_2year + age_2year + height_2year + weight_2year + vol_2year + bmi_2year', data=low_reading_neg).fit()
model_high = ols('nihtbx_reading_uncorrected_2year ~ KSADS_BD_y_baseline + KSADS_Anx_y_baseline + KSADS_Suicide_y_baseline + KSADS_Sleep_y_baseline + KSADS_total_y_baseline + KSADS_BD_y_2year + KSADS_Anx_y_2year + KSADS_Eat_y_2year + KSADS_Suicide_y_2year + KSADS_Sleep_y_2year + KSADS_total_y_2year + KSADS_BD_p_baseline + KSADS_Anx_p_baseline + KSADS_Eat_p_baseline + KSADS_Suicide_p_baseline + KSADS_Sleep_p_baseline + KSADS_total_p_baseline + KSADS_BD_p_2year + KSADS_Anx_p_2year + KSADS_Eat_p_2year + KSADS_Suicide_p_2year + KSADS_Sleep_p_2year + KSADS_total_p_2year + cbcl_internal_baseline + cbcl_internal_2year + cbcl_external_baseline + cbcl_external_2year + cbcl_totprob_baseline + cbcl_totprob_2year + upps_premeditation_baseline + upps_premeditation_2year + upps_perseverence_baseline + upps_perseverence_2year + upps_sensation_baseline + upps_sensation_2year + upps_negative_baseline + upps_negative_2year + upps_positive_baseline + upps_positive_2year + BIS_baseline + BIS_2year + BAS_RR_baseline + BAS_RR_2year + BAS_Drive_baseline + BAS_Drive_2year + BAS_Fun_baseline + BAS_Fun_2year + sleep_disturb_total_baseline + sleep_disturb_total_2year + ELS_total_baseline + ELS_total_2year + rh_adi_wsum1_baseline + rh_adi_wsum1_2year + rh_adi_wsum2_baseline + rh_adi_wsum2_2year + risk_alcohol_p_baseline + risk_cigarette_p_baseline + risk_electro_nicotine_p_baseline + risk_marijuana_p_baseline + risk_drug_p_baseline + risk_med_legal_p_baseline + risk_med_marijuana_p_baseline + risk_med_marijuana_pre_p_baseline + risk_family_med_marijuana_p_baseline + risk_alcohol_p_2year + risk_cigarette_p_2year + risk_electro_nicotine_p_2year + risk_marijuana_p_2year + risk_drug_p_2year + risk_med_legal_p_2year + risk_med_marijuana_p_2year + risk_med_marijuana_pre_p_2year + risk_family_med_marijuana_p_2year + risk_alcohol_y_2year + risk_cigarette_y_2year + risk_electro_nicotine_y_2year + risk_marijuana_y_2year + risk_med_legal_y_2year + risk_med_marijuana_y_2year + risk_med_marijuana_pre_y_2year + risk_family_med_marijuana_y_2year + risk_drug_y_2year + risk_gas_y_2year + risk_pain_y_2year + risk_anix_y_2year + risk_stimulant_y_2year + good_school_baseline + good_parent1_baseline + good_parent2_baseline + good_parent_baseline + good_school_2year + prenatal_tobacco_before_baseline + prenatal_alcohol_max_before_baseline + prenatal_alcohol_avg_before_baseline + prenatal_alcohol_eff_before_baseline + prenatal_marijuana_before_baseline + prenatal_tobacco_after_baseline + prenatal_weeks_baseline + screentime_wkday_tv_baseline + screentime_wkday_videos_baseline + screentime_wkday_games_baseline + screentime_wkday_texting_baseline + screentime_wkday_sns_baseline + screentime_wkday_videochat_baseline + screentime_wkend_tv_baseline + screentime_wkend_videos_baseline + screentime_wkend_games_baseline + screentime_wkend_texting_baseline + screentime_wkend_sns_baseline + screentime_wkend_videochat_baseline + screentime_wkday_y_baseline + screentime_wkend_y_baseline + screentime_wkday_p_baseline + screentime_wkend_p_baseline + screentime_maturegames_baseline + screentime_rmovies_baseline + screentime_addict_p_2year + screentime_risk_p_2year + screentime_maturegames_2year + screentime_rmovies_2year + bpm_total_y_2year + bpm_total_t_baseline + bpm_total_t_2year + prosocial_y_baseline + prosocial_p_baseline + detention_baseline + detention_rea_baseline + friends_boys_baseline + friends_girls_baseline + Cfriends_boys_baseline + Cfriends_girls_baseline + friends_same_baseline + friends_diff_baseline + Cfriends_diff_baseline + Cfriends_same_baseline + prosocial_y_2year + prosocial_p_2year + friends_boys_2year + friends_girls_2year + Cfriends_boys_2year + Cfriends_girls_2year + friends_same_2year + friends_diff_2year + Cfriends_diff_2year + Cfriends_same_2year + cpeur2_baseline + eaeur1_baseline + depeur4_baseline + mddeur6_baseline + depmulti_baseline + bmieur4_baseline + bmimulti_baseline + insomniaeur6_baseline + snoringeur1_baseline + iqeur2_baseline + happieur4_baseline + ghappieur2_baseline + ghappimeaneur1_baseline + ghappihealth6_baseline + alcdep_eurauto_baseline + alcdep_afrauto_baseline + alcdep_metaauto_baseline + asdauto_baseline + aspauto_baseline + bipauto_baseline + cannabisauto_baseline + crossauto_baseline + drinkauto_baseline + edauto_baseline + neuroticismauto_baseline + ocdauto_baseline + risk4pcauto_baseline + risktolauto_baseline + scz_eurauto_baseline + scz_easauto_baseline + scz_metaauto_baseline + smokerauto_baseline + worryauto_baseline + anxietyauto_baseline + ptsdeur4_baseline + ptsdmeta6_baseline + adhdeur6_baseline + euro_baseline + sex_baseline + race_g_baseline + parent_identity_baseline + demo_brthdat_v2_baseline + demo_sex_v2_baseline + gender_identity_baseline + parent_age_baseline + foreign_born_family_baseline + married_baseline + high_educ_baseline + high_educ2_baseline + income_baseline + foreign_born_baseline + religion_prefer_baseline + gay_parent_baseline + gay_youth_baseline + race_ethnicity_baseline + age_baseline + family_adversity_baseline + height_baseline + weight_baseline + vol_baseline + bmi_baseline + total_ratio_baseline + history_ratio_baseline + euro_2year + race_g_2year + gay_youth_2year + age_2year + height_2year + weight_2year + vol_2year + bmi_2year', data=high_reading_neg).fit()

# Combine data for Chow Test
combined_data = pd.concat([low_reading_neg, high_reading_neg])
model_combined = ols('nihtbx_reading_uncorrected_2year ~ KSADS_BD_y_baseline + KSADS_Anx_y_baseline + KSADS_Suicide_y_baseline + KSADS_Sleep_y_baseline + KSADS_total_y_baseline + KSADS_BD_y_2year + KSADS_Anx_y_2year + KSADS_Eat_y_2year + KSADS_Suicide_y_2year + KSADS_Sleep_y_2year + KSADS_total_y_2year + KSADS_BD_p_baseline + KSADS_Anx_p_baseline + KSADS_Eat_p_baseline + KSADS_Suicide_p_baseline + KSADS_Sleep_p_baseline + KSADS_total_p_baseline + KSADS_BD_p_2year + KSADS_Anx_p_2year + KSADS_Eat_p_2year + KSADS_Suicide_p_2year + KSADS_Sleep_p_2year + KSADS_total_p_2year + cbcl_internal_baseline + cbcl_internal_2year + cbcl_external_baseline + cbcl_external_2year + cbcl_totprob_baseline + cbcl_totprob_2year + upps_premeditation_baseline + upps_premeditation_2year + upps_perseverence_baseline + upps_perseverence_2year + upps_sensation_baseline + upps_sensation_2year + upps_negative_baseline + upps_negative_2year + upps_positive_baseline + upps_positive_2year + BIS_baseline + BIS_2year + BAS_RR_baseline + BAS_RR_2year + BAS_Drive_baseline + BAS_Drive_2year + BAS_Fun_baseline + BAS_Fun_2year + sleep_disturb_total_baseline + sleep_disturb_total_2year + ELS_total_baseline + ELS_total_2year + rh_adi_wsum1_baseline + rh_adi_wsum1_2year + rh_adi_wsum2_baseline + rh_adi_wsum2_2year + risk_alcohol_p_baseline + risk_cigarette_p_baseline + risk_electro_nicotine_p_baseline + risk_marijuana_p_baseline + risk_drug_p_baseline + risk_med_legal_p_baseline + risk_med_marijuana_p_baseline + risk_med_marijuana_pre_p_baseline + risk_family_med_marijuana_p_baseline + risk_alcohol_p_2year + risk_cigarette_p_2year + risk_electro_nicotine_p_2year + risk_marijuana_p_2year + risk_drug_p_2year + risk_med_legal_p_2year + risk_med_marijuana_p_2year + risk_med_marijuana_pre_p_2year + risk_family_med_marijuana_p_2year + risk_alcohol_y_2year + risk_cigarette_y_2year + risk_electro_nicotine_y_2year + risk_marijuana_y_2year + risk_med_legal_y_2year + risk_med_marijuana_y_2year + risk_med_marijuana_pre_y_2year + risk_family_med_marijuana_y_2year + risk_drug_y_2year + risk_gas_y_2year + risk_pain_y_2year + risk_anix_y_2year + risk_stimulant_y_2year + good_school_baseline + good_parent1_baseline + good_parent2_baseline + good_parent_baseline + good_school_2year + prenatal_tobacco_before_baseline + prenatal_alcohol_max_before_baseline + prenatal_alcohol_avg_before_baseline + prenatal_alcohol_eff_before_baseline + prenatal_marijuana_before_baseline + prenatal_tobacco_after_baseline + prenatal_weeks_baseline + screentime_wkday_tv_baseline + screentime_wkday_videos_baseline + screentime_wkday_games_baseline + screentime_wkday_texting_baseline + screentime_wkday_sns_baseline + screentime_wkday_videochat_baseline + screentime_wkend_tv_baseline + screentime_wkend_videos_baseline + screentime_wkend_games_baseline + screentime_wkend_texting_baseline + screentime_wkend_sns_baseline + screentime_wkend_videochat_baseline + screentime_wkday_y_baseline + screentime_wkend_y_baseline + screentime_wkday_p_baseline + screentime_wkend_p_baseline + screentime_maturegames_baseline + screentime_rmovies_baseline + screentime_addict_p_2year + screentime_risk_p_2year + screentime_maturegames_2year + screentime_rmovies_2year + bpm_total_y_2year + bpm_total_t_baseline + bpm_total_t_2year + prosocial_y_baseline + prosocial_p_baseline + detention_baseline + detention_rea_baseline + friends_boys_baseline + friends_girls_baseline + Cfriends_boys_baseline + Cfriends_girls_baseline + friends_same_baseline + friends_diff_baseline + Cfriends_diff_baseline + Cfriends_same_baseline + prosocial_y_2year + prosocial_p_2year + friends_boys_2year + friends_girls_2year + Cfriends_boys_2year + Cfriends_girls_2year + friends_same_2year + friends_diff_2year + Cfriends_diff_2year + Cfriends_same_2year + cpeur2_baseline + eaeur1_baseline + depeur4_baseline + mddeur6_baseline + depmulti_baseline + bmieur4_baseline + bmimulti_baseline + insomniaeur6_baseline + snoringeur1_baseline + iqeur2_baseline + happieur4_baseline + ghappieur2_baseline + ghappimeaneur1_baseline + ghappihealth6_baseline + alcdep_eurauto_baseline + alcdep_afrauto_baseline + alcdep_metaauto_baseline + asdauto_baseline + aspauto_baseline + bipauto_baseline + cannabisauto_baseline + crossauto_baseline + drinkauto_baseline + edauto_baseline + neuroticismauto_baseline + ocdauto_baseline + risk4pcauto_baseline + risktolauto_baseline + scz_eurauto_baseline + scz_easauto_baseline + scz_metaauto_baseline + smokerauto_baseline + worryauto_baseline + anxietyauto_baseline + ptsdeur4_baseline + ptsdmeta6_baseline + adhdeur6_baseline + euro_baseline + sex_baseline + race_g_baseline + parent_identity_baseline + demo_brthdat_v2_baseline + demo_sex_v2_baseline + gender_identity_baseline + parent_age_baseline + foreign_born_family_baseline + married_baseline + high_educ_baseline + high_educ2_baseline + income_baseline + foreign_born_baseline + religion_prefer_baseline + gay_parent_baseline + gay_youth_baseline + race_ethnicity_baseline + age_baseline + family_adversity_baseline + height_baseline + weight_baseline + vol_baseline + bmi_baseline + total_ratio_baseline + history_ratio_baseline + euro_2year + race_g_2year + gay_youth_2year + age_2year + height_2year + weight_2year + vol_2year + bmi_2year', data=combined_data).fit()

# Number of parameters in the model (excluding the intercept)
k = len(model_combined.params) - 1

# Chow Test formula
numerator = (model_combined.ssr - (model_low.ssr + model_high.ssr)) / k
denominator = (model_low.ssr + model_high.ssr) / (len(df_low) + len(df_high) - 2 * k)
f_statistic = numerator / denominator
p_value = stats.f.sf(f_statistic, k, len(df_low) + len(df_high) - 2 * k)  # Using survival function for p-value

print("F-statistic:", f_statistic)
print("P-value:", p_value)

coef_low = model_low.params
p_low = model_low.pvalues

# Filter coefficients with p-values less than 0.05
sig_coef_low = coef_low[p_low < 0.05].index

coef_combined_low = model_combined.params[sig_coef_low]

coefficient_differences = abs(coef_combined_low - coef_low[sig_coef_low])

diff_low_df = pd.DataFrame({'Difference': coefficient_differences, 'Coefficients': coef_low[sig_coef_low], 'P-values': p_low[sig_coef_low]}).sort_values(by='Difference', ascending=False)

print(diff_low_df)

coef_high = model_high.params
p_high = model_high.pvalues

# Filter coefficients with p-values less than 0.05
sig_coef_high = coef_high[p_high < 0.05].index

coef_combined_high = model_combined.params[sig_coef_high]

coefficient_differences = abs(coef_combined_high - coef_high[sig_coef_high])

diff_high_df = pd.DataFrame({'Difference': coefficient_differences, 'Coefficients': coef_high[sig_coef_high], 'P-values': p_high[sig_coef_high]}).sort_values(by='Difference', ascending=False)

print(diff_high_df)

In [ ]:
nan_columns_low = low_reading_pos.columns[low_reading_pos.isna().any()].tolist()
print(nan_columns_low)
nan_columns_high = high_reading_pos.columns[high_reading_pos.isna().any()].tolist()
print(nan_columns_high)
# remove prenatal_heroin_before_baseline, prenatal_heroin_after_baseline, prenatal_oxycontin_after_baseline

In [ ]:
model_low = ols('nihtbx_reading_uncorrected_2year ~ KSADS_BD_y_baseline + KSADS_Anx_y_baseline + KSADS_Suicide_y_baseline + KSADS_Sleep_y_baseline + KSADS_total_y_baseline + KSADS_BD_y_2year + KSADS_Anx_y_2year + KSADS_Eat_y_2year + KSADS_Suicide_y_2year + KSADS_Sleep_y_2year + KSADS_total_y_2year + KSADS_BD_p_baseline + KSADS_Anx_p_baseline + KSADS_Eat_p_baseline + KSADS_Suicide_p_baseline + KSADS_Sleep_p_baseline + KSADS_total_p_baseline + KSADS_BD_p_2year + KSADS_Anx_p_2year + KSADS_Eat_p_2year + KSADS_Suicide_p_2year + KSADS_Sleep_p_2year + KSADS_total_p_2year + KSADS_Subst_p_baseline + KSADS_Subst_p_2year + cbcl_internal_baseline + cbcl_internal_2year + cbcl_external_baseline + cbcl_external_2year + cbcl_totprob_baseline + cbcl_totprob_2year + upps_premeditation_baseline + upps_premeditation_2year + upps_perseverence_baseline + upps_perseverence_2year + upps_sensation_baseline + upps_sensation_2year + upps_negative_baseline + upps_negative_2year + upps_positive_baseline + upps_positive_2year + BIS_baseline + BIS_2year + BAS_RR_baseline + BAS_RR_2year + BAS_Drive_baseline + BAS_Drive_2year + BAS_Fun_baseline + BAS_Fun_2year + sleep_disturb_total_baseline + sleep_disturb_total_2year + ELS_total_baseline + ELS_total_2year + rh_adi_wsum1_baseline + rh_adi_wsum1_2year + rh_adi_wsum2_baseline + rh_adi_wsum2_2year + risk_alcohol_p_baseline + risk_cigarette_p_baseline + risk_electro_nicotine_p_baseline + risk_marijuana_p_baseline + risk_drug_p_baseline + risk_med_legal_p_baseline + risk_med_marijuana_p_baseline + risk_med_marijuana_pre_p_baseline + risk_family_med_marijuana_p_baseline + risk_alcohol_p_2year + risk_cigarette_p_2year + risk_electro_nicotine_p_2year + risk_marijuana_p_2year + risk_drug_p_2year + risk_med_legal_p_2year + risk_med_marijuana_p_2year + risk_med_marijuana_pre_p_2year + risk_family_med_marijuana_p_2year + risk_alcohol_y_2year + risk_cigarette_y_2year + risk_electro_nicotine_y_2year + risk_marijuana_y_2year + risk_med_legal_y_2year + risk_med_marijuana_y_2year + risk_med_marijuana_pre_y_2year + risk_family_med_marijuana_y_2year + risk_drug_y_2year + risk_gas_y_2year + risk_pain_y_2year + risk_anix_y_2year + risk_stimulant_y_2year + good_school_baseline + good_parent1_baseline + good_parent2_baseline + good_parent_baseline + good_school_2year + prenatal_tobacco_before_baseline + prenatal_alcohol_max_before_baseline + prenatal_alcohol_avg_before_baseline + prenatal_alcohol_eff_before_baseline + prenatal_marijuana_before_baseline + prenatal_cocaine_before_baseline + prenatal_oxycontin_before_baseline + prenatal_tobacco_after_baseline + prenatal_alcohol_max_after_baseline + prenatal_alcohol_avg_after_baseline + prenatal_alcohol_eff_after_baseline + prenatal_marijuana_after_baseline + prenatal_cocaine_after_baseline + prenatal_weeks_baseline + screentime_wkday_tv_baseline + screentime_wkday_videos_baseline + screentime_wkday_games_baseline + screentime_wkday_texting_baseline + screentime_wkday_sns_baseline + screentime_wkday_videochat_baseline + screentime_wkend_tv_baseline + screentime_wkend_videos_baseline + screentime_wkend_games_baseline + screentime_wkend_texting_baseline + screentime_wkend_sns_baseline + screentime_wkend_videochat_baseline + screentime_wkday_y_baseline + screentime_wkend_y_baseline + screentime_wkday_p_baseline + screentime_wkend_p_baseline + screentime_maturegames_baseline + screentime_rmovies_baseline + screentime_addict_p_2year + screentime_risk_p_2year + screentime_maturegames_2year + screentime_rmovies_2year + bpm_total_y_2year + bpm_total_t_baseline + bpm_total_t_2year + prosocial_y_baseline + prosocial_p_baseline + detention_baseline + detention_rea_baseline + friends_boys_baseline + friends_girls_baseline + Cfriends_boys_baseline + Cfriends_girls_baseline + friends_same_baseline + friends_diff_baseline + Cfriends_diff_baseline + Cfriends_same_baseline + prosocial_y_2year + prosocial_p_2year + friends_boys_2year + friends_girls_2year + Cfriends_boys_2year + Cfriends_girls_2year + friends_same_2year + friends_diff_2year + Cfriends_diff_2year + Cfriends_same_2year + cpeur2_baseline + eaeur1_baseline + depeur4_baseline + mddeur6_baseline + depmulti_baseline + bmieur4_baseline + bmimulti_baseline + insomniaeur6_baseline + snoringeur1_baseline + iqeur2_baseline + happieur4_baseline + ghappieur2_baseline + ghappimeaneur1_baseline + ghappihealth6_baseline + alcdep_eurauto_baseline + alcdep_afrauto_baseline + alcdep_metaauto_baseline + asdauto_baseline + aspauto_baseline + bipauto_baseline + cannabisauto_baseline + crossauto_baseline + drinkauto_baseline + edauto_baseline + neuroticismauto_baseline + ocdauto_baseline + risk4pcauto_baseline + risktolauto_baseline + scz_eurauto_baseline + scz_easauto_baseline + scz_metaauto_baseline + smokerauto_baseline + worryauto_baseline + anxietyauto_baseline + ptsdeur4_baseline + ptsdmeta6_baseline + adhdeur6_baseline + euro_baseline + sex_baseline + race_g_baseline + parent_identity_baseline + demo_brthdat_v2_baseline + demo_sex_v2_baseline + gender_identity_baseline + parent_age_baseline + foreign_born_family_baseline + married_baseline + high_educ_baseline + high_educ2_baseline + income_baseline + foreign_born_baseline + religion_prefer_baseline + gay_parent_baseline + gay_youth_baseline + race_ethnicity_baseline + age_baseline + family_adversity_baseline + height_baseline + weight_baseline + vol_baseline + bmi_baseline + total_ratio_baseline + history_ratio_baseline + euro_2year + race_g_2year + gay_youth_2year + age_2year + height_2year + weight_2year + vol_2year + bmi_2year', data=low_reading_pos).fit()
model_high = ols('nihtbx_reading_uncorrected_2year ~ KSADS_BD_y_baseline + KSADS_Anx_y_baseline + KSADS_Suicide_y_baseline + KSADS_Sleep_y_baseline + KSADS_total_y_baseline + KSADS_BD_y_2year + KSADS_Anx_y_2year + KSADS_Eat_y_2year + KSADS_Suicide_y_2year + KSADS_Sleep_y_2year + KSADS_total_y_2year + KSADS_BD_p_baseline + KSADS_Anx_p_baseline + KSADS_Eat_p_baseline + KSADS_Suicide_p_baseline + KSADS_Sleep_p_baseline + KSADS_total_p_baseline + KSADS_BD_p_2year + KSADS_Anx_p_2year + KSADS_Eat_p_2year + KSADS_Suicide_p_2year + KSADS_Sleep_p_2year + KSADS_total_p_2year + KSADS_Subst_p_baseline + KSADS_Subst_p_2year + cbcl_internal_baseline + cbcl_internal_2year + cbcl_external_baseline + cbcl_external_2year + cbcl_totprob_baseline + cbcl_totprob_2year + upps_premeditation_baseline + upps_premeditation_2year + upps_perseverence_baseline + upps_perseverence_2year + upps_sensation_baseline + upps_sensation_2year + upps_negative_baseline + upps_negative_2year + upps_positive_baseline + upps_positive_2year + BIS_baseline + BIS_2year + BAS_RR_baseline + BAS_RR_2year + BAS_Drive_baseline + BAS_Drive_2year + BAS_Fun_baseline + BAS_Fun_2year + sleep_disturb_total_baseline + sleep_disturb_total_2year + ELS_total_baseline + ELS_total_2year + rh_adi_wsum1_baseline + rh_adi_wsum1_2year + rh_adi_wsum2_baseline + rh_adi_wsum2_2year + risk_alcohol_p_baseline + risk_cigarette_p_baseline + risk_electro_nicotine_p_baseline + risk_marijuana_p_baseline + risk_drug_p_baseline + risk_med_legal_p_baseline + risk_med_marijuana_p_baseline + risk_med_marijuana_pre_p_baseline + risk_family_med_marijuana_p_baseline + risk_alcohol_p_2year + risk_cigarette_p_2year + risk_electro_nicotine_p_2year + risk_marijuana_p_2year + risk_drug_p_2year + risk_med_legal_p_2year + risk_med_marijuana_p_2year + risk_med_marijuana_pre_p_2year + risk_family_med_marijuana_p_2year + risk_alcohol_y_2year + risk_cigarette_y_2year + risk_electro_nicotine_y_2year + risk_marijuana_y_2year + risk_med_legal_y_2year + risk_med_marijuana_y_2year + risk_med_marijuana_pre_y_2year + risk_family_med_marijuana_y_2year + risk_drug_y_2year + risk_gas_y_2year + risk_pain_y_2year + risk_anix_y_2year + risk_stimulant_y_2year + good_school_baseline + good_parent1_baseline + good_parent2_baseline + good_parent_baseline + good_school_2year + prenatal_tobacco_before_baseline + prenatal_alcohol_max_before_baseline + prenatal_alcohol_avg_before_baseline + prenatal_alcohol_eff_before_baseline + prenatal_marijuana_before_baseline + prenatal_cocaine_before_baseline + prenatal_oxycontin_before_baseline + prenatal_tobacco_after_baseline + prenatal_alcohol_max_after_baseline + prenatal_alcohol_avg_after_baseline + prenatal_alcohol_eff_after_baseline + prenatal_marijuana_after_baseline + prenatal_cocaine_after_baseline + prenatal_weeks_baseline + screentime_wkday_tv_baseline + screentime_wkday_videos_baseline + screentime_wkday_games_baseline + screentime_wkday_texting_baseline + screentime_wkday_sns_baseline + screentime_wkday_videochat_baseline + screentime_wkend_tv_baseline + screentime_wkend_videos_baseline + screentime_wkend_games_baseline + screentime_wkend_texting_baseline + screentime_wkend_sns_baseline + screentime_wkend_videochat_baseline + screentime_wkday_y_baseline + screentime_wkend_y_baseline + screentime_wkday_p_baseline + screentime_wkend_p_baseline + screentime_maturegames_baseline + screentime_rmovies_baseline + screentime_addict_p_2year + screentime_risk_p_2year + screentime_maturegames_2year + screentime_rmovies_2year + bpm_total_y_2year + bpm_total_t_baseline + bpm_total_t_2year + prosocial_y_baseline + prosocial_p_baseline + detention_baseline + detention_rea_baseline + friends_boys_baseline + friends_girls_baseline + Cfriends_boys_baseline + Cfriends_girls_baseline + friends_same_baseline + friends_diff_baseline + Cfriends_diff_baseline + Cfriends_same_baseline + prosocial_y_2year + prosocial_p_2year + friends_boys_2year + friends_girls_2year + Cfriends_boys_2year + Cfriends_girls_2year + friends_same_2year + friends_diff_2year + Cfriends_diff_2year + Cfriends_same_2year + cpeur2_baseline + eaeur1_baseline + depeur4_baseline + mddeur6_baseline + depmulti_baseline + bmieur4_baseline + bmimulti_baseline + insomniaeur6_baseline + snoringeur1_baseline + iqeur2_baseline + happieur4_baseline + ghappieur2_baseline + ghappimeaneur1_baseline + ghappihealth6_baseline + alcdep_eurauto_baseline + alcdep_afrauto_baseline + alcdep_metaauto_baseline + asdauto_baseline + aspauto_baseline + bipauto_baseline + cannabisauto_baseline + crossauto_baseline + drinkauto_baseline + edauto_baseline + neuroticismauto_baseline + ocdauto_baseline + risk4pcauto_baseline + risktolauto_baseline + scz_eurauto_baseline + scz_easauto_baseline + scz_metaauto_baseline + smokerauto_baseline + worryauto_baseline + anxietyauto_baseline + ptsdeur4_baseline + ptsdmeta6_baseline + adhdeur6_baseline + euro_baseline + sex_baseline + race_g_baseline + parent_identity_baseline + demo_brthdat_v2_baseline + demo_sex_v2_baseline + gender_identity_baseline + parent_age_baseline + foreign_born_family_baseline + married_baseline + high_educ_baseline + high_educ2_baseline + income_baseline + foreign_born_baseline + religion_prefer_baseline + gay_parent_baseline + gay_youth_baseline + race_ethnicity_baseline + age_baseline + family_adversity_baseline + height_baseline + weight_baseline + vol_baseline + bmi_baseline + total_ratio_baseline + history_ratio_baseline + euro_2year + race_g_2year + gay_youth_2year + age_2year + height_2year + weight_2year + vol_2year + bmi_2year', data=high_reading_pos).fit()

# Combine data for Chow Test
combined_data = pd.concat([low_reading_pos, high_reading_pos])
model_combined = ols('nihtbx_reading_uncorrected_2year ~ KSADS_BD_y_baseline + KSADS_Anx_y_baseline + KSADS_Suicide_y_baseline + KSADS_Sleep_y_baseline + KSADS_total_y_baseline + KSADS_BD_y_2year + KSADS_Anx_y_2year + KSADS_Eat_y_2year + KSADS_Suicide_y_2year + KSADS_Sleep_y_2year + KSADS_total_y_2year + KSADS_BD_p_baseline + KSADS_Anx_p_baseline + KSADS_Eat_p_baseline + KSADS_Suicide_p_baseline + KSADS_Sleep_p_baseline + KSADS_total_p_baseline + KSADS_BD_p_2year + KSADS_Anx_p_2year + KSADS_Eat_p_2year + KSADS_Suicide_p_2year + KSADS_Sleep_p_2year + KSADS_total_p_2year + KSADS_Subst_p_baseline + KSADS_Subst_p_2year + cbcl_internal_baseline + cbcl_internal_2year + cbcl_external_baseline + cbcl_external_2year + cbcl_totprob_baseline + cbcl_totprob_2year + upps_premeditation_baseline + upps_premeditation_2year + upps_perseverence_baseline + upps_perseverence_2year + upps_sensation_baseline + upps_sensation_2year + upps_negative_baseline + upps_negative_2year + upps_positive_baseline + upps_positive_2year + BIS_baseline + BIS_2year + BAS_RR_baseline + BAS_RR_2year + BAS_Drive_baseline + BAS_Drive_2year + BAS_Fun_baseline + BAS_Fun_2year + sleep_disturb_total_baseline + sleep_disturb_total_2year + ELS_total_baseline + ELS_total_2year + rh_adi_wsum1_baseline + rh_adi_wsum1_2year + rh_adi_wsum2_baseline + rh_adi_wsum2_2year + risk_alcohol_p_baseline + risk_cigarette_p_baseline + risk_electro_nicotine_p_baseline + risk_marijuana_p_baseline + risk_drug_p_baseline + risk_med_legal_p_baseline + risk_med_marijuana_p_baseline + risk_med_marijuana_pre_p_baseline + risk_family_med_marijuana_p_baseline + risk_alcohol_p_2year + risk_cigarette_p_2year + risk_electro_nicotine_p_2year + risk_marijuana_p_2year + risk_drug_p_2year + risk_med_legal_p_2year + risk_med_marijuana_p_2year + risk_med_marijuana_pre_p_2year + risk_family_med_marijuana_p_2year + risk_alcohol_y_2year + risk_cigarette_y_2year + risk_electro_nicotine_y_2year + risk_marijuana_y_2year + risk_med_legal_y_2year + risk_med_marijuana_y_2year + risk_med_marijuana_pre_y_2year + risk_family_med_marijuana_y_2year + risk_drug_y_2year + risk_gas_y_2year + risk_pain_y_2year + risk_anix_y_2year + risk_stimulant_y_2year + good_school_baseline + good_parent1_baseline + good_parent2_baseline + good_parent_baseline + good_school_2year + prenatal_tobacco_before_baseline + prenatal_alcohol_max_before_baseline + prenatal_alcohol_avg_before_baseline + prenatal_alcohol_eff_before_baseline + prenatal_marijuana_before_baseline + prenatal_cocaine_before_baseline + prenatal_oxycontin_before_baseline + prenatal_tobacco_after_baseline + prenatal_alcohol_max_after_baseline + prenatal_alcohol_avg_after_baseline + prenatal_alcohol_eff_after_baseline + prenatal_marijuana_after_baseline + prenatal_cocaine_after_baseline + prenatal_weeks_baseline + screentime_wkday_tv_baseline + screentime_wkday_videos_baseline + screentime_wkday_games_baseline + screentime_wkday_texting_baseline + screentime_wkday_sns_baseline + screentime_wkday_videochat_baseline + screentime_wkend_tv_baseline + screentime_wkend_videos_baseline + screentime_wkend_games_baseline + screentime_wkend_texting_baseline + screentime_wkend_sns_baseline + screentime_wkend_videochat_baseline + screentime_wkday_y_baseline + screentime_wkend_y_baseline + screentime_wkday_p_baseline + screentime_wkend_p_baseline + screentime_maturegames_baseline + screentime_rmovies_baseline + screentime_addict_p_2year + screentime_risk_p_2year + screentime_maturegames_2year + screentime_rmovies_2year + bpm_total_y_2year + bpm_total_t_baseline + bpm_total_t_2year + prosocial_y_baseline + prosocial_p_baseline + detention_baseline + detention_rea_baseline + friends_boys_baseline + friends_girls_baseline + Cfriends_boys_baseline + Cfriends_girls_baseline + friends_same_baseline + friends_diff_baseline + Cfriends_diff_baseline + Cfriends_same_baseline + prosocial_y_2year + prosocial_p_2year + friends_boys_2year + friends_girls_2year + Cfriends_boys_2year + Cfriends_girls_2year + friends_same_2year + friends_diff_2year + Cfriends_diff_2year + Cfriends_same_2year + cpeur2_baseline + eaeur1_baseline + depeur4_baseline + mddeur6_baseline + depmulti_baseline + bmieur4_baseline + bmimulti_baseline + insomniaeur6_baseline + snoringeur1_baseline + iqeur2_baseline + happieur4_baseline + ghappieur2_baseline + ghappimeaneur1_baseline + ghappihealth6_baseline + alcdep_eurauto_baseline + alcdep_afrauto_baseline + alcdep_metaauto_baseline + asdauto_baseline + aspauto_baseline + bipauto_baseline + cannabisauto_baseline + crossauto_baseline + drinkauto_baseline + edauto_baseline + neuroticismauto_baseline + ocdauto_baseline + risk4pcauto_baseline + risktolauto_baseline + scz_eurauto_baseline + scz_easauto_baseline + scz_metaauto_baseline + smokerauto_baseline + worryauto_baseline + anxietyauto_baseline + ptsdeur4_baseline + ptsdmeta6_baseline + adhdeur6_baseline + euro_baseline + sex_baseline + race_g_baseline + parent_identity_baseline + demo_brthdat_v2_baseline + demo_sex_v2_baseline + gender_identity_baseline + parent_age_baseline + foreign_born_family_baseline + married_baseline + high_educ_baseline + high_educ2_baseline + income_baseline + foreign_born_baseline + religion_prefer_baseline + gay_parent_baseline + gay_youth_baseline + race_ethnicity_baseline + age_baseline + family_adversity_baseline + height_baseline + weight_baseline + vol_baseline + bmi_baseline + total_ratio_baseline + history_ratio_baseline + euro_2year + race_g_2year + gay_youth_2year + age_2year + height_2year + weight_2year + vol_2year + bmi_2year', data=combined_data).fit()

# Number of parameters in the model (excluding the intercept)
k = len(model_combined.params) - 1

# Chow Test formula
numerator = (model_combined.ssr - (model_low.ssr + model_high.ssr)) / k
denominator = (model_low.ssr + model_high.ssr) / (len(df_low) + len(df_high) - 2 * k)
f_statistic = numerator / denominator
p_value = stats.f.sf(f_statistic, k, len(df_low) + len(df_high) - 2 * k)  # Using survival function for p-value

print("F-statistic:", f_statistic)
print("P-value:", p_value)

coef_low = model_low.params
p_low = model_low.pvalues

# Filter coefficients with p-values less than 0.05
sig_coef_low = coef_low[p_low < 0.05].index

coef_combined_low = model_combined.params[sig_coef_low]

coefficient_differences = abs(coef_combined_low - coef_low[sig_coef_low])

diff_low_df = pd.DataFrame({'Difference': coefficient_differences, 'Coefficients': coef_low[sig_coef_low], 'P-values': p_low[sig_coef_low]}).sort_values(by='Difference', ascending=False)

print(diff_low_df)

coef_high = model_high.params
p_high = model_high.pvalues

# Filter coefficients with p-values less than 0.05
sig_coef_high = coef_high[p_high < 0.05].index

coef_combined_high = model_combined.params[sig_coef_high]

coefficient_differences = abs(coef_combined_high - coef_high[sig_coef_high])

diff_high_df = pd.DataFrame({'Difference': coefficient_differences, 'Coefficients': coef_high[sig_coef_high], 'P-values': p_high[sig_coef_high]}).sort_values(by='Difference', ascending=False)

print(diff_high_df)